## Thank you for attending or viewing my KotlinConf '23 talk, "Replacing SQL with Kotlin's `dataframe` on the Las Vegas Strip"

### This is the Jupyter notebook that I demo'd during the talk. It is a simplified, stripped-down simulation, but it demonstrates the actual process of executing a casino marketing campaign.

### Some of the feedback I received indicated some people were seeking more direct SQL-to-dataframe replacement code. To facilitate this, I've added comments specifying equivalent SQL statements for each dataframe instruction. With that said, the SQL is rather complex, but that's because `dataframe`'s procedural pipelines become more valuable as the equivalent SQL gets more and more difficult.

### If you are reading this, and have further questions, please feel free to ask! Happy to walk you through the code or answer any questions about `dataframe` that I'm able to. Contact me at andy [at] cfnine.com

#### The `%use` directive is the preferred way to import compatible packages in Kotlin-Jupyter, as it also includes dependencies.
#### If it isn't working, however, jars can be imported via `DependsOn`

In [1]:
%use dataframe(0.9.1)
// @file:DependsOn("E:\\Presentations\\dependencies\\dataframe-0.9.1.jar")
// @file:DependsOn("E:\\Presentations\\dependencies\\commons-csv-1.10.0.jar")
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import kotlinx.datetime.LocalDate as KDate
import org.apache.commons.csv.CSVFormat

#### Be sure to edit `csvpath` to match the directory on your system

#### For this demo, we are loading our data from CSV. In reality, we pull in data from database tables with simple SQL `select` statements -- however that's the only SQL we will use

In [2]:
val csvpath : String = "E:\\Presentations\\KotlinConfDataframe\\assets2\\data\\"

val dfCalendars = DataFrame.readCSV("${csvpath}calendars.csv", delimiter = ';')
val dfHotelOffers = DataFrame.readCSV("${csvpath}hoteloffers.csv", delimiter = ';')
val dfTierOffers = DataFrame.readCSV("${csvpath}tierofferssimple.csv", delimiter = ';')
val dfPlayers = DataFrame.readCSV("${csvpath}players.csv", delimiter = ';')
val dfMessages = DataFrame.readCSV("${csvpath}msg.csv", delimiter = ';')
val dfTiers = DataFrame.readCSV("${csvpath}tierdefinitionssimple.csv", delimiter = ';')

In [3]:
/* 
Definition of each player tier, or segment. 

There's a min/max value for the specified masure (in this simple case, we always use ADT, or Average Daily Theoretical loss)
  and a Geographic distinction, either local customers from Las Vegas or California customers. 
It would be expected that in general, the local guests are more frequent, but lower spend per visit, while California guests
  are far less frequent, but tend to spend more per visit.
*/

dfTiers

DataFrame: rowsCount = 16, columnsCount = 6

In [4]:
/* 
Table containing marketing copy. 

The double characters (~~, ##, @@, etc) are variable placeholders which will be filled with specific values later

Note that each row contains a TierMin/TierMax columns... 
  we will need to match up each of these with inidividual tiers defined above
*/

dfMessages

DataFrame: rowsCount = 8, columnsCount = 6

```sql
select 
    t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
from 
    tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
```

In [5]:
// dfMessages provides a TierMin & TierMax
// with explode(), we can create a row for every individual tier in the range
// then we can innerJoin with dfTiers, now every Message is matched with each of the corresponding tiers
val dfMsgTiers = dfMessages
    .remove { Category }
    .add("Tier") { (TierMin..TierMax).toList() }
    .explode("Tier")
    .innerJoin(dfTiers) { "Tier"<Int>() match(right.Tier) }
    .remove { "OfferMonth1" and TierMin and TierMax }
    .move("Tier").after { OfferMonth }

```sql
select 
    t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
from 
    tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
where
    t.Tier = 2003
order by
    m.Placement, t.Tier
```

In [6]:
/*
This cell provides a simple view of all of the messages for a single tier

It also demonstrates how the Kotlin kernel re-evaluates all active variables including dataframes upon every cell execution

Where "Tier" was not a known variable in the cell above, requiring quotes, because it was not previously a part of dfMessages,
  just one cell later, it is known and part of code completion
*/

dfMsgTiers
    .filter { Tier == 2003 }
    .sortBy { Placement and Tier }

DataFrame: rowsCount = 5, columnsCount = 8

#### note how `pivot` works in T-SQL (SQL Server), all the potential values for `Placement` must be hard-coded into the query, twice

#### this makes this an awkward, non-dynamic query that must be updated by hand when new data is introduced

#### we do not have this restriction with `dataframe`, further, the `pivot` syntax is simpler
```sql
select 
    OfferMonth, Tier, 
    GamingSubhead, DiningSubhead, OutsideHeadline, GamingHeadline, DiningHeadline
from (
  select 
    t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
  from 
    tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
) as tblMsgTiers

pivot (
  max(Message)
  for Placement in ([GamingSubhead], [DiningSubhead], [OutsideHeadline], [GamingHeadline], [DiningHeadline])
) as pvt

```

In [7]:
val dfMsgPivot = dfMsgTiers
    .groupBy { OfferMonth and Tier }
        .pivot { Placement }
            .with { Message }
    .flatten()
    
dfMsgPivot

DataFrame: rowsCount = 16, columnsCount = 7

In [8]:
dfTierOffers

DataFrame: rowsCount = 16, columnsCount = 10

#### `STRING_SPLIT` is similar to `dataframe`'s `explode` but is proprietary to SQL Server and only available 2016+ (surprisingly, many SQL Server installations are still running older versions). Older versions, and other RDBMS, would likely require a temporary table to be `CROSS JOIN`ed with `tblTierOffers`
```sql
/*
create table #GamePrefs (
    GamePref varchar(40)
)

insert into #GamePrefs 
    (GamePref)
values 
    ('Slot'),
    ('Table')
*/

select 
    OfferMonth, Tier, 
    value as GamePref,
    case when value = 'Slot' then SlotAmount else TableAmount end as GamingAmount,
    case when value = 'Slot' then 'SlotCredits' else TableType end as GamingType,
    GamingFrequency, DiningAmount, DiningFrequency, HotelCode, HotelFrequency
from
    tblTierOffers
cross apply 
    STRING_SPLIT('Slot;Table', ';')
```

In [9]:
val GamePref by column<String>()
val GamingAmount by column<Int>()

val dfTierOffersByGame = dfTierOffers
    .add(GamePref) { listOf("Slot", "Table") }
    .explode { GamePref }
    .add(GamingAmount) { if (GamePref() == "Slot") SlotAmount else TableAmount }
    .update { TableType }.with { if (GamePref() == "Slot") "Slot Credits" else it }
    .rename { TableType }.into("GamingType")
    .move { GamePref and GamingAmount }.after { Tier }
    .remove { SlotAmount and TableAmount }
    
dfTierOffersByGame

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 10

#### beyond here, instead of attempting to write an overly complex query with multiple sub-queries, we use CTEs to break up our SQL

#### we also see the awkwardness of relying on SQL for string replacement and formatting. it's wordy in Kotlin but at least we can chain the `replace` statements rather than having to nest them `replace(replace(replace(...)))` etc.

```sql
with vMsgPivot as (
    select 
        OfferMonth, Tier, 
        GamingSubhead, DiningSubhead, OutsideHeadline, GamingHeadline, DiningHeadline
    from (
      select 
        t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
      from 
        tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
    ) as tblMsgTiers

    pivot (
      max(Message)
      for Placement in ([GamingSubhead], [DiningSubhead], [OutsideHeadline], [GamingHeadline], [DiningHeadline])
    ) as pvt
),

vTierOffersbyGame as (
    select 
        OfferMonth, Tier, 
        value as GamePref,
        case when value = 'Slot' then SlotAmount else TableAmount end as GamingAmount,
        case when value = 'Slot' then 'SlotCredits' else TableType end as GamingType,
        GamingFrequency, DiningAmount, DiningFrequency, HotelCode, HotelFrequency
    from
        tblTierOffers
    cross apply 
        STRING_SPLIT('Slot;Table', ';')
)

select
    t.OfferMonth, t.Tier, t.GamePref,
    replace(
      replace(m.OutsideHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
      '^^', format(t.GamingAmount * t.GamingFrequency + t.DiningAmount * t.DiningFrequency, 'C0')
    ) as OutsideHeadline,
    h.HotelHeadline,
    h.HotelDescription,
    t.HotelCode,
    
    replace(
      replace(
        replace(
          replace(m.GamingHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '**', t.GamingType
        ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
      ), '~~', format(t.GamingAmount, 'C0')
    ) as GamingHeadline,
    
    replace(
      replace(
        replace(
          replace(m.GamingSubhead, '##', FORMAT(t.OfferMonth, 'MMMM')),
          '**', t.GamingType
        ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
      ), '~~', format(t.GamingAmount, 'C0')
    ) as GamingSubhead,
    
    replace(
      replace(
        replace(m.DiningHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
        '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
      ), '~~', format(t.DiningAmount, 'C0')
    ) as DiningHeadline,

    replace(
      replace(
        replace(m.DiningSubhead, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
        '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
      ), '~~', format(t.DiningAmount, 'C0')
    ) as DiningSubhead
    
    
from
    vTierOffersbyGame t 
      left join vMsgPivot m on t.OfferMonth = m.OfferMonth and t.Tier = m.Tier
      left join tblHotelOffers h on t.OfferMonth = h.OfferMonth and t.HotelCode = h.HotelCode
```

In [10]:
val dfTieredOffersPlusHotel = dfTierOffersByGame
    .leftJoin(dfMsgPivot) { OfferMonth and Tier }
    .leftJoin(dfHotelOffers) { OfferMonth and HotelCode }
    .update("GamingHeadline", "GamingSubhead").with { 
        it.toString()
            .replace("~~", GamingAmount.let{ g -> "\$${"%,d".format(g)}" })
            .replace("@@", GamingAmount.times(GamingFrequency).let{ g ->"\$${"%,d".format(g)}" })
            .replace("**", GamingType)
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")))
    }.update("DiningHeadline", "DiningSubhead").with { 
        it.toString()
            .replace("~~", DiningAmount.let {d -> "\$${"%,d".format(d)}" })
            .replace("@@", DiningAmount.times(DiningFrequency).let{ d ->"\$${"%,d".format(d)}" })
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")))
    }.update("OutsideHeadline").with { 
        it.toString()
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")).uppercase())
            .replace("^^", (GamingAmount.times(GamingFrequency)).plus(DiningAmount.times(DiningFrequency)).let{ t ->"\$${"%,d".format(t)}" })
    }
    .move("GamePref", "OutsideHeadline", "HotelHeadline", "HotelDescription", "HotelCode", 
             "GamingHeadline", "GamingSubhead", "DiningHeadline", "DiningSubhead").after { Tier }

    
dfTieredOffersPlusHotel

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 17

In [11]:
dfTierOffersByGame.describe()

DataFrame: rowsCount = 10, columnsCount = 12

In [12]:
dfCalendars

DataFrame: rowsCount = 18, columnsCount = 5

In [13]:
fun LocalDate.showAs(pattern: String) : String = this.format(DateTimeFormatter.ofPattern(pattern))
fun LocalDate.showAsOrAbbr(pattern: String, maximumMonthLetters: Int) : String =
    (if (this.showAs("MMMM").length > maximumMonthLetters) pattern.replace("MMMM", "MMM.") else pattern).let { this.showAs(it) }
fun areSameMonth(firstDate: LocalDate, secondDate: LocalDate) : Boolean = (firstDate.month == secondDate.month && firstDate.year == secondDate.year)

fun createReadableDateRange(startDate: LocalDate, endDate: LocalDate, maximumMonthLetters: Int = 12) : String =
    when {
        areSameMonth(startDate, endDate) -> listOf<String>(
                                                startDate.showAsOrAbbr("MMMM d", maximumMonthLetters),
                                                endDate.showAs("d, yyyy")
                                            ).joinToString("-")
        else -> "${startDate.showAs("MMM. d")} to ${endDate.showAs("MMM. d, yyyy")}"
    }


data class OfferCalendar(
    val OfferMonth: LocalDate,
    val CalendarName: String,
    val Period: Int,
    val StartDate: LocalDate,
    val EndDate: LocalDate
)

fun OfferCalendar.readableRange(maxMonthLetters: Int) = createReadableDateRange(StartDate, EndDate, maxMonthLetters)
fun OfferCalendar.types() = when {
    CalendarName.contains("All") -> listOf("gaming", "dining", "hotel")
    CalendarName.contains("Hotel") -> listOf("hotel")
    CalendarName.contains("Gaming") -> listOf("gaming")
    CalendarName.contains("Dining") -> listOf("dining")
    else -> listOf()
}

In [14]:
val calendarList : List<OfferCalendar> = dfCalendars.toListOf<OfferCalendar>()
calendarList

[OfferCalendar(OfferMonth=2023-06-01, CalendarName=HotelCalendar, Period=1, StartDate=2023-06-01, EndDate=2023-06-15), OfferCalendar(OfferMonth=2023-06-01, CalendarName=HotelCalendar, Period=2, StartDate=2023-06-16, EndDate=2023-06-30), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=1, StartDate=2023-06-01, EndDate=2023-06-03), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=2, StartDate=2023-06-04, EndDate=2023-06-06), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=3, StartDate=2023-06-07, EndDate=2023-06-09), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=4, StartDate=2023-06-10, EndDate=2023-06-12), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=5, StartDate=2023-06-13, EndDate=2023-06-15), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=6, StartDate=2023-06-16, EndDate=2023-06-18), OfferCalendar(OfferMonth=2023-06-01, CalendarName

#### we greatly benefit from `dataframe`'s ability to hold arrays within a cell, which a few databases allow (Postgres) but many don't (SQL Server), as well as its ability to hold Kotlin class instances, from which we can call methods. Once again with SQL we are stuck with very data-specific `pivot` as opposed to `dataframe`'s `split` which will work regardless of the data contained in an array.
```sql
with vMsgPivot as (
    select 
        OfferMonth, Tier, Geo,
        GamingSubhead, DiningSubhead, OutsideHeadline, GamingHeadline, DiningHeadline
    from (
      select 
        t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
      from 
        tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
    ) as tblMsgTiers

    pivot (
      max(Message)
      for Placement in ([GamingSubhead], [DiningSubhead], [OutsideHeadline], [GamingHeadline], [DiningHeadline])
    ) as pvt
),

vTierOffersbyGame as (
    select 
        OfferMonth, Tier,
        value as GamePref,
        case when value = 'Slot' then SlotAmount else TableAmount end as GamingAmount,
        case when value = 'Slot' then 'SlotCredits' else TableType end as GamingType,
        GamingFrequency, DiningAmount, DiningFrequency, HotelCode, HotelFrequency
    from
        tblTierOffers
    cross apply 
        STRING_SPLIT('Slot;Table', ';')
),

vTieredOffersPlusHotel as (
  select
      t.OfferMonth, t.Tier, m.Geo, t.GamePref,
      replace(
        replace(m.OutsideHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
        '^^', format(t.GamingAmount * t.GamingFrequency + t.DiningAmount * t.DiningFrequency, 'C0')
      ) as OutsideHeadline,
      h.HotelHeadline,
      h.HotelDescription,
      t.HotelCode,

      replace(
        replace(
          replace(
            replace(m.GamingHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingHeadline,

      replace(
        replace(
          replace(
            replace(m.GamingSubhead, '##', FORMAT(t.OfferMonth, 'MMMM')),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingSubhead,

      replace(
        replace(
          replace(m.DiningHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningHeadline,

      replace(
        replace(
          replace(m.DiningSubhead, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningSubhead


  from
      vTierOffersbyGame t 
        left join vMsgPivot m on t.OfferMonth = m.OfferMonth and t.Tier = m.Tier
        left join tblHotelOffers h on t.OfferMonth = h.OfferMonth and t.HotelCode = h.HotelCode
),


vCalendars as (
  select 
    OfferMonth, Geo,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [HotelCalendar1] end as HotelCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [HotelCalendar2] end as HotelCalendar2,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [GamingCalendar1] end as GamingCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [GamingCalendar2] end as GamingCalendar2,
    [GamingCalendar3], [GamingCalendar4], 
    [GamingCalendar5], [GamingCalendar6],
    [GamingCalendar7], [GamingCalendar8], 
    [GamingCalendar9], [GamingCalendar10],
    case when Geo = 'California' then [CalGeoAllCalendar1] else [DiningCalendar1] end as DiningCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [DiningCalendar2] end as DiningCalendar2,
    [DiningCalendar3], [DiningCalendar4]
    
  from (
    select 
      OfferMonth, 
      concat(CalendarName, Period) as CalName,
      case when 
        datepart(m, StartDate) = datepart(m, EndDate) and datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + '-' + format(EndDate, 'd, yyyy')
      when 
        datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + ' to ' + format(EndDate, 'MMM. d, yyyy')
      else
        format(StartDate, 'MMM. d, yyyy') + ' to ' + format(EndDate, 'MMM. d, yyyy') end 
      as DateRange,
      case when left(CalendarName, 3) = 'Cal' then 'California' else 'LasVegas' end as Geo
    from
      tblCalendars
  ) as cc
  pivot (
    max(DateRange)
    for CalName in (
      [HotelCalendar1], [HotelCalendar2],
      [GamingCalendar1], [GamingCalendar2], [GamingCalendar3], [GamingCalendar4], 
      [GamingCalendar5], [GamingCalendar6], [GamingCalendar7], [GamingCalendar8], 
      [GamingCalendar9], [GamingCalendar10], 
      [DiningCalendar1], [DiningCalendar2], [DiningCalendar3], [DiningCalendar4],
      [CalGeoAllCalendar1], [CalGeoAllCalendar2]
    )
  ) as pvt
  
)



select 
  t.OfferMonth, t.Tier, t.GamePref,
  t.OutsideHeadline, t.HotelHeadline, t.HotelDescription, t.HotelCode, 
  t.GamingHeadline, t.GamingSubhead, 
  t.DiningHeadline, t.DiningSubhead, 
  coalesce(c.HotelCalendar1, '') as HotelCalendar1, coalesce(c.HotelCalendar2, '') as HotelCalendar2, 
  coalesce(c.GamingCalendar1, '') as GamingCalendar1, coalesce(c.GamingCalendar2, '') as GamingCalendar2, 
  coalesce(c.GamingCalendar3, '') as GamingCalendar3, coalesce(c.GamingCalendar4, '') as GamingCalendar4, 
  coalesce(c.GamingCalendar5, '') as GamingCalendar5, coalesce(c.GamingCalendar6, '') as GamingCalendar6, 
  coalesce(c.GamingCalendar7, '') as GamingCalendar7, coalesce(c.GamingCalendar8, '') as GamingCalendar8, 
  coalesce(c.GamingCalendar9, '') as GamingCalendar9, coalesce(c.GamingCalendar10, '') as GamingCalendar10, 
  coalesce(c.DiningCalendar1, '') as DiningCalendar1, coalesce(c.DiningCalendar2, '') as DiningCalendar2, 
  coalesce(c.DiningCalendar3, '') as DiningCalendar3, coalesce(c.DiningCalendar4, '') as DiningCalendar4
  
from 
  vTieredOffersPlusHotel t
    left join vCalendars c on t.OfferMonth = c.OfferMonth and t.Geo = c.Geo

```

In [15]:
val ValidDates by column<List<OfferCalendar>>()
val ValidDateStrings by column<List<String>>()

val dfTieredOffersAndDates = dfTieredOffersPlusHotel
    // we got to here in an earlier cell
    // now we're going to bring in OfferCalendar objects, and assign a list of them (based on geo) to each tier
    .add(ValidDates) { dfCalendars.filter { when {
                                                    Tier <= 1999 -> !CalendarName.contains("Geo")
                                                    Tier >= 2000 -> CalendarName.startsWith("CalGeo")
                                                    else -> true
                                                } }.toListOf<OfferCalendar>() }
    // filter each specific type of OfferCalendar objects into their own columns (each contains a list)
    .add {
        "HotelOffers" from ValidDates.map { it.filter { it.types().contains("hotel") } }
        "GamingOffers" from ValidDates.map { it.filter { it.types().contains("gaming") } }
        "DiningOffers" from ValidDates.map { it.filter { it.types().contains("dining") } }
    }
    // split the lists so each element is in its own column
    .split { "HotelOffers"<List<OfferCalendar>>() }.into { it -> "hotel$it" }
    .split { "GamingOffers"<List<OfferCalendar>>() }.into { it -> "gaming$it" }
    .split { "DiningOffers"<List<OfferCalendar>>() }.into { it -> "dining$it" }
    .remove { ValidDates }
    // now convert the OfferCalendar objects to the string representation of date range
    // don't forget the ? - some columns contain null values because lists were not all same size
    .convert { colsOf<OfferCalendar?>() }.with { it?.readableRange(5) ?: "" }
    
    
dfTieredOffersAndDates

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 33

```sql
with vMsgPivot as (
    select 
        OfferMonth, Tier, Geo,
        GamingSubhead, DiningSubhead, OutsideHeadline, GamingHeadline, DiningHeadline
    from (
      select 
        t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
      from 
        tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
    ) as tblMsgTiers

    pivot (
      max(Message)
      for Placement in ([GamingSubhead], [DiningSubhead], [OutsideHeadline], [GamingHeadline], [DiningHeadline])
    ) as pvt
),

vTierOffersbyGame as (
    select 
        OfferMonth, Tier,
        value as GamePref,
        case when value = 'Slot' then SlotAmount else TableAmount end as GamingAmount,
        case when value = 'Slot' then 'SlotCredits' else TableType end as GamingType,
        GamingFrequency, DiningAmount, DiningFrequency, HotelCode, HotelFrequency
    from
        tblTierOffers
    cross apply 
        STRING_SPLIT('Slot;Table', ';')
),

vTieredOffersPlusHotel as (
  select
      t.OfferMonth, t.Tier, m.Geo, t.GamePref,
      replace(
        replace(m.OutsideHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
        '^^', format(t.GamingAmount * t.GamingFrequency + t.DiningAmount * t.DiningFrequency, 'C0')
      ) as OutsideHeadline,
      h.HotelHeadline,
      h.HotelDescription,
      t.HotelCode,

      replace(
        replace(
          replace(
            replace(m.GamingHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingHeadline,

      replace(
        replace(
          replace(
            replace(m.GamingSubhead, '##', FORMAT(t.OfferMonth, 'MMMM')),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingSubhead,

      replace(
        replace(
          replace(m.DiningHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningHeadline,

      replace(
        replace(
          replace(m.DiningSubhead, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningSubhead


  from
      vTierOffersbyGame t 
        left join vMsgPivot m on t.OfferMonth = m.OfferMonth and t.Tier = m.Tier
        left join tblHotelOffers h on t.OfferMonth = h.OfferMonth and t.HotelCode = h.HotelCode
),


vCalendars as (
  select 
    OfferMonth, Geo,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [HotelCalendar1] end as HotelCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [HotelCalendar2] end as HotelCalendar2,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [GamingCalendar1] end as GamingCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [GamingCalendar2] end as GamingCalendar2,
    [GamingCalendar3], [GamingCalendar4], 
    [GamingCalendar5], [GamingCalendar6],
    [GamingCalendar7], [GamingCalendar8], 
    [GamingCalendar9], [GamingCalendar10],
    case when Geo = 'California' then [CalGeoAllCalendar1] else [DiningCalendar1] end as DiningCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [DiningCalendar2] end as DiningCalendar2,
    [DiningCalendar3], [DiningCalendar4]
    
  from (
    select 
      OfferMonth, 
      concat(CalendarName, Period) as CalName,
      case when 
        datepart(m, StartDate) = datepart(m, EndDate) and datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + '-' + format(EndDate, 'd, yyyy')
      when 
        datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + ' to ' + format(EndDate, 'MMM. d, yyyy')
      else
        format(StartDate, 'MMM. d, yyyy') + ' to ' + format(EndDate, 'MMM. d, yyyy') end 
      as DateRange,
      case when left(CalendarName, 3) = 'Cal' then 'California' else 'LasVegas' end as Geo
    from
      tblCalendars
  ) as cc
  pivot (
    max(DateRange)
    for CalName in (
      [HotelCalendar1], [HotelCalendar2],
      [GamingCalendar1], [GamingCalendar2], [GamingCalendar3], [GamingCalendar4], 
      [GamingCalendar5], [GamingCalendar6], [GamingCalendar7], [GamingCalendar8], 
      [GamingCalendar9], [GamingCalendar10], 
      [DiningCalendar1], [DiningCalendar2], [DiningCalendar3], [DiningCalendar4],
      [CalGeoAllCalendar1], [CalGeoAllCalendar2]
    )
  ) as pvt
  
),

vAllOfferData as (
  select 
    t.OfferMonth, t.Tier, t.GamePref,
    t.OutsideHeadline, t.HotelHeadline, t.HotelDescription, t.HotelCode, 
    t.GamingHeadline, t.GamingSubhead, 
    t.DiningHeadline, t.DiningSubhead, 
    coalesce(c.HotelCalendar1, '') as HotelCalendar1, coalesce(c.HotelCalendar2, '') as HotelCalendar2, 
    coalesce(c.GamingCalendar1, '') as GamingCalendar1, coalesce(c.GamingCalendar2, '') as GamingCalendar2, 
    coalesce(c.GamingCalendar3, '') as GamingCalendar3, coalesce(c.GamingCalendar4, '') as GamingCalendar4, 
    coalesce(c.GamingCalendar5, '') as GamingCalendar5, coalesce(c.GamingCalendar6, '') as GamingCalendar6, 
    coalesce(c.GamingCalendar7, '') as GamingCalendar7, coalesce(c.GamingCalendar8, '') as GamingCalendar8, 
    coalesce(c.GamingCalendar9, '') as GamingCalendar9, coalesce(c.GamingCalendar10, '') as GamingCalendar10, 
    coalesce(c.DiningCalendar1, '') as DiningCalendar1, coalesce(c.DiningCalendar2, '') as DiningCalendar2, 
    coalesce(c.DiningCalendar3, '') as DiningCalendar3, coalesce(c.DiningCalendar4, '') as DiningCalendar4

  from 
    vTieredOffersPlusHotel t
      left join vCalendars c on t.OfferMonth = c.OfferMonth and t.Geo = c.Geo
)

select 
    p.OfferMonth,
    p.Tier,
    p.GamePref, 
    p.PlayerID, 
    p.FirstName, p.LastName, 
    p.Address, p.City, p.State, p.Country, p.ZipCode, 
    lower(rtrim(ltrim(p.Email))) as Email, 
    case when datepart(m, p.Birthday) = datepart(m, p.OfferMonth) then 'true' else 'false' end as Birthday,
    v.OutsideHeadline, v.HotelHeadline, v.HotelDescription, v.HotelCode, 
    v.GamingHeadline, v.GamingSubhead, 
    v.DiningHeadline, v.DiningSubhead, 
    v.HotelCalendar1, v.HotelCalendar2, 
    v.GamingCalendar1, v.GamingCalendar2, 
    v.GamingCalendar3, v.GamingCalendar4, 
    v.GamingCalendar5, v.GamingCalendar6, 
    v.GamingCalendar7, v.GamingCalendar8, 
    v.GamingCalendar9, v.GamingCalendar10, 
    v.DiningCalendar1, v.DiningCalendar2, 
    v.DiningCalendar3, v.DiningCalendar4
from 
    tblPlayers p inner join 
        vAllOfferData aod on p.OfferMonth = aod.OfferMonth and p.Tier = aod.Tier and p.GamePref = aod.GamePref
where
    p.FilterCode = 0
    and p.OKToMail = 'Y'
order by
    p.Tier, p.GamePref, p.PlayerID
```

In [17]:
val dfPlayerOffers = dfPlayers
    .filter { FilterCode == 0 && OKToMail == "Y" }
    .sortBy { Tier and GamePref and PlayerID } 
    .update { Email }.with { it?.lowercase()?.trim() ?: "" }
    .remove { EvaluationDate and FilterCode and OKToMail and Geo and ADT and Measure }
    .innerJoin(dfTieredOffersAndDates) { OfferMonth and Tier and GamePref }
    .move { Tier and GamePref  }.after { OfferMonth }
    .convert { Birthday }.with { Birthday.month == OfferMonth.month }
    .remove("OfferMonth1")
    
dfPlayerOffers

... showing only top 20 of 761 rows DataFrame: rowsCount = 761, columnsCount = 43

#### SQL itself has no inherent ability to export data or write to CSV. Many SQL IDEs do, but this typically requires manual execution of the query(ies) and proprietary non-SQL code to perform the export. `dataframe` on the other hand, has `writeCSV` built-in.

In [18]:
dfPlayerOffers
    .remove { Email and Birthday and 
                GamingAmount and GamingFrequency and DiningAmount and DiningFrequency and HotelFrequency }
    .writeCSV("MailFile.csv", CSVFormat.DEFAULT.withDelimiter(';'))
    
dfPlayerOffers
    .filter { Email.contains("@") }
    .remove { Address and City and State and Country and ZipCode and 
                GamingAmount and GamingFrequency and DiningAmount and DiningFrequency and HotelFrequency }
    .writeCSV("EmailFile.csv", CSVFormat.DEFAULT.withDelimiter(';'))    

In [ ]:
dfPlayerOffers.size()

### http://sqlfiddle.com/#!18/e2e99/1/0

```sql
CREATE TABLE tblTiers (
    [OfferMonth] date, 
    [Tier] int, 
    [MinValue] float, 
    [MaxValue] float, 
    [Measure] varchar(40), 
    [Geo] varchar(40)
)
;
    
INSERT INTO tblTiers
    ([OfferMonth], [Tier], [MinValue], [MaxValue], [Measure], [Geo])
VALUES
    ('2023-06-01', 1001, 1250.0, 9999999.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1002, 750.0, 1250.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1003, 400.0, 750.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1004, 200.0, 400.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1005, 125.0, 200.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1006, 80.0, 125.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1007, 50.0, 80.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1008, 25.0, 50.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 2001, 4000.0, 9999999.0, 'ADT', 'California'),
    ('2023-06-01', 2002, 2000.0, 4000.0, 'ADT', 'California'),
    ('2023-06-01', 2003, 800.0, 2000.0, 'ADT', 'California'),
    ('2023-06-01', 2004, 400.0, 800.0, 'ADT', 'California'),
    ('2023-06-01', 2005, 250.0, 400.0, 'ADT', 'California'),
    ('2023-06-01', 2006, 150.0, 250.0, 'ADT', 'California'),
    ('2023-06-01', 2007, 75.0, 150.0, 'ADT', 'California'),
    ('2023-06-01', 2008, 25.0, 75.0, 'ADT', 'California')
;


CREATE TABLE tblMessages (
    [OfferMonth] date, 
    [Category] varchar(40), 
    [Placement] varchar(40),
    [TierMin] int, 
    [TierMax] int, 
    [Message] varchar(200)
)
;
    
INSERT INTO tblMessages
    ([OfferMonth], [Category], [Placement], [TierMin], [TierMax], [Message])
VALUES
    ('2023-06-01', 'gaming', 'GamingSubhead', 2001, 2199, 'Start with ~~ to play your favorite games, twice in ##!'),
    ('2023-06-01', 'dining', 'DiningSubhead', 2001, 2199, 'Enjoy ~~ per visit at our 9 fantastic restaurants!'),
    ('2023-06-01', 'totals', 'OutsideHeadline', 1001, 1199, '10 ## OFFERS WORTH ^^ INSIDE!'),
    ('2023-06-01', 'totals', 'OutsideHeadline', 2001, 2199, 'GET UP TO ^^ WHEN YOU VISIT US IN ##!'),
    ('2023-06-01', 'gaming', 'GamingSubhead', 1001, 1199, 'Visit up to 10 times in ## and start with ~~.'),
    ('2023-06-01', 'dining', 'DiningSubhead', 1001, 1199, '~~ on us, available once per week.'),
    ('2023-06-01', 'gaming', 'GamingHeadline', 1001, 9999, '@@ in **'),
    ('2023-06-01', 'dining', 'DiningHeadline', 1001, 9999, '@@ Complimentary Dining')
;


CREATE TABLE tblTierOffers (
    [OfferMonth] date, 
    [Tier] int, 
    [SlotAmount] int, 
    [TableAmount] int, 
    [TableType] varchar(40), 
    [GamingFrequency] int, 
    [DiningAmount] int, 
    [DiningFrequency] int, 
    [HotelCode] varchar(20), 
    [HotelFrequency] int
)
;
    
INSERT INTO tblTierOffers
    ([OfferMonth], [Tier], [SlotAmount], [TableAmount], [TableType], [GamingFrequency], [DiningAmount], [DiningFrequency], [HotelCode], [HotelFrequency])
VALUES
    ('2023-06-01', 1001, 400, 300, 'Bonus Chips', 10, 100, 4, 'LV6231', 2),
    ('2023-06-01', 1002, 200, 150, 'Bonus Chips', 10, 60, 4, 'LV6231', 2),
    ('2023-06-01', 1003, 100, 75, 'Bonus Chips', 10, 40, 4, 'LV6232', 2),
    ('2023-06-01', 1004, 50, 45, 'Bonus Chips', 10, 30, 4, 'LV6232', 2),
    ('2023-06-01', 1005, 40, 30, 'Bonus Chips', 10, 20, 4, 'LV6233', 2),
    ('2023-06-01', 1006, 30, 30, 'Matched Bet', 10, 15, 4, 'LV6233', 2),
    ('2023-06-01', 1007, 20, 20, 'Matched Bet', 10, 10, 4, 'LV6234', 2),
    ('2023-06-01', 1008, 15, 10, 'Matched Bet', 10, 7, 4, 'LV6234', 2),
    ('2023-06-01', 2001, 1200, 1600, 'Bonus Chips', 2, 200, 2, 'CA6231', 2),
    ('2023-06-01', 2002, 600, 750, 'Bonus Chips', 2, 150, 2, 'CA6231', 2),
    ('2023-06-01', 2003, 300, 400, 'Bonus Chips', 2, 75, 2, 'CA6232', 2),
    ('2023-06-01', 2004, 150, 200, 'Bonus Chips', 2, 40, 2, 'CA6232', 2),
    ('2023-06-01', 2005, 100, 125, 'Bonus Chips', 2, 30, 2, 'CA6233', 2),
    ('2023-06-01', 2006, 60, 75, 'Bonus Chips', 2, 20, 2, 'CA6233', 2),
    ('2023-06-01', 2007, 30, 35, 'Matched Bet', 2, 15, 2, 'CA6234', 2),
    ('2023-06-01', 2008, 10, 20, 'Matched Bet', 2, 10, 2, 'CA6234', 2)
;



CREATE TABLE tblHotelOffers (
    [OfferMonth] date, 
    [HotelCode] varchar(20), 
    [HotelHeadline] varchar(200), 
    [HotelDescription] varchar(400)
)
;
    
INSERT INTO tblHotelOffers
    ([OfferMonth], [HotelCode], [HotelHeadline], [HotelDescription])
VALUES
    ('2023-06-01', 'LV6231', '2 NIGHT STAY-CATION', 'Stay with us for two nights in a gorgeous suite without any resort fees'),
    ('2023-06-01', 'LV6232', '2 NIGHT STAY-CATION', 'Stay with us for two nights in a high-floor room without any resort fees'),
    ('2023-06-01', 'LV6233', '1 NIGHT STAY-CATION', 'Play here and stay here in a beautiful room'),
    ('2023-06-01', 'LV6234', 'RATES STARTING AT $39', 'Call soon for special casino rates as low as $39 weekdays and $89 Friday & Saturday'),
    ('2023-06-01', 'CA6231', '3 NIGHTS IN A SUITE', 'Be our guest for 3 nights in a gorgeous, multi-room suite without resort fees'),
    ('2023-06-01', 'CA6232', '3 NIGHTS + UPGRADE', 'Be our guest for 3 nights in a gorgeous, high-floor room without resort fees'),
    ('2023-06-01', 'CA6233', '2 COMPED NIGHTS', 'Be our guest for 2 nights with your choice of King bed or 2 doubles'),
    ('2023-06-01', 'CA6234', 'RATES STARTING AT $39', 'Call soon for special casino rates as low as $39 weekdays and $89 Friday & Saturday')
;




CREATE TABLE tblCalendars (
    [OfferMonth] date,
    [Geo] varchar(40),
    [CalendarName] varchar(40), 
    [Period] int, 
    [StartDate] date, 
    [EndDate] date
)
;
    
INSERT INTO tblCalendars
    ([OfferMonth], [CalendarName], [Period], [StartDate], [EndDate])
VALUES
    ('2023-06-01', 'HotelCalendar', 1, '2023-06-01', '2023-06-15'),
    ('2023-06-01', 'HotelCalendar', 2, '2023-06-16', '2023-06-30'),
    ('2023-06-01', 'GamingCalendar', 1, '2023-06-01', '2023-06-03'),
    ('2023-06-01', 'GamingCalendar', 2, '2023-06-04', '2023-06-06'),
    ('2023-06-01', 'GamingCalendar', 3, '2023-06-07', '2023-06-09'),
    ('2023-06-01', 'GamingCalendar', 4, '2023-06-10', '2023-06-12'),
    ('2023-06-01', 'GamingCalendar', 5, '2023-06-13', '2023-06-15'),
    ('2023-06-01', 'GamingCalendar', 6, '2023-06-16', '2023-06-18'),
    ('2023-06-01', 'GamingCalendar', 7, '2023-06-19', '2023-06-21'),
    ('2023-06-01', 'GamingCalendar', 8, '2023-06-22', '2023-06-24'),
    ('2023-06-01', 'GamingCalendar', 9, '2023-06-25', '2023-06-27'),
    ('2023-06-01', 'GamingCalendar', 10, '2023-06-28', '2023-06-30'),
    ('2023-06-01', 'DiningCalendar', 1, '2023-06-01', '2023-06-08'),
    ('2023-06-01', 'DiningCalendar', 2, '2023-06-09', '2023-06-15'),
    ('2023-06-01', 'DiningCalendar', 3, '2023-06-16', '2023-06-22'),
    ('2023-06-01', 'DiningCalendar', 4, '2023-06-23', '2023-07-03'),
    ('2023-06-01', 'CalGeoAllCalendar', 1, '2023-06-01', '2023-06-15'),
    ('2023-06-01', 'CalGeoAllCalendar', 2, '2023-06-16', '2023-06-30')
;



CREATE TABLE tblPlayers (
    [EvaluationDate] date, 
    [OfferMonth] date, 
    [PlayerID] int, 
    [FirstName] varchar(40), 
    [LastName] varchar(40), 
    [FilterCode] int, 
    [OKToMail] varchar(1), 
    [Address] varchar(80), 
    [City] varchar(40), 
    [State] varchar(20), 
    [Country] varchar(20), 
    [ZipCode] varchar(10), 
    [Email] varchar(80), 
    [Birthday] date, 
    [Geo] varchar(40),
    [GamePref] varchar(20), 
    [ADT] float, 
    [Tier] int, 
    [Measure] varchar(20)
)
;
    


CREATE TABLE tblTiers (
    [OfferMonth] date, 
    [Tier] int, 
    [MinValue] float, 
    [MaxValue] float, 
    [Measure] varchar(40), 
    [Geo] varchar(40)
)
;
    
INSERT INTO tblTiers
    ([OfferMonth], [Tier], [MinValue], [MaxValue], [Measure], [Geo])
VALUES
    ('2023-06-01', 1001, 1250.0, 9999999.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1002, 750.0, 1250.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1003, 400.0, 750.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1004, 200.0, 400.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1005, 125.0, 200.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1006, 80.0, 125.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1007, 50.0, 80.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 1008, 25.0, 50.0, 'ADT', 'LasVegas'),
    ('2023-06-01', 2001, 4000.0, 9999999.0, 'ADT', 'California'),
    ('2023-06-01', 2002, 2000.0, 4000.0, 'ADT', 'California'),
    ('2023-06-01', 2003, 800.0, 2000.0, 'ADT', 'California'),
    ('2023-06-01', 2004, 400.0, 800.0, 'ADT', 'California'),
    ('2023-06-01', 2005, 250.0, 400.0, 'ADT', 'California'),
    ('2023-06-01', 2006, 150.0, 250.0, 'ADT', 'California'),
    ('2023-06-01', 2007, 75.0, 150.0, 'ADT', 'California'),
    ('2023-06-01', 2008, 25.0, 75.0, 'ADT', 'California')
;


CREATE TABLE tblMessages (
    [OfferMonth] date, 
    [Category] varchar(40), 
    [Placement] varchar(40),
    [TierMin] int, 
    [TierMax] int, 
    [Message] varchar(200)
)
;
    
INSERT INTO tblMessages
    ([OfferMonth], [Category], [Placement], [TierMin], [TierMax], [Message])
VALUES
    ('2023-06-01', 'gaming', 'GamingSubhead', 2001, 2199, 'Start with ~~ to play your favorite games, twice in ##!'),
    ('2023-06-01', 'dining', 'DiningSubhead', 2001, 2199, 'Enjoy ~~ per visit at our 9 fantastic restaurants!'),
    ('2023-06-01', 'totals', 'OutsideHeadline', 1001, 1199, '10 ## OFFERS WORTH ^^ INSIDE!'),
    ('2023-06-01', 'totals', 'OutsideHeadline', 2001, 2199, 'GET UP TO ^^ WHEN YOU VISIT US IN ##!'),
    ('2023-06-01', 'gaming', 'GamingSubhead', 1001, 1199, 'Visit up to 10 times in ## and start with ~~.'),
    ('2023-06-01', 'dining', 'DiningSubhead', 1001, 1199, '~~ on us, available once per week.'),
    ('2023-06-01', 'gaming', 'GamingHeadline', 1001, 9999, '@@ in **'),
    ('2023-06-01', 'dining', 'DiningHeadline', 1001, 9999, '@@ Complimentary Dining')
;


CREATE TABLE tblTierOffers (
    [OfferMonth] date, 
    [Tier] int, 
    [SlotAmount] int, 
    [TableAmount] int, 
    [TableType] varchar(40), 
    [GamingFrequency] int, 
    [DiningAmount] int, 
    [DiningFrequency] int, 
    [HotelCode] varchar(20), 
    [HotelFrequency] int
)
;
    
INSERT INTO tblTierOffers
    ([OfferMonth], [Tier], [SlotAmount], [TableAmount], [TableType], [GamingFrequency], [DiningAmount], [DiningFrequency], [HotelCode], [HotelFrequency])
VALUES
    ('2023-06-01', 1001, 400, 300, 'Bonus Chips', 10, 100, 4, 'LV6231', 2),
    ('2023-06-01', 1002, 200, 150, 'Bonus Chips', 10, 60, 4, 'LV6231', 2),
    ('2023-06-01', 1003, 100, 75, 'Bonus Chips', 10, 40, 4, 'LV6232', 2),
    ('2023-06-01', 1004, 50, 45, 'Bonus Chips', 10, 30, 4, 'LV6232', 2),
    ('2023-06-01', 1005, 40, 30, 'Bonus Chips', 10, 20, 4, 'LV6233', 2),
    ('2023-06-01', 1006, 30, 30, 'Matched Bet', 10, 15, 4, 'LV6233', 2),
    ('2023-06-01', 1007, 20, 20, 'Matched Bet', 10, 10, 4, 'LV6234', 2),
    ('2023-06-01', 1008, 15, 10, 'Matched Bet', 10, 7, 4, 'LV6234', 2),
    ('2023-06-01', 2001, 1200, 1600, 'Bonus Chips', 2, 200, 2, 'CA6231', 2),
    ('2023-06-01', 2002, 600, 750, 'Bonus Chips', 2, 150, 2, 'CA6231', 2),
    ('2023-06-01', 2003, 300, 400, 'Bonus Chips', 2, 75, 2, 'CA6232', 2),
    ('2023-06-01', 2004, 150, 200, 'Bonus Chips', 2, 40, 2, 'CA6232', 2),
    ('2023-06-01', 2005, 100, 125, 'Bonus Chips', 2, 30, 2, 'CA6233', 2),
    ('2023-06-01', 2006, 60, 75, 'Bonus Chips', 2, 20, 2, 'CA6233', 2),
    ('2023-06-01', 2007, 30, 35, 'Matched Bet', 2, 15, 2, 'CA6234', 2),
    ('2023-06-01', 2008, 10, 20, 'Matched Bet', 2, 10, 2, 'CA6234', 2)
;



CREATE TABLE tblHotelOffers (
    [OfferMonth] date, 
    [HotelCode] varchar(20), 
    [HotelHeadline] varchar(200), 
    [HotelDescription] varchar(400)
)
;
    
INSERT INTO tblHotelOffers
    ([OfferMonth], [HotelCode], [HotelHeadline], [HotelDescription])
VALUES
    ('2023-06-01', 'LV6231', '2 NIGHT STAY-CATION', 'Stay with us for two nights in a gorgeous suite without any resort fees'),
    ('2023-06-01', 'LV6232', '2 NIGHT STAY-CATION', 'Stay with us for two nights in a high-floor room without any resort fees'),
    ('2023-06-01', 'LV6233', '1 NIGHT STAY-CATION', 'Play here and stay here in a beautiful room'),
    ('2023-06-01', 'LV6234', 'RATES STARTING AT $39', 'Call soon for special casino rates as low as $39 weekdays and $89 Friday & Saturday'),
    ('2023-06-01', 'CA6231', '3 NIGHTS IN A SUITE', 'Be our guest for 3 nights in a gorgeous, multi-room suite without resort fees'),
    ('2023-06-01', 'CA6232', '3 NIGHTS + UPGRADE', 'Be our guest for 3 nights in a gorgeous, high-floor room without resort fees'),
    ('2023-06-01', 'CA6233', '2 COMPED NIGHTS', 'Be our guest for 2 nights with your choice of King bed or 2 doubles'),
    ('2023-06-01', 'CA6234', 'RATES STARTING AT $39', 'Call soon for special casino rates as low as $39 weekdays and $89 Friday & Saturday')
;



CREATE TABLE tblCalendars (
    [OfferMonth] date,
    [CalendarName] varchar(40), 
    [Period] int, 
    [StartDate] date, 
    [EndDate] date
)
;
    
INSERT INTO tblCalendars
    ([OfferMonth], [CalendarName], [Period], [StartDate], [EndDate])
VALUES
    ('2023-06-01', 'HotelCalendar', 1, '2023-06-01', '2023-06-15'),
    ('2023-06-01', 'HotelCalendar', 2, '2023-06-16', '2023-06-30'),
    ('2023-06-01', 'GamingCalendar', 1, '2023-06-01', '2023-06-03'),
    ('2023-06-01', 'GamingCalendar', 2, '2023-06-04', '2023-06-06'),
    ('2023-06-01', 'GamingCalendar', 3, '2023-06-07', '2023-06-09'),
    ('2023-06-01', 'GamingCalendar', 4, '2023-06-10', '2023-06-12'),
    ('2023-06-01', 'GamingCalendar', 5, '2023-06-13', '2023-06-15'),
    ('2023-06-01', 'GamingCalendar', 6, '2023-06-16', '2023-06-18'),
    ('2023-06-01', 'GamingCalendar', 7, '2023-06-19', '2023-06-21'),
    ('2023-06-01', 'GamingCalendar', 8, '2023-06-22', '2023-06-24'),
    ('2023-06-01', 'GamingCalendar', 9, '2023-06-25', '2023-06-27'),
    ('2023-06-01', 'GamingCalendar', 10, '2023-06-28', '2023-06-30'),
    ('2023-06-01', 'DiningCalendar', 1, '2023-06-01', '2023-06-08'),
    ('2023-06-01', 'DiningCalendar', 2, '2023-06-09', '2023-06-15'),
    ('2023-06-01', 'DiningCalendar', 3, '2023-06-16', '2023-06-22'),
    ('2023-06-01', 'DiningCalendar', 4, '2023-06-23', '2023-07-03'),
    ('2023-06-01', 'CalGeoAllCalendar', 1, '2023-06-01', '2023-06-15'),
    ('2023-06-01', 'CalGeoAllCalendar', 2, '2023-06-16', '2023-06-30')
;



CREATE TABLE tblPlayers (
    [EvaluationDate] date, 
    [OfferMonth] date, 
    [PlayerID] int, 
    [FirstName] varchar(40), 
    [LastName] varchar(40), 
    [FilterCode] int, 
    [OKToMail] varchar(1), 
    [Address] varchar(80), 
    [City] varchar(40), 
    [State] varchar(20), 
    [Country] varchar(20), 
    [ZipCode] varchar(10), 
    [Email] varchar(80), 
    [Birthday] date, 
    [Geo] varchar(40),
    [GamePref] varchar(20), 
    [ADT] float, 
    [Tier] int, 
    [Measure] varchar(20)
)
;
    
INSERT INTO tblPlayers
    ([EvaluationDate], [OfferMonth], [PlayerID], [FirstName], [LastName], [FilterCode], [OKToMail], [Address], [City], [State], [Country], [ZipCode], [Email], [Birthday], [Geo], [GamePref], [ADT], [Tier], [Measure])
VALUES
    ('2023-03-14', '2023-06-01', 757213450, 'Ch*********', 'Son*', 0, 'Y', '1995 *********** Dr', 'SANTA MONICA', 'CA', 'USA', '90404', NULL, '1973-02-05', 'California', 'Slot', 4691.93, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 552801883, 'Er*****', 'Hag**', 0, 'Y', '611 ************ Ave', 'LYNWOOD', 'CA', 'USA', '90262', NULL, '1981-12-09', 'California', 'Table', 1150.56, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 225853387, 'Vi****', 'Har*****', 0, 'Y', '5347 ************ Pl', 'CERRITTOS', 'CA', 'USA', '90703', 'v*****************@rocketmail.com', '1951-05-27', 'California', 'Slot', 174.64, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 387851482, 'Mi*****', 'Ahs**', 0, 'Y', '7775 ********** Dr', 'LAS VEGAS', 'NV', 'USA', '89139', 'm************@gmail.com', '1971-07-21', 'LasVegas', 'Table', 73.71, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 544892264, 'Ja****', 'Gar****', 0, 'Y', '3824 ****** Rd', 'LOS ANGELES', 'CA', 'USA', '90047', 't**********@verizon.net', '1991-02-01', 'California', 'Table', 5388.92, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 354888621, 'Ni****', 'Cal****', 0, 'Y', '6951 ********* St', 'RENO', 'NV', 'USA', '89506', NULL, '1984-12-26', 'California', 'Slot', 544.68, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 49348303, 'Al**', 'Xue', 0, 'Y', '2718 ********* Dr', 'COLTON', 'CA', 'USA', '92324', NULL, '1986-03-06', 'California', 'Table', 90.30, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 92026200, 'Sa******', 'Car*****', 0, 'Y', '3050 ******* Pl', 'DOLAN SPRINGS', 'AZ', 'USA', '86441', 'f**************@sbcglobal.net', '1975-04-03', 'California', 'Slot', 35.36, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 906764189, 'Ju****', 'Eme****', 0, 'N', '8171 ******* St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'j*********@rocketmail.com', '1986-08-13', 'California', 'Table', 66.80, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 196815879, 'Gu*', 'Ler****', 0, 'Y', '3258 ************ Ave', 'ROSEMEAD', 'CA', 'USA', '91770', NULL, '1966-05-23', 'California', 'Table', 774.33, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 462763220, 'An*****', 'Por****', 0, 'Y', '7837 ****** Ct', '"SACRAMENTO "', 'AZ', 'USA', '85392', 'a**************@sbcglobal.net', '1986-07-19', 'California', 'Slot', 2442.07, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 349644892, 'Da****', 'Fra***', 0, 'Y', '2056 ******* Dr', 'LAS VEGAS', 'NV', 'USA', '89169', 'b****************@comcast.net', '1970-11-27', 'LasVegas', 'Table', 183.06, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 969432943, 'Ch******', 'Bar****', 0, 'Y', '7502 ************ Ct', 'GLENDALE', 'CA', 'USA', '91205', 'c**********@roadrunner.com', '1975-12-12', 'California', 'Slot', 155.68, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 871696986, 'Mi********', 'Liu', 0, 'Y', '8230 ***** Pl', 'CARSON', 'CA', 'USA', '90746', 'm*********@att.net', '1974-11-23', 'California', 'Slot', 1366.35, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 367530849, 'Fr*****', 'Col****', 0, 'Y', '638 ***** Dr', 'SYLMAR', 'CA', 'USA', '91342', NULL, '1968-11-08', 'California', 'Table', 57.42, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 499695617, 'Be******', 'Alt*******', 0, 'Y', '816 *********** Dr', 'TUCSON', 'AZ', 'USA', '85746', NULL, '1977-12-24', 'California', 'Slot', 785.64, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 970385681, 'Ki******', 'Sot*', 0, 'N', '1937 ********** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'k*******@netscape.net', '1970-07-18', 'California', 'Slot', 599.98, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 271755963, 'Yu****', 'Chi***', 0, 'Y', '6577 ******** Ave', 'ROSEMEAD', 'CA', 'USA', '91770', NULL, '1992-03-13', 'California', 'Table', 189.73, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 664275558, 'Ku**', 'Cho', 0, 'Y', '8687 *********** St', 'LOS ANGELES', 'CA', 'USA', '90015', 'k*********@cox.net', '1971-06-11', 'California', 'Table', 90.58, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 719375825, 'Ca*****', 'Ran****', 0, 'Y', '6341 ****** Ave', 'OXNARD', 'CA', 'USA', '93033', 'c********@aim.com', '1956-01-08', 'California', 'Slot', 76.24, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 193065103, 'Go****', 'Olm**', 0, 'N', '2386 *********** Ave', 'LA CRESCENTA', 'CA', 'UNK', 'UNKNOWN', 'g**********@comcast.net', '1975-01-24', 'California', 'Slot', 1520.20, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 209347099, 'Da****', 'Col****', 0, 'Y', '5404 ************ Dr', 'LAS VEGAS', 'NV', 'USA', '89109', NULL, '1965-05-18', 'LasVegas', 'Table', 179.52, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 134802498, 'Mi*****', 'Bar*****', 0, 'Y', '9488 ******** St', 'GRANADA HILLS', 'CA', 'USA', '91344', NULL, '1973-02-09', 'California', 'Table', 2684.90, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 28669554, 'An**', 'Sen***', 0, 'N', '1103 *********** Rd', 'LA MESA', 'AZ', 'UNK', 'UNKNOWN', 'o*****@aim.com', '1982-09-08', 'California', 'Table', 81.89, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 340698475, 'Gi***', 'Cam****', 0, 'Y', '6949 ********* Pl', 'LAS VEGAS', 'NV', 'USA', '89138', 'g**********@netscape.net', '1979-12-09', 'LasVegas', 'Table', 303.54, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 740761866, 'La***', 'Bej*****', 0, 'Y', '31 *********** Dr', 'ALTADENA', 'CA', 'USA', '91001', 'l*************@cox.net', '1974-03-18', 'California', 'Slot', 749.50, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 710702509, 'Al********', 'Pon*', 0, 'Y', '216 ****** Ct', 'LOS ANGELES', 'CA', 'USA', '90063', 's********@netscape.net', '1991-11-09', 'California', 'Slot', 54.03, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 137349667, 'Yu*', 'Gao**', 0, 'Y', '989 ************ St', 'LOS ANGELES', 'CA', 'USA', '90004', 'y*********@hotmail.com', '1954-02-09', 'California', 'Table', 5837.75, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 4981105, 'Ch*****', 'Dan***', 0, 'Y', '732 ****** St', 'YUCAIPA', 'CA', 'USA', '92399', 'c******@cox.net', '1958-12-26', 'California', 'Table', 172.95, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 20158897, 'Er**', 'Por******', 0, 'Y', '8558 ***** St', 'SAN DIEGO', 'CA', 'USA', '92120', 'e***@hotmail.net', '1968-03-07', 'California', 'Slot', 840.67, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 495662048, 'Ro****', 'Jes****', 0, 'Y', '6815 ************ Ct', 'SCOTTSDALE', 'AZ', 'USA', '85251', 'r******@att.net', '1986-01-11', 'California', 'Table', 38.78, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 620817213, 'Ca**', 'Gol******', 0, 'N', '8114 ******** Rd', 'FONTANA', 'CA', 'UNK', 'UNKNOWN', NULL, '1948-01-14', 'California', 'Slot', 901.72, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 786275837, 'Za*', 'Pat***', 0, 'Y', '5329 ******** Ave', 'WEST HILLS', 'CA', 'USA', '84770', 'g******@roadrunner.com', '1977-12-10', 'California', 'Table', 315.77, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 679212783, 'Jo**', 'Fro*****', 0, 'Y', '9193 ******** St', 'POMONA', 'CA', 'USA', '91766', 'n********@yahoo.com', '1976-10-23', 'California', 'Slot', 75.78, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 662379852, 'Ru*****', 'Shw********', 0, 'Y', '8502 *********** Pl', 'MURRIETA', 'CA', 'USA', '92562', 'r*********@rocketmail.com', '1961-09-09', 'California', 'Slot', 3559.94, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 236154367, 'Jo****', 'Koz*', 0, 'Y', '4554 ******* Rd', 'LAS VEGAS', 'NV', 'USA', '89147', 'j********@mac.com', '1977-04-20', 'LasVegas', 'Slot', 126.42, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 912195305, 'Je*****', 'Sab***', 0, 'Y', '9022 ****** Pl', 'PAYSON', 'AZ', 'USA', '85541', 'j**********@cox.net', '1953-05-04', 'California', 'Slot', 219.40, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 851293765, 'Gl****', 'Kan*', 0, 'Y', '4945 ****** Ct', 'LAS VEGAS', 'NV', 'USA', '89109', 'f******@roadrunner.com', '1944-11-27', 'LasVegas', 'Slot', 370.22, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 83653618, 'Mi*****', 'Col***', 0, 'Y', '2535 ******** Ave', 'HOLTVILLE', 'CA', 'USA', '92250', 'v***********@aol.com', '1986-07-05', 'California', 'Slot', 227.99, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 825191972, 'Ma***', 'Ros******', 0, 'Y', '285 ********** Ave', 'COVINA', 'CA', 'USA', '91723', 'm*********@roadrunner.com', '1971-06-28', 'California', 'Table', 2151.67, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 47831068, 'Jo**', 'Lig**', 0, 'Y', '786 ******* Pl', 'PARADISE VALLEY', 'AZ', 'USA', '85253', 'j*******@mac.com', '1970-10-25', 'California', 'Table', 63.83, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 622594309, 'Pa*****', 'Smi**', 0, 'Y', '1662 ******** Ave', 'SAN DIEGO', 'CA', 'USA', '92106', 'p***********************@netscape.net', '1972-01-24', 'California', 'Slot', 340.87, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 576951115, 'Le*', 'Guo', 0, 'Y', '8963 *********** Ct', 'PEORIA', 'AZ', 'USA', '85383', NULL, '1946-04-27', 'California', 'Table', 3020.57, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 69230328, 'Ly**', 'Par**********', 0, 'Y', '1288 *********** Pl', 'LAS VEGAS', 'NV', 'UNK', '76063', 'l***************@netscape.net', '1949-11-13', 'California', 'Slot', 305.94, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 573750403, 'Ri*****', 'Gre*******', 0, 'Y', '6613 *********** Dr', 'NEWPORT BEACH', 'CA', 'USA', '92660', 'n************@verizon.net', '1949-07-14', 'California', 'Table', 80.46, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 441634753, 'Va*****', 'Sir***', 0, 'Y', '7300 ***** Pl', 'LA JOLLA', 'CA', 'USA', '92037', 'e***************@aol.com', '1982-11-11', 'California', 'Slot', 132.01, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 87576739, 'Si****', 'Cas***', 0, 'Y', '3531 *********** Pl', 'LAS VEGAS', 'NV', 'USA', '89117', 's***********@gmail.com', '1994-03-27', 'LasVegas', 'Slot', 496.76, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 920905406, 'Ca*****', 'Mas**', 0, 'Y', '4608 ************ Pl', 'SAN DIEGO', 'CA', 'USA', '92154', 'i************@hotmail.net', '1968-03-17', 'California', 'Table', 60.42, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 150290023, 'Ev*', 'Muj***', 0, 'N', '8354 ***** Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1941-01-20', 'California', 'Slot', 126.49, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 4559862, 'Al**', 'Mun****', 0, 'Y', '2273 ********** Ave', 'SEAL BEACH', 'CA', 'USA', '90740', 'n******@rocketmail.com', '1967-11-12', 'California', 'Slot', 127.78, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 4467050, 'Ry**', 'Cre***', 0, 'Y', '5015 ************ Pl', 'HENDERSON', 'NV', 'USA', '89052', NULL, '1976-08-23', 'LasVegas', 'Table', 984.47, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 228183243, 'By***', 'Wil*****', 0, 'Y', '1354 ********* Rd', 'LOS ANGELES', 'CA', 'USA', '90057', NULL, '1965-01-10', 'California', 'Slot', 593.95, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 25773185, 'Le***', 'Del*****', 0, 'N', '6914 *********** Rd', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'l************@yahoo.com', '1976-01-06', 'California', 'Slot', 1328.49, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 358031317, 'St****', 'Rol****', 0, 'N', '8751 ************ Pl', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 's*********@outlook.com', '1973-06-11', 'California', 'Table', 82.04, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 466237510, 'Ji****', 'Ler***', 0, 'Y', '4347 ********** St', 'LAS VEGAS', 'NV', 'USA', '89139', 'l***********@verizon.net', '1981-11-15', 'LasVegas', 'Slot', 1224.03, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 590446538, 'No*****', 'Lou*****', 0, 'Y', '617 *********** Dr', 'RIDGECREST', 'CA', 'USA', '93555', 'j**************@hotmail.com', '1968-11-15', 'California', 'Slot', 357.45, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 178451154, 'Ed*****', 'Pre****', 0, 'Y', '7197 ***** Ct', 'LONG BEACH', 'CA', 'USA', '90806', NULL, '1954-12-05', 'California', 'Slot', 689.18, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 97860946, 'Er******', 'Irv***', 0, 'N', '1155 ******** Pl', '"SACRAMENTO "', 'AZ', 'UNK', 'UNKNOWN', NULL, '1978-07-20', 'California', 'Slot', 556.52, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 160318245, 'Gr*****', 'Kil*****', 0, 'Y', '8012 ********* Ave', 'LOS ANGELES', 'CA', 'USA', '90019', 's***************@yahoo.com', '1963-11-04', 'California', 'Table', 146.55, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 775613748, 'Je******', 'Noc**', 0, 'Y', '501 ************ St', 'MURRIETA', 'CA', 'USA', '92563', 's*************@netscape.net', '1984-09-15', 'California', 'Slot', 41.26, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 265239626, 'De*', 'Bea*******', 0, 'N', '4962 ************ St', 'COVINA', 'CA', 'UNK', 'UNKNOWN', 'j********@hotmail.net', '1979-02-14', 'California', 'Table', 470.41, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 509172748, 'Ma', 'Lar*****', 0, 'Y', '6119 ****** Ave', 'SAN FERNANDO', 'CA', 'USA', '91340', 'z*********@verizon.net', '1965-01-22', 'California', 'Slot', 1998.80, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 477756985, 'Ho*****', 'Wen*', 0, 'Y', '594 ********* Rd', 'LAS VEGAS', 'NV', 'USA', '89102', 'r*********@comcast.net', '1951-01-21', 'LasVegas', 'Table', 379.33, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 362205678, 'Ki', 'Jar***', 0, 'N', '3252 ******* St', 'HENDERSON', 'NV', 'UNK', 'UNKNOWN', 's******@cox.net', '1971-12-09', 'California', 'Table', 2077.25, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 708045447, 'Ma***', 'Liu', 0, 'Y', '166 *********** Ave', 'LOS ANGELES', 'CA', 'USA', '90006', 'm**********@att.net', '1968-10-25', 'California', 'Slot', 44.67, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 221642283, 'Ch******', 'Tol******', 0, 'Y', '2474 ****** Dr', 'NORTH LAS VEGAS', 'NV', 'USA', '89031', NULL, '1988-03-20', 'LasVegas', 'Table', 879.98, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 620254928, 'Cy*****', 'Ort**********', 0, 'Y', '1750 ***** Pl', 'LOS ANGELES', 'CA', 'USA', '90033', 'c****************@att.net', '1971-03-02', 'California', 'Slot', 753.33, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 25167870, 'Da**', 'Ilo*', 0, 'N', '5935 ********* Ave', 'INGLEWOOD', 'CA', 'UNK', 'UNKNOWN', 'd***************@hotmail.com', '1995-05-23', 'California', 'Slot', 201.48, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 79061398, 'Al***', 'Sca*****', 0, 'N', '7466 ********** Ave', 'PHOENIX', 'AZ', 'UNK', 'UNKNOWN', 'e**********@verizon.net', '1961-06-02', 'California', 'Slot', 4317.79, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 863518037, 'Ha****', 'Mcm****', 0, 'Y', '6144 *********** Dr', 'FONTANA', 'CA', 'USA', '92336', 'h*******@mac.com', '1968-05-17', 'California', 'Slot', 107.65, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 758578873, 'Ri*****', 'Lam', 0, 'N', '3877 ********** Rd', 'SYLMAR', 'CA', 'UNK', 'UNKNOWN', 'r***********@roadrunner.com', '1967-10-05', 'California', 'Slot', 430.26, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 629425631, 'Me*****', 'Sis**', 0, 'Y', '1254 ***** Ave', 'BAKERSFIELD', 'CA', 'USA', '93308', 'm****************@rocketmail.com', '1967-05-03', 'California', 'Slot', 40.65, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 392203360, 'Je****', 'Che*', 0, 'Y', '6773 ******** Ave', 'LAS VEGAS', 'NV', 'USA', '89161', 'j***************@cox.net', '1972-12-12', 'LasVegas', 'Table', 76.60, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 681011084, 'Pa**', 'Shi*', 0, 'N', '4898 ******* Dr', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'p***********@verizon.net', '1967-06-11', 'California', 'Slot', 104.32, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 12762817, 'Da****', 'Cer************', 0, 'Y', '2934 ******** Dr', 'LONG BEACH', 'CA', 'USA', '90813', 't***********@aim.com', '1986-11-28', 'California', 'Table', 63.41, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 676229802, 'Ma*****', 'Gil**', 0, 'Y', '4082 ******* St', 'RENO', 'NV', 'USA', '89506', 'p**********@cox.net', '1946-06-21', 'California', 'Slot', 124.41, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 803201080, 'Ja*****', 'Que****', 0, 'Y', '3125 ************ Pl', 'SAN DIMAS', 'CA', 'USA', '91773', 'j******@sbcglobal.net', '1950-11-06', 'California', 'Slot', 1447.80, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 629855652, 'Je******', 'Loc*****', 0, 'Y', '2488 ********** Ave', 'RENO', 'NV', 'USA', '89511', 'a******@verizon.net', '1975-02-28', 'California', 'Slot', 2362.79, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 813013602, 'Do*****', 'Kes****', 0, 'Y', '8121 ******* Rd', 'LAS VEGAS', 'NV', 'USA', '89170', NULL, '1968-03-22', 'LasVegas', 'Slot', 571.35, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 697271523, 'Ju******', 'Ros****', 0, 'Y', '556 ********** Rd', 'RIVERSIDE', 'CA', 'USA', '92503', 'j**********@att.net', '1962-02-26', 'California', 'Slot', 3022.08, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 224074572, 'So***', 'Egl*', 0, 'Y', '3636 ************ Ave', 'GRANADA HILLS', 'CA', 'USA', '91344', 's*******@outlook.com', '1967-12-15', 'California', 'Slot', 1694.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 702467551, 'Ro****', 'Gor***', 0, 'Y', '4704 ********* Dr', 'LA MIRADA', 'CA', 'USA', '90638', 'r***********@verizon.net', '1984-03-25', 'California', 'Table', 181.18, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 281121338, 'Ja**', 'Bur******', 0, 'Y', '8994 ********* Ave', 'INDIAN WELLS', 'CA', 'USA', '92210', 'j*****@yahoo.com', '1944-02-23', 'California', 'Table', 284.89, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 201606274, 'Ab*****', 'Wor***', 0, 'N', '3530 ********** Dr', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'a*********@roadrunner.com', '1989-11-17', 'California', 'Table', 404.20, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 612130828, 'Ni*****', 'Paw*****', 0, 'Y', '675 ******** Rd', 'LONG BEACH', 'CA', 'USA', '90805', NULL, '1967-12-11', 'California', 'Slot', 838.15, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 860139056, 'Ro****', 'Fue********', 0, 'Y', '97 ******* St', 'SANTA ANA', 'CA', 'USA', '92705', 'r****************@sbcglobal.net', '1989-01-13', 'California', 'Table', 56.89, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 34448308, 'Hl*', 'Vig******', 0, 'Y', '8104 ****** Dr', 'ALHAMBRA', 'CA', 'USA', '91801', NULL, '1958-10-08', 'California', 'Table', 2435.97, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 827835829, 'Ma***', 'Fre***', 0, 'Y', '7215 ***** Dr', 'WEST COVINA', 'CA', 'USA', '91790', 'm************@sbcglobal.net', '1946-04-02', 'California', 'Slot', 85.52, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 972839989, 'Ke*****', 'Gan**', 0, 'Y', '8348 ****** Rd', 'LOS ANGELES', 'CA', 'USA', '90061', NULL, '1967-04-10', 'California', 'Slot', 5080.68, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 426556806, 'Ma*********', 'Rob*****', 0, 'Y', '9077 *********** Ct', '"SUN VALLEY "', 'NV', 'USA', '89433', 'v****************@cox.net', '1974-03-10', 'California', 'Slot', 29.69, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 731948065, 'Al****', 'Waw', 0, 'Y', '6268 ***** Dr', 'SUN TAN VALLEY', 'AZ', 'USA', '85140', 'a*******@cox.net', '1974-03-02', 'California', 'Slot', 66.51, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 703606577, 'Em**', 'Bar******', 0, 'Y', '7530 ******* Ave', 'BAKERSFIELD', 'CA', 'USA', '93308', 'e********@yahoo.com', '1963-12-25', 'California', 'Slot', 47.64, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 246022985, 'Cr******', 'Joh****', 0, 'Y', '4628 ***** Dr', 'LAS VEGAS', 'NV', 'USA', '89169', NULL, '1985-01-16', 'LasVegas', 'Slot', 75.26, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 764833196, 'Ty***', 'Lea****', 0, 'Y', '2220 ******** Ave', 'LOS ANGELES', 'CA', 'USA', '90002', 'm*************@hotmail.com', '1961-11-13', 'California', 'Slot', 267.87, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 488355529, 'Ro****', 'Pea***', 0, 'Y', '2639 ******** Pl', 'IMPERIAL', 'CA', 'UNK', 'UNK', 'c*******@hotmail.net', '1954-08-07', 'California', 'Slot', 29.36, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 766368666, 'Co***', 'Gue*****', 0, 'Y', '4606 ******* Ave', 'SAUGUS', 'CA', 'USA', '91350', NULL, '1995-10-28', 'California', 'Both', 443.90, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 582852962, 'Th****', 'Myi**', 0, 'N', '7201 ***** Ave', 'INDIAN WELLS', 'CA', 'UNK', 'UNKNOWN', NULL, '1953-04-05', 'California', 'Slot', 2506.14, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 974500666, 'Ch*********', 'Fis***', 0, 'Y', '8686 ********* St', 'LAS VEGAS', 'NV', 'USA', '89109', NULL, '1988-08-28', 'LasVegas', 'Table', 127.91, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 977071952, 'Th*******', 'Net****', 0, 'N', '9456 ********* Ave', 'GOODYEAR', 'AZ', 'UNK', 'UNKNOWN', 'p*********@outlook.com', '1989-10-23', 'California', 'Slot', 1222.31, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 794934125, 'Im**', 'Roc********', 0, 'N', '6430 ********** Ct', 'SAN DIMAS', 'CA', 'UNK', 'UNKNOWN', NULL, '1971-09-25', 'California', 'Slot', 133.85, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 251310164, 'Ma***', 'Gor***', 0, 'Y', '7784 ******** Ct', 'LAS VEGAS', 'NV', 'USA', '89102', 'm******@rocketmail.com', '1975-11-19', 'LasVegas', 'Slot', 249.39, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 805296513, 'Th*', 'Men**********', 0, 'N', '175 ****** Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1959-06-26', 'California', 'Table', 46.41, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 746550919, 'Hi***', 'Alo***', 0, 'Y', '7598 ****** Ave', 'IMPERIAL', 'CA', 'USA', '92251', 'e************@hotmail.com', '1973-01-22', 'California', 'Slot', 212.09, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 334249571, 'Wi******', 'Lar**', 0, 'Y', '6072 ************ Ct', 'FRESNO', 'CA', 'USA', '93727', 'd****************@verizon.net', '1960-01-11', 'California', 'Slot', 3552.48, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 472714440, 'Ja*', 'Elg************', 0, 'N', '9355 ******** Ct', 'CHULA VISTA', 'CA', 'UNK', 'UNKNOWN', 'g*****@mac.com', '1958-06-28', 'California', 'Slot', 46.67, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 855732271, 'Ri*****', 'Sto**', 0, 'Y', '4474 ***** Dr', 'PICO RIVERA', 'CA', 'USA', '90660', 'p***********@aim.com', '1968-10-16', 'California', 'Slot', 46.59, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 194260808, 'Kw***', 'Mer****', 0, 'Y', '4952 *********** St', 'LOS ANGELES', 'CA', 'USA', '90019', 's******************@mac.com', '1977-07-28', 'California', 'Table', 1666.87, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 751140237, 'Jo**', 'Str*****', 0, 'Y', '7407 ****** Dr', 'OXNARD', 'CA', 'USA', '93036', 'b**********@gmail.com', '1990-05-06', 'California', 'Table', 139.63, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 677659814, 'Ha****', 'Mun****', 0, 'Y', '2272 ******** Pl', 'NORTH HOLLYWOOD', 'CA', 'USA', '91606', 'h***********@gmail.com', '1970-11-22', 'California', 'Slot', 1655.73, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 85435467, 'Na*', 'Beh****', 0, 'Y', '3058 ************ Ave', 'HENDERSON', 'NV', 'USA', '89014', 'n*****@aim.com', '1971-04-13', 'LasVegas', 'Slot', 1292.72, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 4708005, 'Da****', 'Bat*************', 0, 'Y', '3274 *********** Pl', 'WEST HILLS', 'CA', 'USA', '91304', 'd**********@comcast.net', '1973-11-26', 'California', 'Table', 5412.14, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 974174340, 'Si***', 'Ngu***', 0, 'N', '4086 *********** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'i***@outlook.com', '1970-04-26', 'California', 'Slot', 2429.40, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 441368421, 'Le*', 'Mcl***', 0, 'Y', '1153 ****** Rd', 'OXNARD', 'CA', 'USA', '93036', NULL, '1968-05-19', 'California', 'Table', 80.01, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 257503995, 'Ch***', 'Fer******', 0, 'Y', '3681 ************ Rd', 'TUCSON', 'AZ', 'USA', '85706', NULL, '1962-04-02', 'California', 'Slot', 1596.87, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 157370712, 'Ga***', 'Tae*', 0, 'Y', '130 ********** Rd', 'GLENDALE', 'CA', 'USA', '91202', 'a********@aol.com', '1978-04-13', 'California', 'Both', 168.54, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 993654215, 'Ch******', 'Pim*****', 0, 'Y', '4206 *********** Pl', 'LOS ANGELES', 'CA', 'USA', '90006', NULL, '1973-11-15', 'California', 'Both', 1536.40, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 7251580, 'Ra****', 'Ham*', 0, 'N', '982 ******** St', 'ALHAMBRA', 'CA', 'UNK', 'UNKNOWN', 'c*************@gmail.com', '1979-12-15', 'California', 'Slot', 1781.66, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 731187439, 'Ka***', 'Ben****', 0, 'Y', '3906 ******** Ct', 'TUCSON', 'AZ', 'USA', '85743', 'f******@gmail.com', '1968-12-09', 'California', 'Slot', 465.31, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 525378472, 'Ro*****', 'Bra**', 0, 'N', '494 ****** Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'b******@aol.com', '1975-05-01', 'California', 'Slot', 1074.06, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 506282637, 'Ma*******', 'Kuo', 0, 'Y', '8009 ****** St', 'WOODLAND HILLS', 'CA', 'USA', '91364', 's**************@aim.com', '1960-11-12', 'California', 'Slot', 66.53, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 67617367, 'Ro******', 'Chi*', 0, 'Y', '856 *********** St', 'SOUTH PASADENA', 'CA', 'USA', '91030', 'r***************@yahoo.com', '1951-07-15', 'California', 'Slot', 3874.61, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 105961733, 'Ca***', 'Lan*', 0, 'Y', '7882 ****** Rd', 'GILBERT', 'AZ', 'USA', '85297', 'c****@hotmail.net', '1980-09-04', 'California', 'Slot', 100.09, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 972514768, 'Pa******', 'Cox', 0, 'N', '4162 ************ Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1955-02-15', 'California', 'Slot', 246.07, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 353597042, 'Wa***', 'O''N***', 0, 'Y', '4211 ********** Dr', 'NORTHRIDGE', 'CA', 'USA', '91327', 'w*********@sbcglobal.net', '1969-05-27', 'California', 'Table', 34.15, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 383559564, 'Sa****', 'Mkr******', 0, 'N', '8114 ******* Rd', 'CHULA VISTA', 'CA', 'UNK', 'UNKNOWN', 's************@att.net', '1955-12-05', 'California', 'Slot', 28.75, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 983677302, 'Ar****', 'Zhe**', 0, 'N', '6303 ************ Pl', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'a************@aol.com', '1981-08-10', 'California', 'Slot', 377.06, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 987751378, 'Sa***', 'Lle********', 0, 'N', '6175 ********** Dr', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1986-08-03', 'California', 'Both', 1740.45, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 14881504, 'Li', 'Fre****', 0, 'Y', '377 ********* Dr', 'LAS VEGAS', 'NV', 'USA', '89139', NULL, '1977-06-11', 'LasVegas', 'Table', 94.98, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 384822734, 'Qi****', 'Oli**********', 0, 'Y', '1157 *********** Ave', 'LAS VEGAS', 'NV', 'USA', '89148', NULL, '1963-06-27', 'LasVegas', 'Table', 124.85, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200372920, 'Ka*****', 'Agh*************', 0, 'Y', '3179 ************ Rd', 'FRESNO', 'CA', 'USA', '93722', NULL, '1986-06-13', 'California', 'Both', 204.42, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 600394028, 'Je*****', 'Hud***', 0, 'Y', '5235 *********** Ave', 'SAN PEDRO', 'CA', 'USA', '90731', 'j******@sbcglobal.net', '1954-03-24', 'California', 'Slot', 1469.97, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 68142352, 'Ml****', 'Wei***', 0, 'N', '2643 ********** Ct', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1962-09-24', 'California', 'Slot', 749.50, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 52004790, 'Ai*****', 'Smi***', 0, 'N', '2221 ***** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1969-04-20', 'California', 'Slot', 1105.66, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 69843227, 'Ya**', 'Son*', 0, 'Y', '2599 ******* Pl', 'YORBA LINDA', 'CA', 'USA', '92886', NULL, '1992-03-18', 'California', 'Table', 78.26, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 370763471, 'Ka*', 'Dum***********', 0, 'Y', '6399 ************ St', 'NUEVO', 'CA', 'USA', '92567', 'k******@gmail.com', '1980-02-23', 'California', 'Slot', 101.77, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 756342606, 'El***', 'Els****', 0, 'Y', '5005 *********** St', 'ORANGE', 'CA', 'USA', '92866', 'd******@hotmail.net', '1975-06-10', 'California', 'Slot', 4903.48, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 853966165, 'Te****', 'Lee', 0, 'Y', '5250 ******* Dr', 'LAKE HAVASU CITY', 'AZ', 'USA', '86406', 'f*****************@att.net', '1966-02-04', 'California', 'Slot', 3774.41, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 695884708, 'Me****', 'Lal****', 0, 'Y', '3451 ***** Rd', 'RENO', 'NV', 'USA', '89506', 'k****************@netscape.net', '1990-09-15', 'California', 'Both', 45.89, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 932222535, 'Ga**', 'Bur******', 0, 'N', '6087 ******** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'b******@yahoo.com', '1972-05-27', 'California', 'Slot', 145.55, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 271494236, 'Di***', 'Her', 0, 'Y', '785 ************ Ave', 'SANTA CLARITA', 'CA', 'USA', '91390', 'd***********@att.net', '1979-02-10', 'California', 'Slot', 71.95, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 843412216, 'Ro****', 'Low', 0, 'Y', '7889 ********* Ct', 'SANTA CLARITA', 'CA', 'USA', '91350', 'r***********@hotmail.net', '1963-09-07', 'California', 'Slot', 3324.92, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 291487270, 'Co*****', 'Hos*', 0, 'Y', '1655 ************ Ct', 'CARSON', 'CA', 'USA', '90745', 'p*************@outlook.com', '1943-02-24', 'California', 'Slot', 190.05, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 598965863, 'Li***', 'Ram******', 0, 'Y', '3651 ******* Dr', 'GLENDALE', 'CA', 'USA', '91205', 'l*************@verizon.net', '1981-04-15', 'California', 'Slot', 837.06, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 481771338, 'Ja***', 'Pat**', 0, 'Y', '4811 ************ St', 'TOLLESON', 'AZ', 'USA', '85353', 'd*********@mac.com', '1962-09-27', 'California', 'Slot', 450.88, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 283123731, 'Ry**', 'Kha******', 0, 'Y', '9122 ***** Ave', 'RENO', 'NV', 'USA', '89508', 'r************@comcast.net', '1984-07-28', 'California', 'Slot', 68.93, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 900778642, 'Am***', 'Ada*', 0, 'N', '3011 *********** Ave', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'a*******@gmail.com', '1982-05-11', 'California', 'Slot', 1576.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 878823885, 'Wi*****', 'Fre******', 0, 'Y', '8896 ******** Dr', 'SCOTTSDALE', 'AZ', 'USA', '85257', 'w*******@hotmail.com', '1989-04-08', 'California', 'Table', 240.37, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 18115688, 'Wa****', 'Mur***', 0, 'Y', '626 ********** Ave', 'CARSON CITY', 'NV', 'USA', '89706', NULL, '1958-11-02', 'California', 'Slot', 4285.96, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 205745220, 'Hr***', 'Ste*****', 0, 'N', '8574 ********** Ave', 'SYLMAR', 'CA', 'UNK', 'UNKNOWN', NULL, '1959-04-24', 'California', 'Table', 27.21, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 339443025, 'Mi**', 'San***', 0, 'Y', '961 ***** Dr', 'LOS ANGELES', 'CA', 'USA', '90017', NULL, '1973-03-06', 'California', 'Table', 3497.49, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 412807835, 'In******', 'Ros*', 0, 'Y', '6473 ***** Ave', 'NORTH LAS VEGAS', 'NV', 'USA', '89030', 'm**************@gmail.com', '1978-06-19', 'LasVegas', 'Slot', 67.11, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 533961844, 'Vi******', 'Paj******', 0, 'Y', '6501 ******** Ct', 'GLENDALE', 'AZ', 'USA', '85306', 'd*****@yahoo.com', '1973-03-12', 'California', 'Slot', 3874.22, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 114020873, 'Da***', 'Mar******', 0, 'Y', '533 *********** Rd', 'DESERT HOT SPRINGS', 'CA', 'USA', '92240', 'f************@hotmail.com', '1954-10-11', 'California', 'Slot', 72.73, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 56045225, 'Lo****', 'Mar****', 0, 'Y', '3326 ********* Rd', 'LOS ANGELES', 'CA', 'USA', '90011', NULL, '1966-12-19', 'California', 'Slot', 30.45, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 813998884, 'Xi*****', 'Jen****', 0, 'Y', '1147 ******* Dr', 'LA CRESCENTA', 'CA', 'USA', '91214', 'h*********@outlook.com', '1984-01-14', 'California', 'Table', 26.96, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 52438092, 'La******', 'Hil*', 0, 'Y', '1892 ********** St', 'LAS VEGAS', 'NV', 'USA', '89148', 'v*********@aim.com', '1968-05-17', 'LasVegas', 'Table', 61.97, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 584181126, 'Fr*******', 'Sin***', 0, 'Y', '8316 ***** St', 'LOS ANGELES', 'CA', 'USA', '90001', NULL, '1988-07-13', 'California', 'Table', 114.86, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 514396058, 'St****', 'Dav**', 0, 'Y', '8519 ********** Ct', 'GLENDALE', 'AZ', 'USA', '85301', 't*****@verizon.net', '1974-04-28', 'California', 'Both', 2695.13, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 173915259, 'Or*****', 'Jav********', 0, 'Y', '9481 ************ Dr', 'LAS VEGAS', 'NV', 'USA', '89148', 'l*******@roadrunner.com', '1988-12-19', 'LasVegas', 'Table', 83.09, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 78564509, 'Ra*****', 'Hil****', 0, 'Y', '580 ***** Dr', 'ONTARIO', 'CA', 'USA', '91761', 'r*********@gmail.com', '1973-05-05', 'California', 'Slot', 925.84, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 871553626, 'Jo***', 'Mil***', 0, 'Y', '7867 ******** St', 'ALHAMBRA', 'CA', 'USA', '91803', 'j***************@aol.com', '1966-01-02', 'California', 'Slot', 508.82, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 426358112, 'Ev****', 'Ric****', 0, 'Y', '2084 *********** Dr', 'NORTH HOLLYWOOD', 'CA', 'USA', '91601', NULL, '1980-08-20', 'California', 'Slot', 2463.90, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 96109735, 'Me***', 'Ben****', 0, 'Y', '8863 ************ Ct', 'LAS VEGAS', 'NV', 'USA', '89103', 'm*******@sbcglobal.net', '1988-09-09', 'LasVegas', 'Table', 1185.36, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 513938654, 'Ca*******', 'Tho***', 0, 'N', '8735 ******* St', 'ROSEMEAD', 'CA', 'UNK', 'UNKNOWN', 'c*****************@comcast.net', '1989-03-10', 'California', 'Slot', 1734.02, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 23400344, 'Ro***', 'Yin', 0, 'Y', '3824 *********** Rd', 'INDIO', 'CA', 'USA', '92203', 'r********@att.net', '1958-02-13', 'California', 'Slot', 611.55, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 185003293, 'Ri***', 'Lal****', 0, 'Y', '2454 *********** Ave', 'VASILIA', 'CA', 'USA', '93291', 'r*********@gmail.com', '1959-09-26', 'California', 'Slot', 226.33, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 143009552, 'An**', 'Rio*', 0, 'Y', '8901 ********* Pl', 'LAS VEGAS', 'NV', 'USA', '89138', 'a*******@hotmail.net', '1982-02-12', 'LasVegas', 'Table', 46.79, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 248285216, 'Ki*', 'Ree***', 0, 'Y', '4168 ********* St', 'SCOTTSDALE', 'AZ', 'USA', '85254', NULL, '1983-11-24', 'California', 'Table', 3708.19, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 691349065, 'Wi*****', 'Oji**********', 0, 'Y', '3566 ******** Dr', 'CHULA VISTA', 'CA', 'USA', '91915', 'b*******@aol.com', '1963-09-03', 'California', 'Table', 571.41, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 362061607, 'Ma**', 'Man****', 0, 'Y', '6981 ************ Rd', 'GARDEN GROVE', 'CA', 'USA', '92840', 'm******@aim.com', '1976-04-17', 'California', 'Table', 4660.60, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 85966333, 'Mo****', 'Jim****', 0, 'Y', '1289 ********* Ct', 'NORTH HOLLYWOOD', 'CA', 'USA', '91602', 'm**************@hotmail.net', '1979-01-28', 'California', 'Slot', 40.11, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 913251569, 'No***', 'Men***', 0, 'Y', '7965 ********** Dr', 'VASILIA', 'CA', 'USA', '93291', 'n****************@att.net', '1943-12-05', 'California', 'Slot', 119.02, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 926518489, 'Ro***', 'Lee', 0, 'Y', '7335 ******** St', 'CHINO', 'CA', 'USA', '91710', 'r*******@att.net', '1952-07-22', 'California', 'Slot', 57.08, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 7313411, 'To**********', 'Gre**', 0, 'Y', '4222 ****** Dr', 'GLENDALE', 'CA', 'USA', '91204', NULL, '1987-08-13', 'California', 'Table', 55.04, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 725214812, 'Mi*****', 'Can**', 0, 'Y', '8907 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89102', NULL, '1964-03-04', 'LasVegas', 'Table', 328.82, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 430817463, 'Th****', 'Kri******', 0, 'Y', '2037 ********* Pl', 'SANTEE', 'CA', 'USA', '92071', NULL, '1961-12-27', 'California', 'Slot', 3857.60, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 533141688, 'Mi**', 'Kim', 0, 'Y', '476 ********* St', 'GLENDORA', 'CA', 'USA', '91741', NULL, '1971-08-21', 'California', 'Table', 58.46, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 48199808, 'Ro****', 'Kyu**', 0, 'Y', '8566 ******* Dr', 'CHINO', 'CA', 'USA', '91710', 'r*********@hotmail.net', '1961-09-28', 'California', 'Slot', 255.88, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 620983164, 'Do***', 'Mor***', 0, 'Y', '3675 ********** Ave', 'LAS VEGAS', 'NV', 'USA', '89144', 'v********@aim.com', '1962-04-12', 'LasVegas', 'Slot', 1017.19, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 748313197, 'Co*******', 'Lim', 0, 'Y', '443 ************ St', 'HENDERSON', 'NV', 'USA', '89052', 'c********@aim.com', '1951-02-04', 'LasVegas', 'Slot', 165.74, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 540353922, 'Je**', 'Bla***', 0, 'Y', '1886 ******** Pl', 'GILBERT', 'AZ', 'USA', '85296', 'j************@cox.net', '1963-07-13', 'California', 'Slot', 3470.31, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 502232662, 'Jo****', 'Pyt***********', 0, 'Y', '7842 ******** Ave', 'SYLMAR', 'CA', 'USA', '91342', 'j************@gmail.com', '1984-11-26', 'California', 'Slot', 33.03, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 471053519, 'Er**', 'Lem**', 0, 'N', '2417 ***** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'm******@sbcglobal.net', '1984-07-04', 'California', 'Slot', 6545.94, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 408696388, 'Ke*****', 'Ngu***', 0, 'N', '8869 ********* Dr', 'FRESNO', 'CA', 'UNK', 'UNKNOWN', 'k*********@gmail.com', '1978-08-17', 'California', 'Slot', 531.71, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 46639459, 'Ry**', 'Pra****', 0, 'Y', '1716 ******** Ave', 'PHOENIX', 'AZ', 'USA', '85048', NULL, '1980-05-20', 'California', 'Table', 4238.23, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 915424159, 'Sc***', 'Bur****', 0, 'Y', '2743 ******* Ct', 'SPARKS', 'NV', 'USA', '89436', 'n**********@verizon.net', '1985-02-25', 'California', 'Table', 94.83, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 232721698, 'Je*****', 'Ell**', 0, 'Y', '4826 ******* Ave', 'LAS VEGAS', 'NV', 'USA', '89117', 'j********@hotmail.com', '1965-01-21', 'LasVegas', 'Slot', 1944.77, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 434787623, 'Te****', 'Col***', 0, 'N', '5380 ********* Ave', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 't********@cox.net', '1966-11-01', 'California', 'Slot', 715.94, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 816999126, 'Do****', 'Mas***', 0, 'Y', '8614 ********** Ct', 'YUMA', 'AZ', 'USA', '85367', NULL, '1962-10-10', 'California', 'Slot', 3149.82, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 119060060, 'Jo**', 'Bro**', 0, 'Y', '3178 ******* Pl', 'PHOENIX', 'AZ', 'USA', '85044', 'j********@gmail.com', '1972-02-09', 'California', 'Slot', 110.00, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 232225964, 'Br*****', 'Moo**', 0, 'Y', '823 ***** Pl', 'INGLEWOOD', 'CA', 'USA', '90305', 's*********@gmail.com', '1956-09-16', 'California', 'Table', 571.58, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 151682905, 'Lo***', 'Yam*****', 0, 'Y', '5703 ************ Dr', 'MESA', 'AZ', 'USA', '85209', 'b*********@outlook.com', '1968-06-20', 'California', 'Slot', 719.32, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 51550800, 'Jo**', 'Mag****', 0, 'Y', '5242 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89135', 'j***************@gmail.com', '1967-05-07', 'LasVegas', 'Slot', 91.06, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 808914966, 'No***', 'Sas*****', 0, 'Y', '1734 ********* Pl', 'FOWLER', 'CA', 'USA', '93625', 'n*********@roadrunner.com', '1962-03-01', 'California', 'Slot', 41.01, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 313600238, 'Ma*****', 'Hug***', 0, 'Y', '5153 ************ St', 'LA QUINTA', 'CA', 'USA', '92253', 'm*********@att.net', '1976-11-01', 'California', 'Slot', 98.61, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 768925514, 'Ba**', 'Wal****', 0, 'Y', '8813 ****** Pl', 'FONTANA', 'CA', 'USA', '92336', 'r************@hotmail.com', '1983-06-03', 'California', 'Slot', 5602.12, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 79661233, 'Jo*****', 'Oxe*****', 0, 'Y', '5620 ********** Ave', 'LOS ANGELES', 'CA', 'USA', '90006', 'j*****@rocketmail.com', '1991-10-05', 'California', 'Slot', 1448.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 98211691, 'Qu*****', 'Arm**', 0, 'Y', '3661 ********* Dr', 'LOS ANGELES', 'CA', 'USA', '90015', 'l************@gmail.com', '1985-04-21', 'California', 'Table', 59.58, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 891088926, 'Ad****', 'Gui******', 0, 'Y', '7923 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89119', 'a*****@sbcglobal.net', '1987-10-21', 'LasVegas', 'Table', 119.20, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 788409804, 'La*****', 'Hef******', 0, 'Y', '3102 ***** Ave', 'LAKE ELSINORE', 'CA', 'USA', '92532', 'l*************@sbcglobal.net', '1976-02-13', 'California', 'Slot', 5255.34, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 146828762, 'Na****', 'Rus*', 0, 'Y', '5927 *********** Dr', 'HENDERSON', 'NV', 'USA', '89012', 'r******@rocketmail.com', '1977-04-15', 'LasVegas', 'Slot', 57.40, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 327303118, 'Ar***', 'Lem**', 0, 'Y', '185 ************ Rd', 'LAS VEGAS', 'NV', 'USA', '89102', 'f*******@aim.com', '1968-05-02', 'LasVegas', 'Table', 310.23, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 812127819, 'Ju**', 'Pie*******', 0, 'Y', '3069 ***** Pl', 'SAN DIEGO', 'CA', 'USA', '92123', 'j*******@mac.com', '1978-02-05', 'California', 'Slot', 310.78, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 892188590, 'Mi*****', 'Del***', 0, 'Y', '3750 ********* Pl', 'YUCAIPA', 'CA', 'USA', '84701', 'm******@aim.com', '1987-04-06', 'California', 'Table', 5643.73, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 901132256, 'Ch*********', 'Van**********', 0, 'Y', '5453 *********** Ave', 'HENDERSON', 'NV', 'USA', '89052', 'c*********@aol.com', '1973-09-06', 'LasVegas', 'Table', 474.95, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 667367352, 'Am****', 'Red**', 0, 'N', '110 ***** Rd', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'p************@hotmail.net', '1992-09-08', 'California', 'Slot', 1041.90, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 31095766, 'Je***', 'Lim', 0, 'Y', '1970 ****** Pl', 'SPARKS', 'NV', 'USA', '89431', 'j*****@yahoo.com', '1980-02-22', 'California', 'Table', 1143.67, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 487321759, 'Je*******', 'Pat******', 0, 'Y', '7905 ********** Rd', 'INDIO', 'CA', 'USA', '92203', 'r********@rocketmail.com', '1939-10-22', 'California', 'Slot', 60.33, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 713172544, 'Wa***', 'Dia*', 0, 'Y', '5589 *********** Rd', 'RENO', 'NV', 'USA', '89511', 'b********@mac.com', '1974-11-13', 'California', 'Table', 37.18, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 751457583, 'Ta*', 'Lam***', 0, 'Y', '2790 ********** St', 'MONTROSE', 'CA', 'USA', '91020', 'e*********@yahoo.com', '1971-11-11', 'California', 'Slot', 36.25, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 859882808, 'Mi*', 'Whi**', 0, 'N', '1033 ******** Rd', 'CERRITTOS', 'CA', 'UNK', 'UNKNOWN', 'c************@cox.net', '1979-08-09', 'California', 'Table', 38.80, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 246231644, 'Jo**', 'Ava****', 0, 'Y', '3906 *********** Pl', 'LONG BEACH', 'CA', 'USA', '90808', 'j***@verizon.net', '1974-08-06', 'California', 'Slot', 56.63, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 39416820, 'Ki******', 'Col****', 0, 'Y', '8493 *********** St', 'SANTA MONICA', 'CA', 'USA', '90404', NULL, '1967-10-22', 'California', 'Slot', 2923.10, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 55953744, 'We***', 'Ray', 0, 'Y', '3786 *********** Ave', 'SAN GABRIEL', 'CA', 'USA', '91776', NULL, '1974-09-26', 'California', 'Table', 47.46, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 235806763, 'Ed***', 'Rei*', 0, 'Y', '8956 ************ Rd', 'Surpriso', 'AZ', 'USA', '85085', 'l*********@netscape.net', '1975-10-12', 'California', 'Slot', 2577.05, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 480770004, 'Lu**', 'Pet**', 0, 'Y', '7267 ************ St', 'TULARE', 'CA', 'USA', '93274', 'l**********@verizon.net', '1956-06-24', 'California', 'Slot', 169.85, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 227344455, 'Mi*****', 'Ahm**', 0, 'N', '741 ******* St', 'POMONA', 'CA', 'UNK', 'UNKNOWN', 'b**************@roadrunner.com', '1985-11-07', 'California', 'Table', 466.47, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 29639972, 'Al****', 'Rom***', 0, 'Y', '4131 ******** Rd', 'LONG BEACH', 'CA', 'USA', '90807', 'a***********@aim.com', '1960-02-04', 'California', 'Slot', 94.73, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 201380023, 'Th****', 'Tel***', 0, 'N', '2000 *********** Ave', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1978-05-12', 'California', 'Slot', 3754.46, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 701911689, 'Ja***', 'Kim', 0, 'Y', '2118 ***** Rd', 'GOODYEAR', 'AZ', 'USA', '85395', 'j***********@mac.com', '1981-07-08', 'California', 'Slot', 72.31, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 832602875, 'Ba****', 'Fan', 0, 'Y', '5542 ***** Rd', 'GLENDALE', 'AZ', 'USA', '85304', '1*********@verizon.net', '1965-07-24', 'California', 'Slot', 6351.21, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 588563196, 'Je***', 'Pro****', 0, 'Y', '8666 ***** St', 'EL CENTRO', 'CA', 'USA', '92243', 'j**********@att.net', '1979-04-19', 'California', 'Slot', 206.77, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 542739108, 'Mi*****', 'Par***', 0, 'Y', '1963 ******* St', 'HENDERSON', 'NV', 'USA', '89052', 'm******@hotmail.com', '1969-01-06', 'LasVegas', 'Slot', 27.41, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 999436554, 'Us*****', 'Tan', 0, 'Y', '6049 ***** St', 'LOS ANGELES', 'CA', 'USA', '90022', 'e**********@gmail.com', '1981-06-06', 'California', 'Slot', 1030.50, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 38439693, 'Er**', 'Mor******', 0, 'Y', '7866 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89135', 'a***********@yahoo.com', '1974-01-08', 'LasVegas', 'Table', 26.22, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 497593899, 'Me****', 'Soz****', 0, 'N', '3915 ******* St', 'LA CRESCENTA', 'CA', 'UNK', 'UNKNOWN', 'm**************@yahoo.com', '1957-05-17', 'California', 'Table', 2454.31, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 326889229, 'Da****', 'Gof*', 0, 'Y', '6136 ******** St', 'LONG BEACH', 'CA', 'USA', '90806', NULL, '1987-08-05', 'California', 'Table', 223.05, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 423362229, 'Ri***', 'Smi***********', 0, 'Y', '7664 ****** Rd', 'VASILIA', 'CA', 'USA', '93291', 'r*********@netscape.net', '1959-06-17', 'California', 'Slot', 2683.83, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 34471196, 'Br***', 'Sas**', 0, 'Y', '9454 *********** Rd', 'RIDGECREST', 'CA', 'USA', '84790', NULL, '1981-07-10', 'California', 'Table', 52.65, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 701006128, 'Jo****', 'Gam****', 0, 'Y', '3937 ************ Ct', 'VALENCIA', 'CA', 'USA', '91354', NULL, '1975-12-03', 'California', 'Table', 1686.82, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 378357889, 'Ju*', 'Sae**', 0, 'Y', '1903 ************ Rd', 'MONTEREY PARK', 'CA', 'USA', '91755', NULL, '1965-09-13', 'California', 'Slot', 147.11, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 302291810, 'Ad******', 'Fig*****', 0, 'N', '963 ******** Dr', 'SEAL BEACH', 'CA', 'UNK', 'UNKNOWN', 't*********@yahoo.com', '1964-12-28', 'California', 'Slot', 1856.22, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 793195136, 'Br***', 'Fat******', 0, 'Y', '5224 ******* Ave', 'VENICE', 'CA', 'USA', '90291', NULL, '1944-07-02', 'California', 'Slot', 52.87, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 978930094, 'Ma**', 'Cha**********', 0, 'Y', '998 ******* Ct', 'HUNTINGTON BEACH', 'CA', 'USA', '90255', 'b******************@att.net', '1964-10-08', 'California', 'Slot', 4151.40, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 75370945, 'Ne****', 'And*****', 0, 'N', '3264 ******** St', 'BAKERSFIELD', 'CA', 'UNK', 'UNKNOWN', 'v**************@aol.com', '1967-04-23', 'California', 'Table', 115.36, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 205888881, 'Wi*****', 'Dic******', 0, 'Y', '787 ***** Rd', 'HESPERIA', 'CA', 'USA', '92345', NULL, '1963-03-06', 'California', 'Slot', 1084.74, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 636652134, 'Je****', 'Atk***', 0, 'Y', '7443 *********** Ave', 'LAVEEN VILLAGE', 'AZ', 'USA', '85339', 'j************@yahoo.com', '1982-02-24', 'California', 'Table', 145.90, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 706390280, 'Jo***', 'Kri******', 0, 'Y', '1564 ****** Ave', 'SANTA ANA', 'CA', 'USA', '92703', 'j**************@verizon.net', '1979-07-27', 'California', 'Slot', 33.42, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 631046111, 'Ja***', 'Vir***', 0, 'Y', '3517 ******* St', 'NORTH HOLLYWOOD', 'CA', 'USA', '91605', 'j***********@hotmail.net', '1982-01-08', 'California', 'Slot', 63.79, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 762434088, 'De****', 'Bra****', 0, 'Y', '5656 ************ Ave', 'TEMECULA', 'CA', 'USA', '92592', 'd********@hotmail.net', '1969-08-08', 'California', 'Slot', 34.53, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 190772736, 'Ts*******', 'Yan*', 0, 'N', '5207 ************ Dr', 'BAKERSFIELD', 'CA', 'UNK', 'UNKNOWN', NULL, '1984-10-25', 'California', 'Both', 668.14, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 307924998, 'Da*', 'Wil***', 0, 'Y', '3872 ********* Ct', 'BUENA PARK', 'CA', 'USA', '90621', 't**************@att.net', '1959-04-03', 'California', 'Slot', 389.67, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 142404814, 'Lu**', 'Col****', 0, 'Y', '4793 ******* St', 'LOS ANGELES', 'CA', 'USA', '90003', 'l***********@att.net', '1972-03-06', 'California', 'Slot', 73.20, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 628507005, 'Ju****', 'Bra****', 0, 'Y', '2105 ******* Dr', 'CANOGA PARK', 'CA', 'USA', '91303', 'j*************@verizon.net', '1992-03-08', 'California', 'Slot', 80.72, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 692393500, 'Cl****', 'Val***', 0, 'Y', '3094 ***** Rd', 'SUN CITY', 'CA', 'USA', '92585', 'c**********@sbcglobal.net', '1957-09-13', 'California', 'Table', 810.86, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 67097222, 'Va*****', 'Sto***', 0, 'Y', '2245 ******** Rd', 'WINCHESTER', 'CA', 'USA', '92596', 'c***************@roadrunner.com', '1979-11-09', 'California', 'Slot', 129.58, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 272274200, 'Al***', 'Roy***', 0, 'Y', '3128 *********** Ct', 'MONTCLAIR', 'CA', 'USA', '91763', 'h*********@gmail.com', '1966-04-07', 'California', 'Slot', 175.97, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 283858991, 'Ch*****', 'Jov***********', 0, 'Y', '1108 *********** St', 'LAS VEGAS', 'NV', 'USA', '89166', 'e*********@gmail.com', '1967-03-04', 'LasVegas', 'Slot', 772.86, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 44418281, 'Ro****', 'Lop**', 0, 'Y', '979 ********* St', 'LAS VEGAS', 'NV', 'USA', '89102', 'r**********@verizon.net', '1963-03-13', 'LasVegas', 'Slot', 1446.93, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 84804950, 'Ci**', 'Red*****', 0, 'N', '5185 ***** Pl', 'CHULA VISTA', 'CA', 'UNK', 'UNKNOWN', 'c***********@rocketmail.com', '1959-01-14', 'California', 'Slot', 730.62, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 5428876, 'Br******', 'Tsa*', 0, 'Y', '8955 ****** Pl', 'GLENDORA', 'CA', 'USA', '91740', NULL, '1965-01-17', 'California', 'Slot', 921.68, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 886726098, 'La***', 'Lim', 0, 'Y', '7658 *********** Dr', 'HUNTINGTON BEACH', 'CA', 'USA', '92647', 's**********@att.net', '1982-06-07', 'California', 'Slot', 1003.65, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 66719355, 'Mi*****', 'Mun**', 0, 'Y', '3751 ***** Ave', 'LOS ANGELES', 'CA', 'USA', '90059', 'm******************@att.net', '1975-03-05', 'California', 'Table', 397.07, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 689763777, 'Lo***', 'Agu****', 0, 'Y', '7242 *********** Dr', 'QUEEN CREEK', 'AZ', 'USA', '85142', 'l*******@netscape.net', '1970-01-23', 'California', 'Slot', 26.13, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 337441347, 'Ki**', 'Rob**', 0, 'Y', '8539 ****** Ct', 'IRVINE', 'CA', 'USA', '92614', 'k*****@rocketmail.com', '1933-12-09', 'California', 'Table', 66.58, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 906690327, 'Ad****', 'Zam***', 0, 'Y', '2210 ****** St', 'RIALTO', 'CA', 'USA', '92377', 'a**************@aim.com', '1985-04-11', 'California', 'Slot', 114.15, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 109549072, 'Je*****', 'Ceb*****', 0, 'Y', '7679 ***** Pl', 'ATWOOD', 'CA', 'USA', '92602', 'j*****@gmail.com', '1966-09-16', 'California', 'Slot', 32.79, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 385271848, 'Da*****', 'Eck***', 0, 'N', '6056 ***** Dr', 'PRESCOTT', 'AZ', 'UNK', 'UNKNOWN', 'd****@hotmail.net', '1966-03-28', 'California', 'Slot', 41.67, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 318859907, 'St****', 'Sch*******', 0, 'Y', '6179 ******** St', 'SILVER SPRINGS', 'NV', 'USA', '89429', 'f**********@cox.net', '1961-11-14', 'California', 'Slot', 177.20, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 931740166, 'Ca******', 'Uri**', 0, 'Y', '1832 ******* Ave', 'MAMMOTH LAKES', 'CA', 'USA', '93546', 'c********@hotmail.net', '1954-01-12', 'California', 'Slot', 42.50, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 69741598, 'Lo**', 'Dul*****', 0, 'Y', '3201 ******** Rd', 'LOS ANGELES', 'CA', 'USA', '90059', 'l****@gmail.com', '1962-11-17', 'California', 'Slot', 44.83, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 226588244, 'Ro****', 'Str***', 0, 'Y', '5975 ******** Pl', 'SCOTTSDALE', 'AZ', 'USA', '85262', 'r***************@mac.com', '1969-11-10', 'California', 'Table', 342.68, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 591388418, 'Me*****', 'Mit*****', 0, 'Y', '2968 ******** Ct', 'SCOTTSDALE', 'AZ', 'USA', '85259', 'm*******@netscape.net', '1967-01-20', 'California', 'Slot', 85.62, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 201473235, 'Le*', 'Hen******', 0, 'Y', '2455 ********** St', 'CORONA', 'CA', 'USA', '92880', 's******@netscape.net', '1981-12-16', 'California', 'Slot', 6555.11, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 949083035, 'La*****', 'Par***', 0, 'Y', '9189 ************ Ave', 'LANCASTER', 'CA', 'USA', '93535', NULL, '1972-03-26', 'California', 'Table', 5388.38, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 18172564, 'Ge******', 'Dom******', 0, 'Y', '8979 ********* Dr', 'LAKE ISABELLA', 'CA', 'USA', '93240', 'm********@att.net', '1958-04-14', 'California', 'Slot', 148.32, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 799275076, 'Li****', 'Par*', 0, 'Y', '920 ***** Rd', 'TUCSON', 'AZ', 'USA', '85756', 'l**************@netscape.net', '1983-09-14', 'California', 'Both', 2126.43, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 158622378, 'Da***', 'Bro**', 0, 'N', '1469 ****** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'c*******@sbcglobal.net', '1956-03-03', 'California', 'Slot', 548.56, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 268598618, 'St*******', 'Far***', 0, 'Y', '4778 ***** Ct', 'SILVERPEAK', 'NV', 'USA', '89047', 'm*************@roadrunner.com', '1962-02-22', 'California', 'Slot', 309.67, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 217522064, 'El***', 'Cal*****', 0, 'N', '1426 ******* Dr', 'ROSEMEAD', 'CA', 'UNK', 'UNKNOWN', 'a**********@att.net', '1982-09-01', 'California', 'Slot', 52.38, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 352215598, 'Ju****', 'Hal*', 0, 'Y', '6940 ********* Ct', 'SAN DIEGO', 'CA', 'USA', '92101', 'j********@netscape.net', '1985-08-27', 'California', 'Table', 399.33, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 17452667, 'Ma******', 'Don**', 0, 'N', '4275 ******* Dr', 'SANTA ANA', 'CA', 'UNK', 'UNKNOWN', 'k******@hotmail.com', '1974-12-20', 'California', 'Slot', 115.57, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 291551044, 'Ar******', 'Hah*', 0, 'Y', '5131 *********** Ave', 'WESTLAKE VILLAGE', 'CA', 'USA', '91361', 'd*********@netscape.net', '1945-02-10', 'California', 'Table', 1593.20, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 886122127, 'Vi******', 'Lin*', 0, 'Y', '5825 *********** Ct', 'CHINO HILLS', 'CA', 'USA', '91709', 'v**********@att.net', '1970-06-07', 'California', 'Slot', 3358.33, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 256077526, 'Lo***', 'Riv***', 0, 'Y', '4529 ********* Pl', 'GOODYEAR', 'AZ', 'USA', '85338', 'l*****************@aim.com', '1987-08-21', 'California', 'Slot', 67.28, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 499024801, 'Ad**', 'Gib***', 0, 'Y', '4007 ********** Dr', 'SANTA BARBARA', 'CA', 'USA', '93101', 'a*********@verizon.net', '1974-07-28', 'California', 'Table', 2501.39, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 329607992, 'Vi******', 'Gar***', 0, 'Y', '4695 ********* Ave', 'PACOIMA', 'CA', 'USA', '91331', NULL, '1959-04-10', 'California', 'Slot', 44.82, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 276167008, 'Br*****', 'Mil***', 0, 'N', '3166 ****** Ave', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'b************@outlook.com', '1986-02-03', 'California', 'Slot', 80.25, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 256998112, 'Su*****', 'Arv***', 0, 'Y', '9457 ******* Dr', 'TUCSON', 'AZ', 'USA', '85715', 's*****@rocketmail.com', '1948-05-26', 'California', 'Slot', 4135.65, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 794886798, 'Ro***', 'Tro***', 0, 'Y', '3043 ******* Ct', 'VICTORVILLE', 'CA', 'USA', '92392', NULL, '1963-01-22', 'California', 'Slot', 353.40, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 932583510, 'Xi*****', 'Kro******', 0, 'Y', '2737 ************ St', 'LAS VEGAS', 'NV', 'USA', '89128', NULL, '1952-12-03', 'LasVegas', 'Table', 44.83, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 393509805, 'An*****', 'Hen******', 0, 'N', '552 ******* Dr', 'INDIAN WELLS', 'CA', 'UNK', 'UNKNOWN', 't*****@aim.com', '1965-09-07', 'California', 'Table', 5417.56, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 722984369, 'Yu*****', 'Far*****', 0, 'Y', '6430 ******** Pl', 'CHULA VISTA', 'CA', 'USA', '91910', NULL, '1983-01-15', 'California', 'Slot', 3981.84, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 769494558, 'Ci**', 'Tho*************', 0, 'Y', '986 ********* Rd', 'LOS ANGELES', 'CA', 'USA', '90065', 'c***********@netscape.net', '1994-08-06', 'California', 'Both', 128.68, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 25516634, 'Ru*****', 'Rot*********', 0, 'Y', '6084 ******** Rd', 'SAN DIEGO', 'CA', 'USA', '92114', 'r********@aim.com', '1953-05-23', 'California', 'Slot', 37.94, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 310436203, 'Re******', 'Dim******', 0, 'Y', '5929 ******** Ave', 'LAS VEGAS', 'NV', 'USA', '89128', 'p********************@cox.net', '1990-04-06', 'LasVegas', 'Table', 228.74, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 180622195, 'Ro****', 'Vic*****', 0, 'Y', '5720 ***** Ct', 'SANTA FE SPRINGS', 'CA', 'USA', '90670', NULL, '1998-06-23', 'California', 'Slot', 3362.03, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 424703377, 'Ju****', 'Boy*', 0, 'Y', '3514 ********** St', 'HENDERSON', 'NV', 'UNK', '650316', 's*******@rocketmail.com', '1987-11-19', 'California', 'Table', 300.58, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 84720478, 'Ed****', 'Gel*********', 0, 'Y', '5579 ********** Ct', 'ESCONDIDO', 'CA', 'USA', '92026', 's****@aim.com', '1968-08-22', 'California', 'Both', 3554.61, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 939288419, 'Os***', 'Gil******', 0, 'Y', '4552 ********** St', 'LAS VEGAS', 'NV', 'USA', '89139', NULL, '1972-12-05', 'LasVegas', 'Slot', 1033.39, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 972762937, 'Ma***', 'Lin', 0, 'Y', '6825 ************ Pl', 'LONG BEACH', 'CA', 'USA', '90807', 't**********@verizon.net', '1969-11-28', 'California', 'Slot', 418.22, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 84610126, 'Th****', 'Nad**', 0, 'N', '8657 *********** Rd', 'PHOENIX', 'AZ', 'UNK', 'UNKNOWN', NULL, '1971-02-23', 'California', 'Both', 655.24, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 679928883, 'Zh*******', 'Chu**', 0, 'N', '1414 ********* Pl', 'HENDERSON', 'NV', 'UNK', 'UNKNOWN', 'z******@aol.com', '1993-05-25', 'California', 'Table', 761.13, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 610681759, 'La***', 'Ola***', 0, 'Y', '103 ***** Pl', 'LONG BEACH', 'CA', 'USA', '90805', 'l*******@sbcglobal.net', '1977-07-28', 'California', 'Slot', 1549.52, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 882273129, 'Ki******', 'Rel*****', 0, 'Y', '7952 ************ Rd', 'SANTA CLARITA', 'CA', 'USA', '91387', 'k*********@roadrunner.com', '1971-12-02', 'California', 'Slot', 70.29, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 495748252, 'De****', 'Bar***', 0, 'Y', '2771 *********** St', 'ALPINE', 'CA', 'USA', '91903', 'd******@cox.net', '1936-05-11', 'California', 'Both', 438.96, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 25565755, 'Te***', 'Ram****', 0, 'Y', '5356 ******** Rd', 'SURPRISE', 'AZ', 'USA', '85388', 't*********@rocketmail.com', '1955-02-07', 'California', 'Slot', 1849.74, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 594796614, 'Lu', 'Ong', 0, 'N', '8013 ******* Dr', 'OXNARD', 'CA', 'UNK', 'UNKNOWN', 'a**********@rocketmail.com', '1981-12-20', 'California', 'Table', 624.34, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 91752467, 'Mi******', 'Dai***', 0, 'Y', '3362 ************ Ave', 'LA MESA', 'AZ', 'USA', '85213', 'm************@cox.net', '1969-02-10', 'California', 'Slot', 25.61, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 6241438, 'An****', 'Vas***********', 0, 'N', '1802 ****** Pl', 'SCOTTSDALE', 'AZ', 'UNK', 'UNKNOWN', 'l************@mac.com', '1971-08-21', 'California', 'Table', 1211.48, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 675746386, 'Pa*****', 'Her******', 0, 'Y', '3420 ******** Pl', 'NATIONAL CITY', 'CA', 'USA', '91951', 'c***********@verizon.net', '1962-01-02', 'California', 'Slot', 3872.67, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 152442297, 'Ro****', 'Laz***', 0, 'Y', '6830 ****** St', 'COMPTON', 'CA', 'USA', '90222', 'r*******@verizon.net', '1981-12-04', 'California', 'Table', 2780.54, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 734817916, 'Oc*******', 'Zhu', 0, 'Y', '8387 *********** Rd', 'OXNARD', 'CA', 'USA', '93033', 'g******@comcast.net', '1977-01-01', 'California', 'Slot', 36.38, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 494067749, 'Ch***', 'Den***', 0, 'Y', '5986 ********** Ave', 'LOS ANGELES', 'CA', 'USA', '90006', 'b*****@roadrunner.com', '1978-04-14', 'California', 'Slot', 1366.08, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 413178221, 'Ly****', 'Pot***', 0, 'Y', '9031 ******* Ct', 'LAKE FOREST', 'CA', 'USA', '92630', 'l***********@yahoo.com', '1976-06-14', 'California', 'Slot', 4998.03, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 952558332, 'Ja***', 'Yar***', 0, 'Y', '8123 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89141', 'j*********@rocketmail.com', '1985-08-13', 'LasVegas', 'Table', 1642.05, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 788809042, 'Ke*****', 'Cha***********', 0, 'Y', '1773 ****** Rd', 'GLENDALE', 'AZ', 'USA', '85302', 'k*********@mac.com', '1955-07-13', 'California', 'Slot', 107.74, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 663381263, 'Ju****', 'Che*', 0, 'Y', '3616 ********* Pl', 'LAS VEGAS', 'NV', 'USA', '89148', 'j********@aol.com', '1998-12-01', 'LasVegas', 'Slot', 44.33, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 141746037, 'Ro*****', 'Kni**', 0, 'Y', '7447 ***** Pl', 'FRESNO', 'CA', 'USA', '84770', 'r*************@netscape.net', '1977-08-10', 'California', 'Table', 2690.82, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 22520648, 'Ar****', 'Pha*', 0, 'N', '4604 ***** St', 'INDIAN WELLS', 'CA', 'UNK', 'UNKNOWN', 'c*******@aim.com', '1953-05-08', 'California', 'Slot', 133.50, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 808126558, 'Fr*******', 'Cal*****', 0, 'N', '4714 ************ St', 'GLENDALE', 'CA', 'UNK', 'UNKNOWN', 't*************@netscape.net', '1970-08-19', 'California', 'Slot', 569.24, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 990245620, 'No****', 'Phi*****', 0, 'Y', '2011 ******** Rd', 'TUCSON', 'AZ', 'USA', '85704', NULL, '1953-12-10', 'California', 'Slot', 5798.76, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 107106749, 'Ni****', 'Gue***', 0, 'Y', '339 ********* Ave', 'BEAUMONT', 'CA', 'USA', '92223', NULL, '1981-12-20', 'California', 'Slot', 72.36, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 277068895, 'Er***', 'Lee', 0, 'Y', '3688 ******* Ave', 'TOLLESON', 'AZ', 'USA', '85353', 'e**************@aol.com', '1990-09-13', 'California', 'Slot', 118.85, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 61551116, 'Ju***', 'Nel***', 0, 'Y', '5008 *********** Dr', 'LOS ANGELES', 'CA', 'USA', '90066', 'j******@hotmail.com', '1968-03-12', 'California', 'Table', 82.85, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 474373743, 'St***', 'Far**', 0, 'Y', '4128 ******* St', 'LAS VEGAS', 'NV', 'USA', '89119', 's**********@verizon.net', '1956-08-18', 'LasVegas', 'Table', 36.90, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 91744094, 'Va*****', 'Her******', 0, 'Y', '766 ***** Rd', 'SAN YSIDRO', 'CA', 'USA', '92173', 'c******@aol.com', '1972-05-21', 'California', 'Slot', 407.48, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 936838711, 'Jo*******', 'Lin****', 0, 'Y', '2817 *********** Rd', 'LAS VEGAS', 'NV', 'USA', '89113', 'j**********@verizon.net', '1976-05-22', 'LasVegas', 'Slot', 42.01, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 567014064, 'De*****', 'Oce*****', 0, 'Y', '2105 ********** Rd', 'HENDERSON', 'NV', 'USA', '89012', 'd******@roadrunner.com', '1968-02-10', 'LasVegas', 'Table', 74.71, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 857209113, 'Um**', 'Chu*', 0, 'N', '4895 *********** Pl', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'u***@verizon.net', '1988-09-19', 'California', 'Slot', 857.05, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 494331692, 'Le*****', 'Mau**********', 0, 'Y', '7045 ****** Pl', 'WHITTIER', 'CA', 'USA', '90606', 'l**********@cox.net', '1966-05-16', 'California', 'Slot', 81.33, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 202567657, 'An**', 'Nak*****', 0, 'Y', '3789 *********** Pl', 'RIVERSIDE', 'CA', 'USA', '92503', NULL, '1959-05-02', 'California', 'Table', 149.70, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 752754942, 'Sh****', 'Rod******', 0, 'N', '2879 ************ St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 's************@comcast.net', '1978-02-20', 'California', 'Table', 1308.71, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 743915907, 'Ch*********', 'And****', 0, 'Y', '231 ********* Rd', 'LAS VEGAS', 'NV', 'USA', '89117', 'o****************@verizon.net', '1984-10-17', 'LasVegas', 'Slot', 799.27, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 727864281, 'Ru***', 'Lem**', 0, 'Y', '2514 *********** Dr', 'OAK VIEW', 'CA', 'USA', '93022', 's*************@outlook.com', '1968-07-18', 'California', 'Slot', 4618.38, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 493179048, 'Al******', 'Ger***', 0, 'Y', '1560 ******** Pl', 'PASADENA', 'CA', 'USA', '91104', 'a**************@hotmail.net', '1972-10-02', 'California', 'Slot', 3158.97, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 19713918, 'Ma****', 'Cel***********', 0, 'Y', '9412 ***** Pl', 'LAKE ISABELLA', 'CA', 'USA', '93240', 'm********@gmail.com', '1969-12-04', 'California', 'Slot', 45.28, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 616729614, 'De****', 'Can***', 0, 'Y', '538 ******* Ct', 'FULLERTON', 'CA', 'USA', '92833', NULL, '1964-12-25', 'California', 'Table', 929.55, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 138211978, 'Ch****', 'Far*', 0, 'Y', '2589 ************ Ct', 'CHINO HILLS', 'CA', 'USA', '91709', NULL, '1995-04-17', 'California', 'Slot', 269.05, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 354880836, 'Va****', 'Yil***', 0, 'Y', '6579 ************ Dr', 'BAKERSFIELD', 'CA', 'USA', '93309', 'x******@verizon.net', '1988-01-07', 'California', 'Slot', 4245.86, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 429887904, 'So**', 'Ada**', 0, 'N', '5382 ****** Pl', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1952-07-19', 'California', 'Both', 5693.61, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 993796437, 'Tw****', 'Jet**', 0, 'Y', '4470 ********* Ct', 'EL MIRAGE', 'AZ', 'USA', '85335', 't*****@att.net', '1963-03-06', 'California', 'Slot', 217.37, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 125696843, 'Al**', 'Joh****', 0, 'Y', '1813 ***** Pl', 'LAS VEGAS', 'NV', 'USA', '89148', 'a*********@comcast.net', '1965-10-02', 'LasVegas', 'Table', 65.50, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 361029420, 'Fa*****', 'Gor**', 0, 'Y', '5296 ********* Pl', 'CALABASAS', 'CA', 'USA', '91302', NULL, '1968-06-22', 'California', 'Table', 307.66, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 156309028, 'La****', 'Rod***************', 0, 'Y', '1544 ******** Dr', 'VENTURA', 'CA', 'USA', '93003', 'l*****@sbcglobal.net', '1969-07-28', 'California', 'Slot', 528.85, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 196328446, 'La***', 'Ham****', 0, 'Y', '603 *********** Dr', 'WEST HOLLYWOOD', 'CA', 'USA', '90069', 'l*******@verizon.net', '1972-10-19', 'California', 'Slot', 58.54, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 20534399, 'De****', 'Gay***', 0, 'Y', '7896 ******* Ct', 'TORRANCE', 'CA', 'USA', '90503', 'r******@comcast.net', '1957-11-05', 'California', 'Table', 146.09, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 394185907, 'Lo****', 'Jhe*', 0, 'Y', '7052 ********* Ave', 'PLAYA VISTA', 'CA', 'USA', '90094', 'm*************@cox.net', '1978-10-11', 'California', 'Table', 1240.79, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 733781749, 'Al******', 'Ham**', 0, 'Y', '3428 ********* St', 'LAS VEGAS', 'NV', 'USA', '89179', 'a******@outlook.com', '1984-11-26', 'LasVegas', 'Table', 251.84, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 241812539, 'Ro****', 'Car***', 0, 'Y', '4012 ************ Pl', 'GRANADA HILLS', 'CA', 'USA', '91344', 'c***********@comcast.net', '1986-04-03', 'California', 'Table', 433.89, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 466956503, 'Je***', 'Don**', 0, 'Y', '7515 ********* Dr', 'ALHAMBRA', 'CA', 'USA', '91802', 'j******@netscape.net', '1984-07-04', 'California', 'Table', 32.71, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 113782448, 'Li*****', 'Hil*', 0, 'Y', '9261 ********* St', 'CAMPO', 'CA', 'USA', '91906', 'l**********@cox.net', '1979-11-06', 'California', 'Slot', 2616.00, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 419498860, 'St*****', 'Lee', 0, 'Y', '8001 ***** Pl', 'HENDERSON', 'NV', 'USA', '89052', 's************@cox.net', '1989-02-18', 'LasVegas', 'Slot', 995.16, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 38744469, 'Bo****', 'Nas********', 0, 'Y', '8920 ******* Dr', 'LOS ANGELES', 'CA', 'USA', '90037', 'b***************@sbcglobal.net', '1973-12-25', 'California', 'Table', 905.51, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 81893353, 'Co**', 'Jay**', 0, 'Y', '7316 ************ Rd', 'YORBA LINDA', 'CA', 'USA', '92886', 'c**********@cox.net', '1978-03-24', 'California', 'Slot', 95.25, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 886902378, 'Ca****', 'Had***', 0, 'Y', '5097 ********** Ave', 'LAS VEGAS', 'NV', 'USA', '89119', 'c**************@gmail.com', '1988-06-17', 'LasVegas', 'Slot', 315.39, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 383642768, 'Ju***', 'Rup***', 0, 'Y', '5605 ****** Dr', 'MAYWOOD', 'CA', 'USA', '90270', 'j*********@mac.com', '1958-02-26', 'California', 'Slot', 28.80, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 72649050, 'Ia*', 'Tak****', 0, 'Y', '5999 ********** Pl', 'ORANGE', 'CA', 'USA', '92867', 'b******@hotmail.com', '1981-05-12', 'California', 'Table', 93.88, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 577982314, 'Pa*****', 'Atk*****', 0, 'Y', '53 ***** Ct', 'WINCHESTER', 'CA', 'USA', '92596', 'p********@hotmail.net', '1969-04-03', 'California', 'Slot', 578.42, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 665294521, 'Ya*****', 'Moo**', 0, 'Y', '2312 ******** Pl', 'LAS VEGAS', 'NV', 'USA', '89117', 'y******@outlook.com', '1957-02-17', 'LasVegas', 'Table', 53.26, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 918281068, 'Ch*********', 'Coo***', 0, 'Y', '2754 ****** Pl', 'GLENDALE', 'AZ', 'USA', '85307', 'c**********@outlook.com', '1970-01-22', 'California', 'Slot', 249.49, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 494686167, 'Ry**', 'For************', 0, 'N', '40 ******* Ct', 'SANTA CLARITA', 'CA', 'UNK', 'UNKNOWN', NULL, '1992-01-02', 'California', 'Table', 175.55, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 452361647, 'Ro****', 'Row*', 0, 'Y', '7239 ******** St', 'PHOENIX', 'AZ', 'USA', '85029', 'c****@gmail.com', '1957-07-01', 'California', 'Slot', 317.25, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 692672219, 'Da****', 'Tay***', 0, 'N', '3179 ********** Dr', 'SANTA ANA', 'CA', 'UNK', 'UNKNOWN', 'f*************@rocketmail.com', '1989-03-27', 'California', 'Table', 1265.60, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 641641460, 'De****', 'Rid**', 0, 'Y', '260 ******* Dr', 'MAMMOTH LAKES', 'CA', 'USA', '93546', 'r********@att.net', '1957-08-23', 'California', 'Slot', 1395.35, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 320292911, 'Be******', 'Alv***********', 0, 'Y', '4711 ******* Dr', 'MISSION VIEJO', 'CA', 'USA', '92692', NULL, '1992-05-23', 'California', 'Table', 40.19, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 906211426, 'Be******', 'Hue***', 0, 'Y', '4682 ************ Pl', 'SAN DIEGO', 'CA', 'USA', '92130', 'g******@aol.com', '1976-08-02', 'California', 'Slot', 50.27, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 84176253, 'Ri***', 'Sou**', 0, 'Y', '4849 ****** Ave', 'SIMI VALLEY', 'CA', 'USA', '93063', 'r************@att.net', '1968-01-27', 'California', 'Slot', 1085.29, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 269212610, 'El*******', 'Lun*', 0, 'Y', '1916 ***** St', 'MANHATTAN BEACH', 'CA', 'USA', '90266', 'r******@gmail.com', '1950-09-12', 'California', 'Slot', 899.02, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 7708513, 'De*****', 'Lim', 0, 'Y', '1411 ****** St', 'BISHOP', 'CA', 'USA', '93514', 'm*************@netscape.net', '1966-04-20', 'California', 'Slot', 710.95, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 217435482, 'Da***', 'Hof*****', 0, 'N', '1362 *********** Dr', 'FOWLER', 'CA', 'UNK', 'UNKNOWN', 'd*****************@hotmail.com', '1948-03-02', 'California', 'Slot', 46.55, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 918850199, 'Ya***', 'Led*****', 0, 'Y', '4125 ********** Ct', 'ENCINO', 'CA', 'USA', '91436', 'y**********@cox.net', '1976-03-21', 'California', 'Table', 2940.40, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 151576956, 'Sa****', 'Sex***', 0, 'Y', '1096 ********* Ct', 'SAN BERNADINO', 'CA', 'USA', '92408', NULL, '1976-11-27', 'California', 'Slot', 3831.46, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 940980549, 'Ja***', 'Bre***', 0, 'Y', '5332 *********** Rd', 'YUCAIPA', 'CA', 'USA', '92399', 'j******************@outlook.com', '1970-09-13', 'California', 'Slot', 43.20, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 296563569, 'Bl****', 'Kup****', 0, 'Y', '3126 ***** Pl', 'SAN YSIDRO', 'CA', 'USA', '92173', NULL, '1953-01-27', 'California', 'Slot', 182.78, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 17906095, 'Ch**', 'Hyt**', 0, 'Y', '1491 ******** Pl', 'FRESNO', 'CA', 'USA', '93727', 's*********@verizon.net', '1980-03-20', 'California', 'Both', 361.80, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000284017, 'Ro****', 'Jac***', 0, 'Y', '6506 ******* Dr', 'BLUE JAY', 'CA', 'USA', '92317', 'r***************@netscape.net', '1978-09-14', 'California', 'Slot', 75.51, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000309753, 'Ro****', 'Ave*****', 0, 'Y', '735 ************ Ave', 'PRESCOTT', 'AZ', 'USA', '86315', 'a**********@rocketmail.com', '1982-05-17', 'California', 'Slot', 1665.87, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000318295, 'An****', 'Man****', 0, 'Y', '422 ******* Dr', 'GILBERT', 'AZ', 'USA', '85296', 'a********@hotmail.com', '1975-12-07', 'California', 'Slot', 2758.38, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 6000043500, 'Ab***', 'Pau****', 0, 'Y', '1742 ***** Pl', 'NORTH LAS VEGAS', 'NV', 'USA', '89030', 'a*************@roadrunner.com', '1998-02-14', 'LasVegas', 'Table', 1138.42, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000314130, 'Ja****', 'Ker*', 0, 'Y', '7068 ****** Ave', 'LAS VEGAS', 'NV', 'USA', '89120', 'j*****@mac.com', '1982-06-20', 'LasVegas', 'Slot', 352.41, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10063081, 'Al***', 'Ham**', 0, 'Y', '935 ******** Rd', 'BREA', 'CA', 'USA', '92821', 'a**********@att.net', '1977-07-20', 'California', 'Slot', 5298.08, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10204678, 'Do*****', 'Jon**', 0, 'Y', '89 *********** Ct', 'SAN DIEGO', 'CA', 'USA', '92123', 'd*******@gmail.com', '1978-02-20', 'California', 'Slot', 2719.49, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10239915, 'Ti****', 'Bal********', 0, 'Y', '8840 ******* St', 'HACIENDA HEIGHTS', 'CA', 'USA', '91745', 't*********@mac.com', '1978-09-19', 'California', 'Table', 99.60, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10555672, 'Er***', 'Lar*******', 0, 'Y', '497 *********** Ct', 'PHOENIX', 'AZ', 'USA', '85028', 'p*******@outlook.com', '1979-04-16', 'California', 'Slot', 482.17, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10348708, 'St***', 'Xio**', 0, 'Y', '9165 ******* Ave', 'CALABASAS', 'CA', 'USA', '91302', 's********@yahoo.com', '1974-02-21', 'California', 'Table', 34.43, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10507443, 'Hr***', 'Wil***', 0, 'Y', '7792 ******* Ct', '"SUN VALLEY "', 'CA', 'USA', '91352', NULL, '1994-03-24', 'California', 'Table', 6512.64, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10550929, 'Ca*****', 'Whi**', 0, 'Y', '6572 ***** Dr', 'LAS VEGAS', 'NV', 'USA', '89178', NULL, '1964-09-02', 'LasVegas', 'Table', 106.08, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10518028, 'Je*****', 'Kel***', 0, 'Y', '1390 ***** Rd', 'LA MESA', 'AZ', 'USA', '85207', NULL, '1986-02-11', 'California', 'Slot', 127.21, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10522579, 'Az**', 'Her******', 0, 'Y', '9341 ********* St', 'NORTHRIDGE', 'CA', 'USA', '91325', 'a***@yahoo.com', '1956-10-07', 'California', 'Table', 374.39, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10536110, 'Su***', 'Jon**', 0, 'Y', '5925 *********** Rd', 'SOUTH GATE', 'CA', 'USA', '90280', 's*********@yahoo.com', '1967-10-27', 'California', 'Table', 2551.41, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10557075, 'La***', 'Kim', 0, 'Y', '1483 *********** Rd', 'SANTA ANA', 'CA', 'USA', '92703', 'l************@sbcglobal.net', '1961-03-25', 'California', 'Slot', 508.62, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10559333, 'Sh***', 'Cal****', 0, 'N', '3653 ********** Ave', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 's*********@outlook.com', '1955-08-05', 'California', 'Table', 2984.84, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10575646, 'Ed****', 'Shi****', 0, 'Y', '6304 ********* Dr', 'HESPERIA', 'CA', 'USA', '92345', NULL, '1981-04-02', 'California', 'Slot', 114.45, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000218401, 'Lu***', 'Bea**', 0, 'Y', '1351 *********** Rd', 'QUEEN CREEK', 'AZ', 'USA', '85142', 'l**************@sbcglobal.net', '1991-05-12', 'California', 'Slot', 1132.86, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000229195, 'Ca****', 'Suh', 0, 'N', '7725 *********** Dr', 'GILBERT', 'AZ', 'UNK', 'UNKNOWN', 'c**********@netscape.net', '1998-10-07', 'California', 'Table', 1387.28, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000241879, 'Ni****', 'Abr******', 0, 'N', '4242 ********** Dr', 'CARSON', 'CA', 'UNK', 'UNKNOWN', 'k*********@yahoo.com', '1983-08-05', 'California', 'Slot', 1172.74, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000301504, 'Da***', 'Cai', 0, 'N', '6933 ******** Ct', 'LONG BEACH', 'CA', 'UNK', 'UNKNOWN', 'd*****@aim.com', '1967-02-12', 'California', 'Slot', 5751.98, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10110931, 'Fa****', 'Nov****', 0, 'Y', '9469 ********* Pl', 'LOS ANGELES', 'CA', 'USA', '90008', 's***********@rocketmail.com', '1984-01-19', 'California', 'Table', 102.60, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10235004, 'Gr****', 'Jah**************', 0, 'Y', '5727 ********* Dr', '"SUN VALLEY "', 'CA', 'USA', '91352', NULL, '1974-04-05', 'California', 'Table', 882.98, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10354600, 'Sa***', 'Oma*', 0, 'Y', '7402 ********* Rd', 'LAS VEGAS', 'NV', 'USA', '89117', 'a**********@verizon.net', '1967-12-03', 'LasVegas', 'Table', 28.81, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10561818, 'Ju***********', 'Kin*', 0, 'N', '429 ******** Ave', 'LYNWOOD', 'CA', 'UNK', 'UNKNOWN', NULL, '1971-10-01', 'California', 'Table', 101.25, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10112846, 'Ho*', 'Tho***', 0, 'Y', '4867 ***** Dr', 'SAN MARINO', 'CA', 'USA', '91108', 'f***********@aol.com', '1958-12-15', 'California', 'Table', 39.54, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10158087, 'Pa****', 'Mad***', 0, 'Y', '8681 ****** Rd', 'MOORPARK', 'CA', 'USA', '93021', NULL, '1964-01-10', 'California', 'Slot', 160.79, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10287135, 'Ju***', 'Wal***', 0, 'Y', '7187 ************ Rd', 'NEW RIVER', 'AZ', 'USA', '85087', 'j********@att.net', '1969-07-17', 'California', 'Slot', 1895.24, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10297147, 'Sa***', 'San****', 0, 'Y', '3181 ********* Pl', 'SANTA FE SPRINGS', 'CA', 'USA', '90670', 's*******@roadrunner.com', '1978-01-11', 'California', 'Table', 73.97, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10317119, 'Sh******', 'Ari**', 0, 'Y', '7135 ********** Ave', 'ONTARIO', 'CA', 'UNK', '300221', NULL, '1982-09-21', 'California', 'Table', 1774.55, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10303802, 'Je******', 'Gem****', 0, 'Y', '6017 ******** Dr', 'POMONA', 'CA', 'USA', '84790', 'f*************@comcast.net', '1972-10-19', 'California', 'Slot', 1144.13, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000315773, 'Hu*****', 'Col****', 0, 'Y', '7726 ******** Ct', 'LAS VEGAS', 'NV', 'USA', '89122', 'h****************@hotmail.com', '1987-03-27', 'LasVegas', 'Table', 946.53, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10118381, 'Fr*******', 'Mkr******', 0, 'Y', '2110 ****** Ave', 'SAN BERNADINO', 'CA', 'USA', '92407', 'c*******@comcast.net', '1983-10-01', 'California', 'Table', 257.08, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10287981, 'Di**', 'Lee', 0, 'Y', '476 ************ Rd', 'BULLHEAD CITY', 'AZ', 'USA', '86442', 'f*******@comcast.net', '1965-04-12', 'California', 'Slot', 3051.23, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10371527, 'Ta****', 'Kan*', 0, 'Y', '361 ******** Rd', 'MOJAVE', 'CA', 'USA', '93501', NULL, '1989-10-06', 'California', 'Slot', 40.88, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10567786, 'Ar****', 'Ber****', 0, 'Y', '7705 ********** Ave', 'TEMECULA', 'CA', 'USA', '92592', 't*******@roadrunner.com', '1967-07-14', 'California', 'Slot', 1751.80, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10573368, 'Ke***', 'Wol**', 0, 'Y', '123 *********** Pl', 'TUCSON', 'AZ', 'USA', '85711', 'm***********@rocketmail.com', '1961-02-16', 'California', 'Slot', 1397.86, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 100674031, 'Mi*****', 'Ara*', 0, 'Y', '7827 ********* Ave', 'SPARKS', 'NV', 'USA', '89436', 'm***************@comcast.net', '1958-08-01', 'California', 'Slot', 644.26, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 101520524, 'Li***', 'Jin', 0, 'Y', '3309 ************ Ct', 'LAS VEGAS', 'NV', 'USA', '89109', 'l********@att.net', '1963-10-03', 'LasVegas', 'Slot', 1325.04, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 943940673, 'Ri*****', 'Zha**', 0, 'Y', '1459 ***** Dr', 'PACOIMA', 'CA', 'USA', '91331', 'a********@gmail.com', '1977-09-04', 'California', 'Slot', 1025.46, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000053106, 'Da**', 'Swe****', 0, 'Y', '5720 ******* Rd', 'SANTA BARBARA', 'CA', 'USA', '93109', NULL, '1948-11-07', 'California', 'Slot', 265.50, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 115595821, 'Jo***', 'Man******', 0, 'Y', '108 ****** Ct', 'LOS ANGELES', 'CA', 'USA', '90002', 'l**********@aim.com', '1955-01-15', 'California', 'Slot', 35.98, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 570339765, 'Ko****', 'Arr******', 0, 'Y', '4933 *********** St', 'PHOENIX', 'AZ', 'USA', '85032', 'k***********@comcast.net', '1981-03-04', 'California', 'Slot', 5854.70, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200389159, 'Jo***', 'Cha****', 0, 'Y', '9157 *********** St', 'LAS VEGAS', 'NV', 'USA', '89135', 'j***********@gmail.com', '1957-04-27', 'LasVegas', 'Slot', 42.73, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000026061, 'An****', 'Bro**', 0, 'Y', '6192 ***** St', 'ADELANTO', 'CA', 'USA', '92301', NULL, '1965-12-09', 'California', 'Slot', 6532.21, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000062234, 'Ju***', 'Olg***', 0, 'Y', '3150 ******** Pl', 'APPLE VALLEY', 'CA', 'USA', '92308', NULL, '1969-12-04', 'California', 'Slot', 751.00, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000065860, 'Ci***', 'Hob**', 0, 'N', '749 *********** Pl', 'GRANADA HILLS', 'CA', 'UNK', 'UNKNOWN', 'c*******@gmail.com', '1977-12-03', 'California', 'Slot', 3613.02, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000140923, 'Ka***', 'Pic****', 0, 'Y', '2553 *********** Pl', 'CARSON', 'CA', 'USA', '90746', 'f******@hotmail.com', '1995-11-22', 'California', 'Table', 96.81, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10195857, 'Ch******', 'Mer****', 0, 'Y', '452 *********** Ct', 'CLOVIS', 'CA', 'USA', '93612', NULL, '1964-08-18', 'California', 'Table', 81.24, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10568167, 'Ga***', 'Her****', 0, 'N', '7916 ********** Pl', 'FONTANA', 'CA', 'UNK', 'UNKNOWN', 'g*************@yahoo.com', '1967-03-23', 'California', 'Table', 289.08, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10179607, 'Ch***', 'Mor****', 0, 'N', '2007 ************ Rd', 'SAN DIEGO', 'CA', 'UNK', 'UNKNOWN', 'c*********@hotmail.net', '1954-08-01', 'California', 'Table', 31.43, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10207599, 'Mi**', 'Ble****', 0, 'Y', '2294 ************ Ct', 'ANAHEIM', 'CA', 'USA', '92806', 'm***@aim.com', '1978-07-01', 'California', 'Slot', 759.07, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10381132, 'Ma***', 'Dra**', 0, 'Y', '6046 ********** St', 'LOS ANGELES', 'CA', 'USA', '90037', 'h***************@netscape.net', '1956-03-11', 'California', 'Slot', 516.98, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 212451278, 'Fi***', 'Str****', 0, 'N', '1934 *********** St', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'f*********@aol.com', '1958-02-03', 'California', 'Slot', 49.97, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 277787384, 'La*****', 'Gre**', 0, 'Y', '5820 ************ Dr', 'LANCASTER', 'CA', 'USA', '93536', 'l*******@sbcglobal.net', '1955-10-21', 'California', 'Slot', 68.42, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000084715, 'Ra*****', 'Tho***', 0, 'Y', '5626 *********** Pl', 'FULLERTON', 'CA', 'USA', '92831', 'r*****@att.net', '1955-01-07', 'California', 'Slot', 2083.49, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000157640, 'Wi****', 'Mar***', 0, 'Y', '2091 ********* Pl', 'SCOTTSDALE', 'AZ', 'USA', '85254', 'w**************@comcast.net', '1969-03-11', 'California', 'Slot', 2216.76, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000143338, 'Sa***', 'Mon*******', 0, 'N', '8920 ****** Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'p*********@outlook.com', '1935-03-01', 'California', 'Slot', 2443.04, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000145628, 'Ma**', 'Pin***', 0, 'Y', '460 ****** Ct', 'LAS VEGAS', 'NV', 'USA', '89148', 'h*********@rocketmail.com', '1985-12-09', 'LasVegas', 'Table', 27.03, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000165016, 'Ni******', 'May*****', 0, 'Y', '1367 ******** Pl', 'HENDERSON', 'NV', 'USA', '89011', 'p**********@mac.com', '1987-03-05', 'LasVegas', 'Table', 40.10, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000211577, 'My', 'Yan*', 0, 'Y', '1837 ****** Ct', 'WESTMINSTER', 'CA', 'USA', '92683', 'm***********@rocketmail.com', '1984-09-16', 'California', 'Table', 2503.64, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000251676, 'Pa****', 'Tol******', 0, 'Y', '6620 ******* Pl', 'HENDERSON', 'NV', 'USA', '89015', NULL, '1991-08-12', 'LasVegas', 'Table', 212.15, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000255672, 'Yo**', 'Her******', 0, 'Y', '3533 ******* Rd', 'CERRITTOS', 'CA', 'USA', '90703', 'y*********@gmail.com', '1967-01-21', 'California', 'Table', 1588.43, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000275527, 'Do*****', 'Ewe*', 0, 'Y', '2057 ******* Rd', 'PHOENIX', 'AZ', 'USA', '85031', 'd********@outlook.com', '1986-05-11', 'California', 'Slot', 1934.69, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000250435, 'Jo***', 'Kir***', 0, 'Y', '1481 *********** St', 'LOS ANGELES', 'CA', 'USA', '90044', 'h************@hotmail.net', '1977-10-10', 'California', 'Slot', 66.83, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000317084, 'Ca****', 'Nic******', 0, 'Y', '8286 ********** Ave', 'LAS VEGAS', 'NV', 'USA', '89129', 'h**************@rocketmail.com', '1996-06-16', 'LasVegas', 'Slot', 74.75, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000253864, 'Fr*******', 'Bel*****', 0, 'Y', '896 ********* Pl', 'GARDENA', 'CA', 'USA', '90248', 'f****@aol.com', '1969-11-07', 'California', 'Table', 61.81, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000316734, 'Sh****', 'Pri**', 0, 'Y', '3755 ********* St', 'HENDERSON', 'NV', 'USA', '89052', 's****@sbcglobal.net', '1965-11-16', 'LasVegas', 'Table', 138.48, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10311377, 'To**', 'Hil*', 0, 'Y', '7492 ******** Ct', 'LAS VEGAS', 'NV', 'USA', '89117', NULL, '1977-04-12', 'LasVegas', 'Table', 1220.60, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000036497, 'Ju***', 'How***', 0, 'Y', '2102 ****** Pl', 'SAN DIEGO', 'CA', 'USA', '92111', NULL, '1961-06-13', 'California', 'Slot', 694.02, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000066828, 'Ni******', 'Ngu***', 0, 'Y', '4810 ********* Rd', 'SUN CITY', 'AZ', 'USA', '85373', NULL, '1966-10-22', 'California', 'Table', 5979.55, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000071711, 'Je******', 'Mir***************', 0, 'Y', '5317 ******* Ave', 'BUENA PARK', 'CA', 'USA', '90620', NULL, '1978-04-04', 'California', 'Slot', 2936.87, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000076222, 'Me*****', 'Nam', 0, 'Y', '7370 ******** Dr', 'PHOENIX', 'AZ', 'USA', '85048', 'm**********@mac.com', '1949-05-15', 'California', 'Slot', 45.81, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000058260, 'An*****', 'Ton***', 0, 'Y', '3921 *********** Dr', 'SAN DIEGO', 'CA', 'USA', '92139', 'i*******@att.net', '1965-07-09', 'California', 'Slot', 174.48, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000157839, 'Bi**', 'Fer*****', 0, 'Y', '7309 ********* Ave', 'LAS VEGAS', 'NV', 'USA', '89123', 'l**********@cox.net', '1954-09-09', 'LasVegas', 'Table', 146.63, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000137866, 'Ja***', 'Whi********', 0, 'Y', '8651 ***** Dr', 'RANCHO CUCAMONGA', 'CA', 'USA', '91730', 'j******@aim.com', '1981-11-19', 'California', 'Table', 110.11, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000146316, 'Wo*****', 'Hui***', 0, 'N', '4917 ************ Ave', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1985-08-13', 'California', 'Table', 39.57, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000123379, 'Da****', 'Faz*', 0, 'Y', '6744 ********** Rd', 'LOMA LINDA', 'CA', 'USA', '92354', 'd**************@hotmail.net', '1964-12-10', 'California', 'Slot', 165.67, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000154497, 'An****', 'Sar******', 0, 'Y', '4762 ********** Ave', 'APACHE JUNCTION', 'AZ', 'USA', '85119', 'a***************@outlook.com', '1968-05-19', 'California', 'Slot', 2377.01, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000161965, 'Ti***', 'Den***', 0, 'Y', '2970 ********** Ave', 'CARLSBAD', 'CA', 'USA', '92009', 'l******@gmail.com', '1981-01-12', 'California', 'Slot', 40.17, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000169127, 'Je****', 'Pug*****', 0, 'Y', '6968 ********* Rd', 'NORWALK', 'CA', 'USA', '90650', 'k*********@yahoo.com', '1975-06-22', 'California', 'Slot', 217.46, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000179589, 'Ch******', 'Hec*************', 0, 'Y', '5329 ******* St', 'ROSEMEAD', 'CA', 'USA', '91770', '2********@roadrunner.com', '1989-02-27', 'California', 'Table', 50.96, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000225941, 'Ty***', 'Rey*****', 0, 'Y', '1378 ****** Rd', 'WEST HOLLYWOOD', 'CA', 'USA', '90069', 't*******@outlook.com', '1990-07-17', 'California', 'Table', 27.37, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000237989, 'Je****', 'Buc*****', 0, 'Y', '8919 ***** Pl', 'LOS ANGELES', 'CA', 'USA', '90049', 'b*********@sbcglobal.net', '1971-11-23', 'California', 'Slot', 525.47, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000257319, 'Ka***', 'Lin***', 0, 'Y', '9195 ***** Ct', 'LAS VEGAS', 'NV', 'USA', '89147', 'b********************@comcast.net', '1972-10-14', 'LasVegas', 'Slot', 1382.43, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10333987, 'Ha***', 'Tho***', 0, 'Y', '9368 *********** Ct', 'GILBERT', 'AZ', 'USA', '85295', 'j*********@gmail.com', '1968-06-09', 'California', 'Table', 2768.61, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 100721093, 'Ge**', 'Jeo*', 0, 'N', '3775 ************ Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'g******@sbcglobal.net', '1973-04-17', 'California', 'Table', 400.33, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 101382071, 'Li**', 'Isa******', 0, 'N', '7836 ********* Ave', 'PICO RIVERA', 'CA', 'UNK', 'UNKNOWN', 'l*********@roadrunner.com', '1972-10-03', 'California', 'Slot', 115.64, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000011256, 'St****', 'Mca******', 0, 'Y', '3256 ***** Dr', 'DEL MAR', 'CA', 'USA', '92014', NULL, '1989-05-21', 'California', 'Table', 42.84, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000012951, 'Vi****', 'Bar***', 0, 'Y', '4116 ******** Ave', 'TUCSON', 'AZ', 'USA', '85746', 'v******@yahoo.com', '1993-12-07', 'California', 'Table', 42.50, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000177385, 'To**', 'Str*****', 0, 'Y', '4613 ********* Rd', 'LAS VEGAS', 'NV', 'USA', '89108', 'm****************@mac.com', '1984-12-10', 'LasVegas', 'Table', 361.95, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000194410, 'Lu**', 'Lin', 0, 'Y', '6799 ************ Dr', 'LOS ANGELES', 'CA', 'USA', '90007', NULL, '1983-07-21', 'California', 'Table', 2760.59, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000307634, 'Mi***', 'Ben****', 0, 'Y', '2199 ****** Rd', 'SAN YSIDRO', 'CA', 'USA', '92173', 'm*************@mac.com', '1954-02-17', 'California', 'Slot', 126.32, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10066059, 'Jo****', 'Ber*', 0, 'Y', '1977 *********** Ave', 'DOWNIEVILLE', 'CA', 'USA', '90241', 'j***********@verizon.net', '1981-02-07', 'California', 'Slot', 61.86, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000201466, 'Fe******', 'Men****', 0, 'Y', '2466 ************ Rd', 'LA PUENTE', 'CA', 'USA', '91744', NULL, '1989-03-02', 'California', 'Table', 111.80, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000210162, 'Sa*****', 'Kee**', 0, 'Y', '1443 ********* Ave', 'DUARTE', 'CA', 'USA', '91010', 's*********@aol.com', '1979-09-22', 'California', 'Table', 5155.92, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000281257, 'Ju***', 'Esp*****', 0, 'Y', '3109 ***** Dr', 'NORTH LAS VEGAS', 'NV', 'USA', '89032', 'c*****************@comcast.net', '1967-02-14', 'LasVegas', 'Slot', 135.25, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10455970, 'Ja***', 'Gla**', 0, 'N', '2513 ******** Dr', 'RENO', 'NV', 'UNK', 'UNKNOWN', NULL, '1978-05-27', 'California', 'Table', 109.62, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10463173, 'Jo**', 'For***', 0, 'Y', '2360 ****** St', 'LAS VEGAS', 'NV', 'USA', '89144', 'j*******@mac.com', '1978-04-11', 'LasVegas', 'Slot', 71.84, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10504251, 'Jo**', 'Dia*', 0, 'Y', '5590 ************ Ave', 'CHANDLER', 'AZ', 'USA', '85225', 's***************@gmail.com', '1960-02-23', 'California', 'Slot', 582.48, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10525314, 'Jo*****', 'Mck***', 0, 'Y', '9472 ******** Rd', 'IMPERIAL', 'CA', 'USA', '84713', 'm**************@verizon.net', '1967-12-08', 'California', 'Slot', 2322.04, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10550807, 'An***', 'Rey**', 0, 'Y', '1148 ********** Ct', 'TUCSON', 'AZ', 'USA', '85716', 'g*********@hotmail.net', '1947-06-02', 'California', 'Slot', 29.33, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10561896, 'Sa*******', 'Zav***', 0, 'Y', '7708 ***** Dr', 'RIDGECREST', 'CA', 'USA', '93555', 'j**************@comcast.net', '1940-01-24', 'California', 'Slot', 47.70, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10529221, 'Ju***', 'Edw****', 0, 'Y', '1916 *********** St', 'FRESNO', 'CA', 'USA', '93722', NULL, '1957-11-23', 'California', 'Slot', 953.46, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 101547998, 'Ra***', 'Car**********', 0, 'N', '442 *********** St', 'ROSEMEAD', 'CA', 'UNK', 'UNKNOWN', 's**************@aim.com', '1960-06-13', 'California', 'Slot', 1545.96, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10573403, 'Jo****', 'Cho******', 0, 'Y', '8948 ***** St', 'CHATSWORTH', 'CA', 'USA', '91311', 'j**********@rocketmail.com', '1985-03-19', 'California', 'Slot', 1441.01, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10579210, 'An*****', 'Rob**************', 0, 'N', '1483 ********** Ct', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 't*************@rocketmail.com', '1969-12-06', 'California', 'Table', 51.83, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 476005807, 'Br****', 'Mat***', 0, 'Y', '9052 ********** Pl', 'PINEDALE', 'CA', 'USA', '93650', 'b*****************@sbcglobal.net', '1953-10-27', 'California', 'Slot', 528.64, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000076511, 'Ir**', 'Hem******', 0, 'Y', '2026 *********** St', 'SAN PEDRO', 'CA', 'USA', '90731', NULL, '1958-09-21', 'California', 'Table', 113.85, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000116428, 'Jo****', 'Hog**', 0, 'N', '8724 ********** St', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'd**************@verizon.net', '1996-01-18', 'California', 'Both', 205.98, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000248204, 'Mi*****', 'Har***', 0, 'Y', '4817 ********** Pl', 'TORRANCE', 'CA', 'USA', '90502', 'm*******@verizon.net', '1989-06-19', 'California', 'Table', 4511.57, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000264182, 'Jo**', 'Sti***', 0, 'N', '8586 ********** Rd', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1968-01-13', 'California', 'Table', 2112.86, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000281898, 'Ch*****', 'Tol******', 0, 'Y', '5450 *********** Ct', 'WINNETKA', 'CA', 'USA', '91306', 'c*******@hotmail.net', '1992-12-17', 'California', 'Slot', 1029.47, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10047155, 'Mi*****', 'Due***', 0, 'Y', '2153 *********** Ct', 'WEST COVINA', 'CA', 'USA', '91792', 'm********@cox.net', '1957-10-11', 'California', 'Table', 5343.31, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10195456, 'Da****', 'Alc******', 0, 'Y', '2055 ********** Ct', 'ALHAMBRA', 'CA', 'UNK', 'DA26FL', NULL, '1981-01-18', 'California', 'Table', 1601.99, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10231663, 'Re******', 'Son', 0, 'Y', '4478 ******* Ave', 'POMONA', 'CA', 'USA', '91766', 'm***********@rocketmail.com', '1944-05-02', 'California', 'Slot', 164.19, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10314388, 'Je***', 'Sco**', 0, 'Y', '3511 ******* Rd', 'ORANGE', 'CA', 'USA', '92868', 'l************@cox.net', '1949-08-04', 'California', 'Slot', 79.89, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10237222, 'Ra****', 'Por********', 0, 'Y', '6619 ********* St', 'CHATSWORTH', 'CA', 'USA', '91311', 'r************@rocketmail.com', '1941-02-27', 'California', 'Slot', 158.35, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10242638, 'De****', 'Ros**', 0, 'Y', '4615 ******* Ave', 'LOS ANGELES', 'CA', 'USA', '90044', NULL, '1964-01-02', 'California', 'Slot', 753.48, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10445609, 'Ja***', 'Edm****', 0, 'Y', '3692 ********* St', 'SAN DIEGO', 'CA', 'USA', '92109', '7******@aim.com', '1971-12-16', 'California', 'Slot', 766.32, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10514986, 'Ja***', 'Rep***', 0, 'Y', '62 ******* Ave', 'LAS VEGAS', 'NV', 'USA', '89148', NULL, '1962-11-27', 'LasVegas', 'Table', 439.18, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 436671431, 'Qi*', 'Chu**', 0, 'Y', '4468 ********** St', 'LAS VEGAS', 'NV', 'USA', '89148', NULL, '1970-08-07', 'LasVegas', 'Table', 1703.94, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 100332741, 'Os***', 'Riv***', 0, 'Y', '4360 ***** Ave', 'RENO', 'NV', 'USA', '89506', 'o*********@cox.net', '1948-01-07', 'California', 'Slot', 262.20, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10571617, 'Pa****', 'Che*', 0, 'Y', '3561 ********* Pl', 'LA MESA', 'AZ', 'USA', '85203', 'p********@hotmail.net', '1969-03-08', 'California', 'Slot', 115.77, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 100660896, 'Da*', 'Con***', 0, 'Y', '430 ****** Ave', 'LITCHFIELD PARK', 'AZ', 'USA', '85340', NULL, '1963-09-21', 'California', 'Both', 418.89, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 100673096, 'Jo**', 'Ngh***', 0, 'Y', '8691 ***** Rd', 'SPARKS', 'NV', 'USA', '89431', 'j*************@roadrunner.com', '1975-03-10', 'California', 'Slot', 25.12, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 200291187, 'Mi*****', 'Lop**', 0, 'Y', '6801 ********* Ave', 'CARLSBAD', 'CA', 'USA', '92008', 'm*************@gmail.com', '1957-04-07', 'California', 'Table', 6258.53, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200335927, 'St*******', 'Ver*', 0, 'Y', '2987 *********** Dr', 'LAS VEGAS', 'NV', 'USA', '89183', 's****************@gmail.com', '1964-08-04', 'LasVegas', 'Slot', 433.03, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000034350, 'Va****', 'Gra**', 0, 'Y', '7086 ****** Pl', 'GLENDALE', 'CA', 'USA', '91206', NULL, '1951-03-19', 'California', 'Slot', 221.49, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200428533, 'He***', 'Wes****', 0, 'Y', '2760 *********** Rd', 'SPRING CREEK', 'NV', 'USA', '89815', 'h******@comcast.net', '1965-12-16', 'California', 'Slot', 243.54, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 313597828, 'Ed****', 'Zha**', 0, 'Y', '2618 ********** Dr', 'LEMORE', 'CA', 'USA', '93245', 'e************@netscape.net', '1969-07-03', 'California', 'Table', 101.15, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 659347439, 'Ap********', 'Wan*', 0, 'Y', '9077 ************ Ct', 'ANTHEM', 'AZ', 'USA', '85086', 'a******************@comcast.net', '1974-12-25', 'California', 'Slot', 133.01, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 893690021, 'An****', 'Ola**', 0, 'Y', '3901 ********** Ct', 'POMONA', 'CA', 'USA', '91766', 'n********@rocketmail.com', '1980-05-12', 'California', 'Slot', 311.87, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000003410, 'Gi**', 'Peg***', 0, 'Y', '8212 ***** Ct', 'SAN GABRIEL', 'CA', 'USA', '91778', 'g**********@verizon.net', '1991-09-23', 'California', 'Table', 2705.93, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000040347, 'Jo**', 'Ash*****', 0, 'Y', '8920 *********** Rd', 'ENCINO', 'CA', 'USA', '91316', 'j******@gmail.com', '1968-07-05', 'California', 'Table', 371.15, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000051665, 'Ro*****', 'Ruv*************', 0, 'Y', '8771 ***** Pl', 'MISSION VIEJO', 'CA', 'USA', '92691', 'r*************@yahoo.com', '1965-04-20', 'California', 'Slot', 92.63, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000276271, 'Da*', 'Far***', 0, 'Y', '1759 ********** Dr', 'SCOTTSDALE', 'AZ', 'USA', '85251', 'd*******@gmail.com', '1990-12-11', 'California', 'Table', 3223.43, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000299803, 'Fi***', 'Jon**', 0, 'Y', '5885 ********** Dr', 'DESERT HOT SPRINGS', 'CA', 'USA', '92240', 'f************@mac.com', '1963-01-20', 'California', 'Slot', 5494.48, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000299577, 'An*****', 'Ged****', 0, 'Y', '6201 ******** Rd', 'LAS VEGAS', 'NV', 'USA', '89139', 'a***************@verizon.net', '1971-08-08', 'LasVegas', 'Slot', 127.31, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10143760, 'Re*', 'Ant******', 0, 'Y', '3670 ***** Dr', 'BREA', 'CA', 'USA', '92821', 'p**********@aim.com', '1976-08-01', 'California', 'Slot', 4217.09, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10329393, 'St*****', 'Agu****', 0, 'Y', '8755 ***** Ave', 'HENDERSON', 'NV', 'USA', '89052', NULL, '1965-03-28', 'LasVegas', 'Table', 270.86, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000264921, 'Ki*******', 'Her******', 0, 'Y', '1918 ******** Dr', 'VAN NUYS', 'CA', 'USA', '91411', 't**********@gmail.com', '1976-01-06', 'California', 'Slot', 69.94, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000235312, 'Ke***', 'Mar*****', 0, 'Y', '8281 ****** Ct', 'LAS VEGAS', 'NV', 'USA', '89117', 'k*********@outlook.com', '1983-11-02', 'LasVegas', 'Table', 72.14, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000252658, 'Me***', 'Wor***', 0, 'Y', '8960 ********* Ave', 'SAN DIEGO', 'CA', 'USA', '92117', NULL, '1981-02-11', 'California', 'Table', 4593.05, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000275956, 'Do****', 'Enr*****', 0, 'Y', '1901 ******** Ave', 'CHULA VISTA', 'CA', 'USA', '91914', 'd*******@netscape.net', '1980-04-18', 'California', 'Table', 1796.33, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000276047, 'Ma***', 'Cha***', 0, 'Y', '5603 ***** Ct', 'BUTTONWILLOW', 'CA', 'USA', '93206', 'p*********@sbcglobal.net', '1977-12-02', 'California', 'Slot', 4551.44, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000307551, 'Fe*', 'Mat******', 0, 'N', '6147 *********** Ave', 'NUEVO', 'CA', 'UNK', 'UNKNOWN', 's*********@hotmail.com', '1971-01-03', 'California', 'Table', 330.37, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000307637, 'Br****', 'Cai***', 0, 'Y', '5695 ***** Ave', 'SAN DIEGO', 'CA', 'USA', '92154', 'b***********@verizon.net', '1973-12-20', 'California', 'Slot', 1892.59, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000311404, 'Er***', 'Wan*', 0, 'N', '1389 ************ Pl', 'Surpriso', 'AZ', 'UNK', 'UNKNOWN', 'e**********@yahoo.com', '1973-05-17', 'California', 'Slot', 4250.96, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10139507, 'Ne****', 'Par*', 0, 'Y', '2638 ********* St', 'LONG BEACH', 'CA', 'USA', '90808', 'n*********@outlook.com', '1977-12-20', 'California', 'Slot', 1010.20, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10178504, 'Di**', 'Hah*', 0, 'Y', '9354 ****** Pl', 'PHOENIX', 'AZ', 'USA', '85050', 'd******@roadrunner.com', '1956-01-13', 'California', 'Slot', 121.24, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10180085, 'La****', 'Tha******', 0, 'Y', '8612 ****** Dr', 'GLENDORA', 'CA', 'USA', '91741', 'l*********@verizon.net', '1990-01-15', 'California', 'Slot', 418.59, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10251111, 'Hi*****', 'Gra*', 0, 'Y', '1882 ************ Ct', 'LAS VEGAS', 'NV', 'USA', '89129', 'k**************@cox.net', '1962-09-19', 'LasVegas', 'Slot', 154.87, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10401464, 'Co***', 'Beh***', 0, 'Y', '7095 ***** Rd', 'PHOENIX', 'AZ', 'USA', '85051', NULL, '1983-12-22', 'California', 'Table', 84.31, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10459901, 'El*****', 'L M******', 0, 'Y', '6326 *********** Ct', 'SYLMAR', 'CA', 'USA', '91342', 't***************@hotmail.com', '1994-02-24', 'California', 'Table', 2930.71, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10519036, 'Ne*****', 'Cor*********', 0, 'Y', '1068 ******** Ave', 'COMPTON', 'CA', 'USA', '90222', 'n*******@netscape.net', '1974-12-18', 'California', 'Table', 58.56, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10555547, 'Pr*******', 'Sis**', 0, 'Y', '3610 ********** Ave', 'LOS ANGELES', 'CA', 'USA', '90011', NULL, '1968-01-04', 'California', 'Slot', 4196.47, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10568992, 'Ka***', 'Mar*****', 0, 'Y', '8700 ********* Ct', 'LAS VEGAS', 'NV', 'USA', '89147', 'k*********@verizon.net', '1981-05-16', 'LasVegas', 'Table', 85.30, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 101388239, 'Ke***', 'Ben***', 0, 'N', '4667 ******* Ct', 'PICO RIVERA', 'CA', 'UNK', 'UNKNOWN', 'w***@comcast.net', '1956-04-24', 'California', 'Slot', 2477.24, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 101452722, 'Br***', 'Hec******', 0, 'Y', '8256 ********** Ct', 'SPARKS', 'NV', 'USA', '89436', 'b*********@rocketmail.com', '1966-12-18', 'California', 'Slot', 131.11, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000030835, 'Ke***', 'Mc ********', 0, 'Y', '3130 *********** Ave', 'LOS ANGELES', 'CA', 'USA', '90057', 'd*******@outlook.com', '1976-04-26', 'California', 'Slot', 28.19, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000053383, 'Su***', 'Gon*****', 0, 'Y', '2271 ********* Rd', 'RIDGECREST', 'CA', 'USA', '93555', 's***********@mac.com', '1963-05-10', 'California', 'Slot', 982.20, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000115999, 'Si*****', 'Sak*****', 0, 'N', '7509 ***** Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1988-07-05', 'California', 'Table', 264.15, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000163310, 'Yu***', 'Kel**', 0, 'Y', '75 ****** Pl', 'LOS ANGELES', 'CA', 'USA', '90066', 'n**********@roadrunner.com', '1997-09-16', 'California', 'Slot', 761.96, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000232008, 'Se**', 'Man*******', 0, 'Y', '5839 ****** Pl', 'LAVEEN VILLAGE', 'AZ', 'USA', '85339', NULL, '1968-09-13', 'California', 'Slot', 42.13, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10576291, 'La**', 'War*', 0, 'N', '2760 ****** Dr', 'SUN TAN VALLEY', 'AZ', 'UNK', 'UNKNOWN', 'l*************@aim.com', '1988-06-02', 'California', 'Slot', 37.36, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 100423819, 'Te******', 'Sar************', 0, 'Y', '8513 ****** Ave', 'RENO', 'NV', 'USA', '89521', 'f******@sbcglobal.net', '1959-11-19', 'California', 'Table', 25.87, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 100482992, 'Ma****', 'Sin**', 0, 'Y', '6275 ********** Ave', 'INDIO', 'CA', 'USA', '92201', NULL, '1961-11-13', 'California', 'Slot', 30.16, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101098957, 'Ja*****', 'War***', 0, 'Y', '1197 ******* Ct', 'RENO', 'NV', 'USA', '89512', NULL, '1982-10-12', 'California', 'Slot', 54.68, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101515504, 'So*****', 'Wil******', 0, 'Y', '6947 ************ Ct', 'SPARKS', 'NV', 'USA', '89436', 'w*******@verizon.net', '1954-09-10', 'California', 'Slot', 919.66, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 200169146, 'An****', 'Lee', 0, 'Y', '2130 ******* St', 'LAS VEGAS', 'NV', 'USA', '89178', 'b*********@outlook.com', '1962-06-26', 'LasVegas', 'Table', 32.30, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 200486961, 'Ch*********', 'Rog***', 0, 'N', '7755 ******* St', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1981-11-26', 'California', 'Table', 80.50, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 769912552, 'Li*****', 'Agu**', 0, 'N', '6851 ***** St', 'SAUGUS', 'CA', 'UNK', 'UNKNOWN', 'c**********@hotmail.com', '1965-06-11', 'California', 'Slot', 446.93, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 861858535, 'Ji**', 'Niu', 0, 'Y', '7186 ************ Ct', 'VENICE', 'CA', 'USA', '90291', 'p*******@sbcglobal.net', '1967-09-28', 'California', 'Slot', 33.86, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000004243, 'Ri***', 'Smi**', 0, 'Y', '576 *********** Dr', 'LOS ANGELES', 'CA', 'USA', '90047', NULL, '1955-03-20', 'California', 'Slot', 54.98, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000024490, 'Ma**', 'Sch*****', 0, 'Y', '4801 ***** Ct', 'LANCASTER', 'CA', 'USA', '93535', 'f*******@gmail.com', '1960-01-27', 'California', 'Slot', 1092.80, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000038596, 'Ky*', 'Rea*', 0, 'Y', '8636 ****** Rd', 'CARSON', 'CA', 'USA', '90746', 'k********@att.net', '1988-02-28', 'California', 'Table', 41.05, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000065259, 'Me*****', 'Sul*****', 0, 'Y', '4326 ****** Dr', 'GLENDALE', 'AZ', 'USA', '85304', NULL, '1971-07-28', 'California', 'Slot', 5882.87, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000066430, 'Ro*', 'Car***', 0, 'Y', '224 *********** Dr', 'HENDERSON', 'NV', 'USA', '89044', 'r*******@verizon.net', '1966-04-12', 'LasVegas', 'Slot', 461.23, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000288759, 'Em******', 'Kim', 0, 'Y', '8114 ******* Ct', 'LAKEWOOD', 'CA', 'USA', '90715', 'n****@aim.com', '1973-01-24', 'California', 'Table', 499.65, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10237522, 'Ca****', 'Bia***', 0, 'Y', '676 ****** St', 'PERRIS', 'CA', 'USA', '92571', 'c******@hotmail.com', '1977-08-02', 'California', 'Slot', 268.51, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10320717, 'Ca***', 'Lim', 0, 'Y', '47 ********* Dr', 'MONROVIA', 'CA', 'USA', '91016', 't*********@aol.com', '1964-06-16', 'California', 'Slot', 481.64, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10322922, 'Er*****', 'Cha*', 0, 'Y', '4678 ************ Ave', 'SAN PEDRO', 'CA', 'USA', '90731', NULL, '1981-02-25', 'California', 'Table', 813.79, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10560204, 'Ja***', 'Col****', 0, 'Y', '2044 ********* Dr', 'CANYON COUNTRY', 'CA', 'USA', '91387', 'j**@gmail.com', '1960-10-25', 'California', 'Table', 34.68, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 200083155, 'Jo**', 'Pri***', 0, 'Y', '1585 ************ Ave', 'ALISO VIEJO', 'CA', 'USA', '92656', 's******@mac.com', '1975-07-23', 'California', 'Table', 5620.07, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200325553, 'Ga**', 'Ryc****', 0, 'Y', '197 ****** Rd', 'FALLON', 'NV', 'USA', '89406', 'g**********@netscape.net', '1945-02-20', 'California', 'Slot', 392.92, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10369041, 'Ke***', 'Gra*', 0, 'Y', '1098 *********** Dr', 'GILBERT', 'AZ', 'USA', '85295', NULL, '1971-02-23', 'California', 'Slot', 237.75, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10519938, 'Ka***', 'Hau****', 0, 'Y', '4251 *********** Rd', 'PHOENIX', 'AZ', 'USA', '85029', 'c****@hotmail.com', '1958-05-08', 'California', 'Slot', 36.29, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10515430, 'Ma***', 'Spr*****', 0, 'Y', '807 ***** Rd', 'BUCKEYE', 'AZ', 'USA', '85396', 'o********@yahoo.com', '1945-05-12', 'California', 'Slot', 2346.78, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000096248, 'An*', 'Nic****', 0, 'Y', '2878 ****** Dr', 'GLENDALE', 'CA', 'USA', '91205', 'a*********@mac.com', '1954-02-15', 'California', 'Slot', 520.35, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000104077, 'Je****', 'Cla**', 0, 'Y', '5961 ********* Dr', 'OCEANSIDE', 'CA', 'USA', '92058', 'k************@cox.net', '1983-09-06', 'California', 'Slot', 64.89, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000123336, 'Mo****', 'Tav****', 0, 'Y', '4112 ******** Dr', 'LOS ANGELES', 'CA', 'USA', '90019', 'm**************@att.net', '1968-06-08', 'California', 'Slot', 69.32, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000158166, 'Jo****', 'Tor***', 0, 'N', '8528 ******** Dr', 'RENO', 'NV', 'UNK', 'UNKNOWN', 'j****************@sbcglobal.net', '1988-07-06', 'California', 'Slot', 4697.10, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000167317, 'Ha*', 'Dot*', 0, 'Y', '2746 ******** Dr', 'LOS ANGELES', 'CA', 'USA', '90046', NULL, '1961-11-24', 'California', 'Table', 27.03, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000168351, 'Sh***', 'Kwo*', 0, 'Y', '8313 *********** Pl', 'SANTA CLARITA', 'CA', 'USA', '91350', 's*************@hotmail.com', '1985-06-18', 'California', 'Table', 2336.95, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000186175, 'Sh********', 'Kol****', 0, 'Y', '6247 ***** Dr', 'MURRIETA', 'CA', 'USA', '92562', NULL, '1993-11-15', 'California', 'Table', 328.16, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000211726, 'Va*****', 'Kee**', 0, 'Y', '691 ********** Ave', 'HENDERSON', 'NV', 'USA', '89011', 'v****@rocketmail.com', '1962-05-25', 'LasVegas', 'Slot', 110.51, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000208525, 'Ja*', 'Sex***', 0, 'Y', '4878 ******* Ave', 'TORRANCE', 'CA', 'USA', '90501', 'j*********@yahoo.com', '1980-04-08', 'California', 'Slot', 2670.18, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000226192, 'Ka***', 'Oli****', 0, 'Y', '8538 ****** Ct', 'WOFFORD HEIGHTS', 'CA', 'USA', '93285', 'k************@aol.com', '1987-10-20', 'California', 'Slot', 3559.94, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000222004, 'Ag***', 'Che****', 0, 'Y', '7462 *********** Pl', 'PACOIMA', 'CA', 'USA', '91331', NULL, '1980-06-18', 'California', 'Slot', 240.09, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000245287, 'Ca*****', 'Esp****', 0, 'Y', '2786 ******* Ct', 'CHANDLER', 'AZ', 'USA', '85226', NULL, '1968-05-16', 'California', 'Slot', 827.08, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000250134, 'Br***', 'Van***************', 0, 'Y', '1640 ************ Dr', 'NORTH LAS VEGAS', 'NV', 'USA', '89032', NULL, '1973-02-17', 'LasVegas', 'Table', 917.91, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000269000, 'Vi***', 'Wil*****', 0, 'Y', '4288 ********* Rd', 'BEATTY', 'NV', 'USA', '89003', 'v********@verizon.net', '1967-10-19', 'California', 'Slot', 1963.92, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000273096, 'Se*******', 'Orn*****', 0, 'Y', '5835 ****** Pl', 'PHOENIX', 'AZ', 'USA', '85040', 'c**************@outlook.com', '1972-08-13', 'California', 'Slot', 646.71, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10324587, 'Ma*****', 'Hec******', 0, 'Y', '3234 ***** Rd', 'LOS ANGELES', 'CA', 'USA', '90020', NULL, '1977-10-04', 'California', 'Table', 138.09, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 113340360, 'Jo**', 'Lee', 0, 'Y', '6112 ********** Dr', 'WHITTIER', 'CA', 'USA', '90606', NULL, '1953-03-13', 'California', 'Table', 106.54, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 312571902, 'Lo*******', 'But****', 0, 'Y', '8415 ****** Ct', 'LAS VEGAS', 'NV', 'USA', '89147', 'l***********@roadrunner.com', '1967-07-10', 'LasVegas', 'Slot', 353.44, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 567151949, 'Br*****', 'Suh', 0, 'Y', '5153 ******** Ave', 'NORTH LAS VEGAS', 'NV', 'USA', '89031', 'b******@roadrunner.com', '1962-04-21', 'LasVegas', 'Slot', 47.52, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 262202275, 'Jo**', 'Ell**', 0, 'Y', '4051 ********** Ct', 'NORTH LAS VEGAS', 'NV', 'USA', '89032', NULL, '1965-12-05', 'LasVegas', 'Slot', 535.71, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 200008877, 'Ja***', 'Dep***********', 0, 'N', '3019 ****** Ave', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'n******************@roadrunner.com', '1988-08-22', 'California', 'Slot', 497.40, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000012505, 'Ju****', 'Han***', 0, 'N', '6415 *********** Ct', 'CHINO', 'CA', 'UNK', 'UNKNOWN', NULL, '1978-07-20', 'California', 'Table', 30.90, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10554459, 'Kw********', 'Tra*', 0, 'Y', '7829 ********* Pl', 'BUENA PARK', 'CA', 'USA', '90620', 'h***********@hotmail.com', '1968-12-19', 'California', 'Table', 638.81, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10573686, 'Gl***', 'Noy**', 0, 'N', '6292 ********* Ave', 'BAKERSFIELD', 'CA', 'UNK', 'UNKNOWN', NULL, '1956-06-04', 'California', 'Slot', 28.60, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10574642, 'Gu*****', 'Qui*****', 0, 'Y', '9484 ********* Ct', 'FONTANA', 'CA', 'USA', '92335', NULL, '1986-04-04', 'California', 'Table', 5089.32, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000012862, 'Pe***', 'Cho*', 0, 'N', '4490 *********** St', 'INDIAN WELLS', 'CA', 'UNK', 'UNKNOWN', NULL, '1998-08-22', 'California', 'Table', 3438.53, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000018619, 'Ta*****', 'Tin********', 0, 'Y', '8142 ************ Ct', 'LONG BEACH', 'CA', 'USA', '90802', 't***************@aim.com', '1985-08-23', 'California', 'Slot', 503.06, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000048033, 'Ra****', 'Lei', 0, 'Y', '2517 ********* Ave', 'TORRANCE', 'CA', 'USA', '90501', 'r**************@verizon.net', '1995-05-10', 'California', 'Slot', 1235.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000093257, 'Se***', 'Wal****', 0, 'Y', '8265 ************ Rd', 'LA MIRADA', 'CA', 'USA', '90638', NULL, '1995-02-06', 'California', 'Table', 32.05, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000101697, 'Be******', 'Leu**', 0, 'N', '5352 ****** Pl', '"SUN VALLEY "', 'NV', 'UNK', 'UNKNOWN', 's************@aol.com', '1979-06-22', 'California', 'Slot', 102.35, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000106820, 'Fr*****', 'Tul****', 0, 'Y', '6540 ********** Pl', 'LOS ANGELES', 'CA', 'USA', '90025', 'f***********@hotmail.com', '1984-08-09', 'California', 'Slot', 387.03, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000109937, 'Ma*****', 'You*', 0, 'N', '4265 ************ Ave', 'ALHAMBRA', 'CA', 'UNK', 'UNKNOWN', 'd*********@rocketmail.com', '1979-01-24', 'California', 'Slot', 81.69, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000186879, 'Ja***', 'Kim', 0, 'N', '8477 ************ Pl', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'j********@sbcglobal.net', '1963-07-12', 'California', 'Slot', 952.51, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000229186, 'Is****', 'Sch*****', 0, 'N', '4900 ********* Dr', 'NORTH LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1986-05-14', 'California', 'Table', 2173.73, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000305771, 'Da***', 'Wen', 0, 'Y', '6366 ************ Rd', 'LONG BEACH', 'CA', 'USA', '90803', 'w*********@gmail.com', '1955-01-16', 'California', 'Table', 291.78, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10044812, 'Vi****', 'Chu**', 0, 'Y', '6895 ****** Ct', 'CHINO', 'CA', 'USA', '91710', NULL, '1983-09-16', 'California', 'Table', 5409.44, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 6000063917, 'Ri*****', 'Van******', 0, 'N', '4229 *********** Dr', 'SCOTTSDALE', 'AZ', 'UNK', 'UNKNOWN', 's***********@att.net', '1944-06-19', 'California', 'Slot', 2777.48, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10213286, 'Gu*******', 'Sco**', 0, 'N', '5227 ********** Ct', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'l********@rocketmail.com', '1964-03-01', 'California', 'Slot', 677.90, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10282073, 'Jj', 'Gwo*', 0, 'Y', '8660 *********** Rd', 'WALNUT CREEK', 'CA', 'USA', '91789', NULL, '1982-05-24', 'California', 'Table', 356.20, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000041248, 'Zh*****', 'Hei******', 0, 'N', '4160 ************ Ct', 'SAN PEDRO', 'CA', 'UNK', 'UNKNOWN', NULL, '1981-09-12', 'California', 'Table', 150.36, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000060668, 'Ra***', 'Sai*******', 0, 'Y', '976 ***** Dr', 'FULLERTON', 'CA', 'USA', '92831', 'r***********@hotmail.net', '1982-03-06', 'California', 'Slot', 104.98, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000064490, 'Ar***', 'Sho***', 0, 'Y', '2137 ******** Ave', 'LAS VEGAS', 'NV', 'USA', '89104', NULL, '1987-01-05', 'LasVegas', 'Table', 67.26, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000011866, 'Ri*****', 'Luk*', 0, 'Y', '6833 *********** Ave', 'APPLE VALLEY', 'CA', 'USA', '92308', 'c**********@yahoo.com', '1976-06-27', 'California', 'Slot', 169.44, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000016921, 'Al*****', 'Cor***', 0, 'Y', '3740 ********** Pl', 'CORONA', 'CA', 'USA', '92881', NULL, '1954-02-22', 'California', 'Slot', 1246.88, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000182050, 'Br*****', 'Hah*', 0, 'Y', '8355 ********* Pl', 'LAS VEGAS', 'NV', 'USA', '89141', 't***********@roadrunner.com', '1981-08-28', 'LasVegas', 'Slot', 295.27, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000182259, 'Er**', 'Din*****', 0, 'Y', '7836 ******* Pl', 'LAS VEGAS', 'NV', 'USA', '89130', 'h*********@verizon.net', '1986-04-04', 'LasVegas', 'Table', 398.98, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000186630, 'Yo****', 'Phi*****', 0, 'N', '3920 *********** Dr', 'GILBERT', 'AZ', 'UNK', 'UNKNOWN', 'e*********@gmail.com', '1996-07-04', 'California', 'Table', 27.02, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000211562, 'Am*', 'Gom**', 0, 'Y', '2690 *********** Dr', 'PALMDALE', 'CA', 'USA', '93551', 'a***********@comcast.net', '1986-07-24', 'California', 'Slot', 82.19, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10345840, 'Fe**', 'Lin*', 0, 'Y', '3742 ********* Ct', 'PAHRUMP', 'NV', 'USA', '89048', 'f***@sbcglobal.net', '1945-05-06', 'California', 'Slot', 50.68, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10194212, 'An***', 'Mei', 0, 'Y', '88 ******* Dr', 'TUCSON', 'AZ', 'USA', '85756', 'f***********@verizon.net', '1982-06-09', 'California', 'Slot', 368.19, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10195181, 'En*****', 'Sha***', 0, 'Y', '1450 ******** St', 'UPLAND', 'CA', 'USA', '91784', NULL, '1964-07-09', 'California', 'Table', 35.72, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10329205, 'Mo**', 'Riv***********', 0, 'Y', '6636 ********** Pl', 'ENCINO', 'CA', 'USA', '91436', 'd***************@att.net', '1972-01-09', 'California', 'Slot', 4651.29, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10517646, 'St****', 'Gon*****', 0, 'Y', '9266 ******** St', 'HENDERSON', 'NV', 'USA', '89009', 's***********@aol.com', '1953-05-01', 'LasVegas', 'Table', 30.43, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10538661, 'Lu***', 'Ara*****', 0, 'Y', '6650 ******** St', 'CORONA', 'CA', 'USA', '92881', 'l**************@aim.com', '1959-06-04', 'California', 'Slot', 151.62, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200064494, 'Re******', 'Wol**', 0, 'N', '6933 ******** Ct', 'OXNARD', 'CA', 'UNK', 'UNKNOWN', 'r*******@gmail.com', '1970-05-16', 'California', 'Slot', 4711.21, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200302085, 'Wi*****', 'Har***', 0, 'N', '4554 ****** Rd', 'YUCAIPA', 'CA', 'UNK', 'UNKNOWN', 'k**********@sbcglobal.net', '1980-07-27', 'California', 'Slot', 187.82, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200419330, 'Da****', 'Tan**', 0, 'Y', '3774 ****** Ave', 'SPARKS', 'NV', 'USA', '89441', 'r******@verizon.net', '1995-05-23', 'California', 'Slot', 371.82, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 200276074, 'Ch***', 'Bra****', 0, 'N', '9017 ******* Rd', 'SAN PEDRO', 'CA', 'UNK', 'UNKNOWN', 'c****@sbcglobal.net', '1975-06-28', 'California', 'Slot', 4250.96, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 409635368, 'Da***', 'Qui*****', 0, 'Y', '2840 *********** Ct', 'PALMDALE', 'CA', 'USA', '93550', 'd*********@hotmail.net', '1967-12-14', 'California', 'Slot', 2176.99, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 217959636, 'Xi*', 'Gar***', 0, 'Y', '5088 ********** Dr', 'LAS VEGAS', 'NV', 'USA', '89113', 'x**@yahoo.com', '1978-10-08', 'LasVegas', 'Table', 1848.08, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000017959, 'Ch*********', 'Tre**', 0, 'Y', '1273 ***** Rd', 'PASADENA', 'CA', 'USA', '91103', 'c**********@netscape.net', '1991-11-01', 'California', 'Table', 5695.32, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000201408, 'Ve*****', 'Ibr****', 0, 'Y', '74 ***** Dr', 'NORTH LAS VEGAS', 'NV', 'USA', '89084', 'v****************@yahoo.com', '1948-02-01', 'LasVegas', 'Slot', 40.44, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000202861, 'Sa****', 'Tan***', 0, 'Y', '133 *********** Ct', 'FONTANA', 'CA', 'USA', '92336', 's*********@hotmail.net', '1976-11-05', 'California', 'Both', 981.32, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000215617, 'Ma*****', 'Win*', 0, 'Y', '2941 ************ Ct', 'THOUSAND OAKS', 'CA', 'USA', '91360', 'm*************@comcast.net', '1986-02-14', 'California', 'Slot', 3478.30, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10525161, 'Pa**', 'Ham*****', 0, 'N', '731 ********** Dr', 'YUCAIPA', 'CA', 'UNK', 'UNKNOWN', NULL, '1966-12-27', 'California', 'Slot', 611.80, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10525388, 'Ma***', 'Diz**', 0, 'Y', '195 *********** St', 'MORENO VALLEY', 'CA', 'USA', '92553', 's*************@outlook.com', '1981-05-15', 'California', 'Slot', 1187.02, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10569201, 'Jo****', 'Moo***', 0, 'N', '834 ********* Rd', 'CARSON', 'CA', 'UNK', 'UNKNOWN', 'j*******@sbcglobal.net', '1981-08-13', 'California', 'Table', 699.59, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10581858, 'Ma****', 'Nev***', 0, 'Y', '576 ******** St', 'CANYON COUNTRY', 'CA', 'USA', '91351', 'm*************@verizon.net', '1995-02-13', 'California', 'Slot', 533.58, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 200076250, 'Ei****', 'Gom**', 0, 'N', '9483 ********* Rd', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'e**************@yahoo.com', '1949-04-08', 'California', 'Slot', 3054.28, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 200152933, 'Sp*****', 'Wan*', 0, 'Y', '1608 ****** Rd', 'PEORIA', 'AZ', 'USA', '85383', 's********@rocketmail.com', '1976-02-09', 'California', 'Slot', 32.06, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 721416081, 'Ze*****', 'Cal*****', 0, 'Y', '7086 *********** Rd', 'LAS VEGAS', 'NV', 'USA', '89102', NULL, '1952-03-24', 'LasVegas', 'Slot', 119.39, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000022706, 'Ma***', 'Ver***', 0, 'Y', '5590 ********* Pl', 'SANTA ANA', 'CA', 'USA', '92704', 'l**********@outlook.com', '1988-02-04', 'California', 'Slot', 26.02, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000096474, 'Ry**', 'Rob***', 0, 'Y', '9083 ******** Ct', 'SCOTTSDALE', 'AZ', 'USA', '85258', NULL, '1981-06-18', 'California', 'Slot', 27.15, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000113981, 'Je***', 'Fow***', 0, 'Y', '8283 ************ Ave', 'HENDERSON', 'NV', 'USA', '89074', 'j*********@yahoo.com', '1974-07-12', 'LasVegas', 'Table', 139.70, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000184789, 'Je*****', 'Eng****', 0, 'N', '5002 ***** Rd', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'q********@netscape.net', '1964-05-04', 'California', 'Slot', 506.44, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000185382, 'Ar**', 'San****', 0, 'N', '9325 *********** Dr', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'a*************@cox.net', '1976-09-08', 'California', 'Slot', 698.55, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000191150, 'Ni**', 'Whi**', 0, 'N', '6721 *********** St', 'SAN DIEGO', 'CA', 'UNK', 'UNKNOWN', NULL, '1988-09-08', 'California', 'Slot', 1399.26, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000193497, 'Ma*', 'Gas***', 0, 'N', '2907 ***** Rd', 'PHOENIX', 'AZ', 'UNK', 'UNKNOWN', NULL, '1984-03-09', 'California', 'Table', 691.94, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000196780, 'Mi*****', 'Fre*****', 0, 'Y', '5766 ***** St', 'PALM SPRINGS', 'CA', 'USA', '92264', 'm******@rocketmail.com', '1964-02-27', 'California', 'Slot', 644.97, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000199379, 'Br***', 'Mcb****', 0, 'Y', '6951 ********* Ave', 'SIMI VALLEY', 'CA', 'USA', '93063', 'b***********@hotmail.com', '1990-08-24', 'California', 'Slot', 919.01, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10567069, 'Le****', 'Men****', 0, 'Y', '555 ********* Pl', 'SOUTH PASADENA', 'CA', 'UNK', '325604', NULL, '1971-12-13', 'California', 'Table', 190.24, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 175290778, 'Ni******', 'Lop************', 0, 'Y', '9152 ********* Ave', 'LAS VEGAS', 'NV', 'USA', '89109', 'n********@comcast.net', '1982-03-24', 'LasVegas', 'Slot', 88.25, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200361252, 'Fr*******', 'Rob*****', 0, 'Y', '602 ******* Ct', 'RENO', 'NV', 'USA', '89521', 'v********@rocketmail.com', '1983-03-05', 'California', 'Slot', 2053.70, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10576810, 'Gu*********', 'Lot*', 0, 'Y', '6022 ******* St', 'OXNARD', 'CA', 'USA', '93030', 'm*******@yahoo.com', '1966-09-10', 'California', 'Slot', 175.39, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 419465420, 'Ro****', 'Fre***', 0, 'Y', '9166 ************ St', 'SCOTTSDALE', 'AZ', 'USA', '85261', 'r*************@hotmail.net', '1968-09-14', 'California', 'Slot', 43.22, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000005244, 'An*****', 'Rob****', 0, 'Y', '703 ****** Pl', 'LAS VEGAS', 'NV', 'USA', '89117', NULL, '1952-07-21', 'LasVegas', 'Table', 221.78, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000133870, 'Ad**', 'Ami*************', 0, 'Y', '4111 ********** Ave', 'MORENO VALLEY', 'CA', 'USA', '92555', 'p************@comcast.net', '1988-11-12', 'California', 'Table', 2836.14, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000136495, 'Ju****', 'Sun', 0, 'Y', '3659 ***** Rd', 'FRESNO', 'CA', 'USA', '93701', 'j*********@hotmail.com', '1981-10-18', 'California', 'Slot', 2157.27, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000189916, 'Sa******', 'Hol***', 0, 'N', '3437 *********** St', 'COVINA', 'CA', 'UNK', 'UNKNOWN', NULL, '1976-05-09', 'California', 'Table', 83.23, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000181970, 'Ke***', 'Ger****', 0, 'Y', '4473 *********** Pl', 'BEVERLY HILLS', 'CA', 'USA', '90210', 'k*******@hotmail.com', '1976-07-09', 'California', 'Table', 1143.90, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000248336, 'Mi*****', 'Zhu', 0, 'Y', '3021 ************ Pl', 'CHANDLER', 'AZ', 'USA', '85224', 'm****************@gmail.com', '1986-01-07', 'California', 'Slot', 136.89, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000272968, 'Te******', 'Mor***', 0, 'N', '3215 ***** Ave', 'NEWPORT BEACH', 'CA', 'UNK', 'UNKNOWN', '7*******@sbcglobal.net', '1973-05-27', 'California', 'Slot', 246.83, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000283647, 'Al**', 'Men************', 0, 'Y', '587 ********* St', 'LYNWOOD', 'CA', 'USA', '90262', 'o**************@comcast.net', '1985-08-12', 'California', 'Table', 331.69, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000312830, 'Ro****', 'Hur****', 0, 'Y', '7319 ****** St', 'LOS ANGELES', 'CA', 'USA', '90033', 'p*****@gmail.com', '1985-09-03', 'California', 'Table', 41.96, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000068782, 'Be******', 'Gol*****', 0, 'Y', '6345 ******** Ct', 'LOS ANGELES', 'CA', 'USA', '90015', NULL, '1986-06-05', 'California', 'Slot', 44.04, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000123153, 'Br*****', 'Mis******', 0, 'Y', '2848 ********* Dr', 'GILBERT', 'AZ', 'USA', '85296', 'b********@att.net', '1964-10-11', 'California', 'Slot', 4336.83, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000199320, 'Ju**', 'Cor***', 0, 'Y', '7179 ********* Ave', 'EL MONTE', 'CA', 'USA', '91732', 'a*************@comcast.net', '1965-05-14', 'California', 'Slot', 578.82, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000214030, 'Ma***', 'Car**', 0, 'Y', '3160 *********** Dr', 'SPRING VALLEY', 'CA', 'USA', '91977', 'm***********@verizon.net', '1964-01-25', 'California', 'Slot', 26.05, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000229129, 'Yo***', 'Han', 0, 'Y', '3471 ****** Rd', 'LOS ANGELES', 'CA', 'USA', '90006', 'y*******@netscape.net', '1958-04-21', 'California', 'Slot', 51.80, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000297615, 'Ch*********', 'Cha**', 0, 'Y', '3533 *********** St', 'LAS VEGAS', 'NV', 'USA', '89147', 'c**********@roadrunner.com', '1992-09-19', 'LasVegas', 'Slot', 208.80, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10064580, 'Ke***', 'Ams****', 0, 'Y', '3008 ******* Rd', 'MESA', 'AZ', 'USA', '85209', 'c******@outlook.com', '1972-06-23', 'California', 'Slot', 107.34, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10364367, 'Nu***', 'Kuh*', 0, 'N', '2261 ****** Dr', 'FONTANA', 'CA', 'UNK', 'UNKNOWN', 'n**********@aol.com', '1992-08-24', 'California', 'Table', 75.19, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10386898, 'Wa*******', 'Mon*******', 0, 'N', '2554 ******** Ave', 'TUCSON', 'AZ', 'UNK', 'UNKNOWN', NULL, '1993-12-25', 'California', 'Slot', 885.72, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10432170, 'Ro***', 'Pol****', 0, 'Y', '1992 ********** St', 'LOS ANGELES', 'CA', 'USA', '90018', NULL, '1982-03-16', 'California', 'Table', 5531.42, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10127577, 'Da***', 'Del*****', 0, 'Y', '1989 *********** Pl', 'LOS ANGELES', 'CA', 'USA', '90004', NULL, '1983-08-23', 'California', 'Table', 650.73, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10538662, 'Gr*****', 'Pri**', 0, 'Y', '2667 ******** Rd', 'CORONA', 'CA', 'USA', '92881', NULL, '1957-06-06', 'California', 'Slot', 403.63, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10567092, 'Je**', 'Tej***', 0, 'N', '1660 ************ Pl', 'LA MIRADA', 'CA', 'UNK', 'UNKNOWN', 'j**@att.net', '1967-04-21', 'California', 'Slot', 5585.34, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10281884, 'Ba*****', 'Mcg****', 0, 'N', '8041 ****** Ct', 'LA MIRADA', 'CA', 'UNK', 'UNKNOWN', '3********@hotmail.net', '1979-04-06', 'California', 'Table', 476.80, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10581602, 'Ja***', 'Lim', 0, 'Y', '2888 *********** Ct', 'CHINO', 'CA', 'USA', '91710', NULL, '1960-10-28', 'California', 'Slot', 25.51, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 200079051, 'Gr*****', 'Pom*******', 0, 'Y', '1051 ******** Rd', 'MAMMOTH LAKES', 'CA', 'USA', '93546', 'g*******@hotmail.net', '1962-03-19', 'California', 'Slot', 4872.20, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200417563, 'Ly*****', 'Lem**', 0, 'Y', '1084 ******** Ct', 'FLAGSTAFF', 'AZ', 'USA', '86001', 'l*********@aol.com', '1970-11-18', 'California', 'Slot', 1944.58, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 101697970, 'Je****', 'Ver**', 0, 'Y', '8911 ************ Pl', 'RENO', 'NV', 'USA', '89502', 'j*************@hotmail.net', '1946-08-03', 'California', 'Slot', 5484.05, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 281195015, 'Ju**', 'Fer******', 0, 'Y', '4523 ******** Ave', 'GRANADA HILLS', 'CA', 'USA', '91344', NULL, '1954-06-28', 'California', 'Slot', 4675.54, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000021523, 'De*****', 'Due***', 0, 'Y', '5115 ************ Ave', '"SACRAMENTO "', 'AZ', 'USA', '85323', 'd**********@gmail.com', '1984-10-13', 'California', 'Slot', 764.71, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 981955604, 'Jo*******', 'Par*', 0, 'Y', '2899 *********** Ave', 'FRESNO', 'CA', 'USA', '93710', 'j**************@cox.net', '1988-05-16', 'California', 'Table', 6023.36, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 609436986, 'Je****', 'Win*', 0, 'Y', '6775 ***** St', 'EASTVALE', 'CA', 'USA', '92880', 'w*********@att.net', '1984-04-23', 'California', 'Table', 115.76, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000005722, 'Ro**', 'Fri*****', 0, 'Y', '8784 ************ Ct', 'LONG BEACH', 'CA', 'USA', '90805', NULL, '1968-11-25', 'California', 'Slot', 1279.72, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000006152, 'Al****', 'Hua**', 0, 'Y', '1361 *********** Ave', 'FRESNO', 'CA', 'USA', '93727', NULL, '1949-12-14', 'California', 'Slot', 192.14, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000010614, 'Ma*****', 'Leo****', 0, 'N', '5314 ****** Rd', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1984-07-13', 'California', 'Table', 700.43, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000071776, 'Ja***', 'Leo*', 0, 'Y', '6053 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89138', 'j*******@yahoo.com', '1974-12-05', 'LasVegas', 'Slot', 97.90, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000049293, 'Wi*****', 'Law', 0, 'Y', '3275 ***** Ct', 'AZUSA', 'CA', 'USA', '91702', NULL, '1997-09-03', 'California', 'Table', 2281.30, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000093336, 'Ly****', 'Wie**', 0, 'Y', '5327 ******* St', 'TUCSON', 'AZ', 'USA', '85719', 'l********@hotmail.com', '1987-10-12', 'California', 'Slot', 626.16, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000103598, 'Jo******', 'Esp***', 0, 'N', '3707 *********** Dr', 'SYLMAR', 'CA', 'UNK', 'UNKNOWN', NULL, '1989-04-05', 'California', 'Table', 587.22, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000255928, 'Ma****', 'Dee*', 0, 'Y', '9123 ********** Pl', 'LAS VEGAS', 'NV', 'USA', '89158', 'm**********@roadrunner.com', '1969-04-24', 'LasVegas', 'Table', 30.57, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10044886, 'Gi******', 'And*****', 0, 'Y', '7723 ***** Dr', 'INGLEWOOD', 'CA', 'UNK', 'NW88LJ', NULL, '1974-06-04', 'California', 'Table', 59.12, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10048610, 'Fr***', 'Gon*****', 0, 'Y', '1194 ********* Pl', 'LONG BEACH', 'CA', 'USA', '90805', 'f*******@gmail.com', '1959-04-10', 'California', 'Slot', 2677.66, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10142332, 'Sh****', 'Car****', 0, 'Y', '640 ********* Dr', 'LAS VEGAS', 'NV', 'USA', '89109', NULL, '1957-06-07', 'LasVegas', 'Slot', 1757.59, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10228982, 'Ne**', 'Alf***', 0, 'Y', '8670 ***** Rd', 'ROSEMEAD', 'CA', 'USA', '91770', 'b********@att.net', '1984-10-02', 'California', 'Slot', 368.67, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10339461, 'Xi******', 'Car*', 0, 'Y', '7327 ************ Pl', 'MESQUITE', 'NV', 'USA', '89027', 'a*********@yahoo.com', '1977-10-14', 'California', 'Table', 266.88, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10418502, 'Ja***', 'Her****', 0, 'Y', '2651 ************ St', 'LAS VEGAS', 'NV', 'USA', '89134', NULL, '1970-02-09', 'LasVegas', 'Slot', 948.14, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10479220, 'Mi****', 'Mar******', 0, 'Y', '1288 *********** Rd', 'LA PUENTE', 'CA', 'USA', '91744', 'm****@yahoo.com', '1966-02-27', 'California', 'Table', 3039.96, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000080850, 'Ti****', 'Sko***', 0, 'Y', '1295 ******* Rd', 'LOS ANGELES', 'CA', 'USA', '90043', 'y**********@roadrunner.com', '1970-04-04', 'California', 'Slot', 131.58, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000126546, 'Pa**', 'Meu****', 0, 'N', '7714 ************ Ct', 'OXNARD', 'CA', 'UNK', 'UNKNOWN', 'h***********@hotmail.com', '1960-10-18', 'California', 'Slot', 395.60, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000136081, 'Ho***', 'Jac****', 0, 'N', '3475 ********** Dr', 'SAN DIEGO', 'CA', 'UNK', 'UNKNOWN', 's********@aol.com', '1964-04-07', 'California', 'Slot', 328.39, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000125172, 'Ch***', 'Zuc*******', 0, 'Y', '6642 ********* Pl', 'RIALTO', 'CA', 'USA', '92377', 'c************@outlook.com', '1985-07-28', 'California', 'Slot', 499.10, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000164259, 'Al**', 'Sei**', 0, 'Y', '1198 ********** Dr', 'EL MIRAGE', 'AZ', 'USA', '85335', 'a***********@cox.net', '1992-07-16', 'California', 'Slot', 700.57, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000173294, 'Te*******', 'Ber****', 0, 'N', '8402 ********* Dr', 'RENO', 'NV', 'UNK', 'UNKNOWN', 't*********@hotmail.com', '1972-08-10', 'California', 'Slot', 1763.23, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000208066, 'Et***', 'Won*', 0, 'Y', '8838 *********** Dr', 'NORTH HOLLYWOOD', 'CA', 'USA', '91605', 'f**********@hotmail.com', '1980-05-17', 'California', 'Slot', 883.42, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000213450, 'Fr*****', 'Zhe**', 0, 'Y', '3668 ******* Ct', 'INGLEWOOD', 'CA', 'USA', '90301', 'f**************@aim.com', '1945-01-03', 'California', 'Slot', 2343.26, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000074063, 'Ro****', 'Abr*******', 0, 'Y', '2616 *********** Rd', 'SIERRA MADRE', 'CA', 'USA', '91024', NULL, '1954-07-16', 'California', 'Table', 36.72, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000108653, 'Ma******', 'Mar*************', 0, 'N', '622 *********** Pl', 'DOLAN SPRINGS', 'AZ', 'UNK', 'UNKNOWN', 'm*********************@hotmail.com', '1966-12-26', 'California', 'Slot', 4057.00, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000123097, 'Ma***', 'Wil*****', 0, 'Y', '6817 ****** Pl', 'SAN DIEGO', 'CA', 'USA', '92114', 'm*************@att.net', '1977-03-23', 'California', 'Table', 31.79, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000146529, 'Pe************', 'Lim', 0, 'Y', '2123 ********* St', 'GILBERT', 'AZ', 'USA', '85297', NULL, '1979-03-09', 'California', 'Table', 568.90, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000188773, 'He***', 'Mor**', 0, 'Y', '3103 ******** St', 'LOS ANGELES', 'CA', 'USA', '90018', 'h*************@outlook.com', '1984-12-17', 'California', 'Slot', 658.98, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000218093, 'Ji', 'Eve****', 0, 'Y', '2884 ********* Ct', 'LAKE FOREST', 'CA', 'USA', '92630', 'a*******@netscape.net', '1948-05-06', 'California', 'Slot', 458.66, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000191647, 'Al********', 'Gra***', 0, 'N', '1096 ******* Pl', 'GLENDALE', 'AZ', 'UNK', 'UNKNOWN', NULL, '1988-07-02', 'California', 'Slot', 1341.31, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000227353, 'Ho***', 'Pon*', 0, 'Y', '406 ****** Pl', 'LAS VEGAS', 'NV', 'USA', '89102', 'c******@mac.com', '1977-02-06', 'LasVegas', 'Table', 95.46, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000254930, 'Fr****', 'Gla**', 0, 'Y', '839 ****** Pl', 'LAS VEGAS', 'NV', 'USA', '89109', 'f*****@roadrunner.com', '1971-07-15', 'LasVegas', 'Table', 66.67, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000189847, 'Jo**', 'Hor***', 0, 'N', '710 *********** St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'b*********@att.net', '1979-05-03', 'California', 'Table', 967.00, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000237572, 'Ja***', 'Alo***', 0, 'Y', '1749 ************ Rd', 'LAS VEGAS', 'NV', 'USA', '89109', 'j***********@comcast.net', '1957-07-14', 'LasVegas', 'Slot', 682.66, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000204169, 'Li****', 'Fel******************', 0, 'Y', '1910 ********** St', 'HENDERSON', 'NV', 'USA', '89011', NULL, '1963-09-26', 'LasVegas', 'Table', 244.69, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000146472, 'Ja*****', 'Mat*****', 0, 'Y', '2444 ***** Ct', 'GARDENA', 'CA', 'USA', '90249', 'j***********@sbcglobal.net', '1976-08-22', 'California', 'Table', 3221.17, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000180790, 'Di***', 'Bro**', 0, 'Y', '518 ********** Ct', 'SAN DIEGO', 'CA', 'USA', '92111', 's**********@sbcglobal.net', '1961-07-01', 'California', 'Slot', 39.67, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000233935, 'Ne****', 'Pac****', 0, 'Y', '8887 ******** Dr', 'OXNARD', 'CA', 'USA', '93030', 'e*********@yahoo.com', '1945-09-12', 'California', 'Slot', 44.61, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000233821, 'So***', 'Han', 0, 'Y', '2226 ********** Rd', 'OXNARD', 'CA', 'USA', '93033', 'e*********@outlook.com', '1971-07-03', 'California', 'Slot', 2460.94, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000253705, 'Ja***', 'Den*****', 0, 'Y', '485 ******* Ct', 'RAMONA', 'CA', 'USA', '92065', 'm***********@comcast.net', '2001-12-07', 'California', 'Slot', 279.42, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000280589, 'Ch*********', 'Luo**', 0, 'Y', '3422 ******* Ave', 'LONG BEACH', 'CA', 'USA', '90815', 'n*****@mac.com', '1987-02-05', 'California', 'Slot', 634.29, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000310705, 'Uz***', 'Alb*', 0, 'N', '3085 ****** Rd', 'NORTH HOLLYWOOD', 'CA', 'UNK', 'UNKNOWN', 'u*******@aim.com', '1984-08-23', 'California', 'Table', 385.68, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10582203, 'Co******', 'Lee', 0, 'Y', '5808 ******* St', 'CANYON COUNTRY', 'CA', 'USA', '91351', 'c*******@sbcglobal.net', '1959-06-09', 'California', 'Slot', 751.90, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 200067408, 'Ti**', 'Mas****', 0, 'Y', '4125 ******* Dr', 'RIVERSIDE', 'CA', 'USA', '92505', 't********@hotmail.com', '1971-05-02', 'California', 'Slot', 115.22, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 681916874, 'Or**', 'Wan*', 0, 'Y', '5639 ************ Rd', 'LOS ANGELES', 'CA', 'USA', '90020', 'y*****@verizon.net', '1974-09-02', 'California', 'Slot', 147.69, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 792173514, 'Sa****', 'Kim', 0, 'Y', '5072 ********** Ave', 'BAKERSFIELD', 'CA', 'USA', '93311', 's***********@hotmail.net', '1962-09-21', 'California', 'Slot', 3680.49, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000317360, 'Se****', 'Nel***', 0, 'Y', '5105 ********** Pl', 'HEMET', 'CA', 'USA', '92545', 'j***************@roadrunner.com', '1965-08-03', 'California', 'Slot', 66.19, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000297798, 'Xi*****', 'Esc******', 0, 'Y', '9447 *********** St', 'YORBA LINDA', 'CA', 'USA', '92886', NULL, '1988-07-21', 'California', 'Table', 2506.14, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10200180, 'Hu**', 'Esc******', 0, 'Y', '5967 ******* Ave', 'IRVINE', 'CA', 'USA', '92618', NULL, '1965-03-06', 'California', 'Slot', 184.71, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10265301, 'Ca***', 'Mce****', 0, 'Y', '9482 ********** Ct', 'LONG BEACH', 'CA', 'USA', '90802', 'b******@sbcglobal.net', '1978-09-10', 'California', 'Slot', 1753.55, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10312520, 'He***', 'Han***', 0, 'Y', '769 ******** Ave', 'FONTANA', 'CA', 'USA', '92336', NULL, '1954-08-11', 'California', 'Slot', 856.62, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000013737, 'Jo***', 'Her**************', 0, 'Y', '3557 ************ Pl', 'LOS ANGELES', 'CA', 'USA', '90057', 't**********@mac.com', '1983-05-09', 'California', 'Slot', 3876.93, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000030946, 'Da****', 'Gon*****', 0, 'Y', '4580 ************ Dr', 'HIGHLAND', 'CA', 'USA', '92346', 'd********@outlook.com', '1967-10-25', 'California', 'Table', 170.84, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000041056, 'Ma****', 'Mai', 0, 'Y', '6432 *********** Pl', 'FRESNO', 'CA', 'USA', '93727', NULL, '1970-10-02', 'California', 'Slot', 4714.51, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000064361, 'Br***', 'Jia****************', 0, 'Y', '5304 ********** Ave', 'SAN DIEGO', 'CA', 'USA', '92127', 'b*********@roadrunner.com', '1961-04-28', 'California', 'Slot', 100.92, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000152726, 'An****', 'Aco***', 0, 'Y', '9094 ********** Ave', 'SAN FERNANDO', 'CA', 'USA', '91340', 'a********@verizon.net', '1963-11-24', 'California', 'Slot', 4376.48, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000178739, 'Sh****', 'Jac*********', 0, 'N', '9193 ******* Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1977-07-28', 'California', 'Table', 324.73, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000185418, 'Mi****', 'Cam****', 0, 'Y', '7175 ************ Ave', 'CHULA VISTA', 'CA', 'USA', '91911', 'd*****@rocketmail.com', '1953-06-07', 'California', 'Slot', 140.68, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10568449, 'Sh****', 'Kim', 0, 'Y', '5184 ****** Pl', 'LOS ANGELES', 'CA', 'USA', '90029', 's*********@outlook.com', '1984-05-09', 'California', 'Slot', 48.95, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10513718, 'Bi***', 'Blo***', 0, 'Y', '2499 ******** Ave', 'LAS VEGAS', 'NV', 'USA', '89147', '1*********@hotmail.com', '1976-04-03', 'LasVegas', 'Table', 1717.11, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10572439, 'Jo******', 'Car*****', 0, 'Y', '1701 ************ Rd', 'GILBERT', 'AZ', 'USA', '85296', 'j**********@yahoo.com', '1976-07-22', 'California', 'Slot', 408.63, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10577376, 'Ma*****', 'Ran****', 0, 'Y', '1425 ******* Ave', 'HENDERSON', 'NV', 'USA', '89014', 'm************@outlook.com', '1989-11-17', 'LasVegas', 'Slot', 274.49, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 101660548, 'Do****', 'Van****', 0, 'Y', '6259 ******** Rd', 'RENO', 'NV', 'USA', '89523', 'd**********@rocketmail.com', '1959-06-17', 'California', 'Slot', 3146.36, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 101720439, 'Ph****', 'Alf***', 0, 'N', '8081 ************ St', 'CHINO', 'CA', 'UNK', 'UNKNOWN', 'c*********@yahoo.com', '1991-04-24', 'California', 'Table', 50.25, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 187285993, 'Di***', 'Waf**', 0, 'Y', '3405 ******** Rd', 'LAS VEGAS', 'NV', 'USA', '89109', 'd******@mac.com', '1950-11-26', 'LasVegas', 'Slot', 292.92, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10308300, 'Lo**', 'Pin***', 0, 'Y', '6560 ***** Ct', 'ESCONDIDO', 'CA', 'USA', '92027', NULL, '1949-03-08', 'California', 'Slot', 309.98, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10564879, 'Or****', 'Coh**', 0, 'Y', '117 ********** St', 'LOS ANGELES', 'CA', 'USA', '90043', NULL, '1963-12-04', 'California', 'Slot', 47.35, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10572135, 'Da****', 'Liz******', 0, 'Y', '2949 ******** Pl', 'EL MONTE', 'CA', 'USA', '91732', 'b**********@att.net', '1974-05-16', 'California', 'Slot', 27.89, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101683075, 'Pa**', 'Sua**', 0, 'Y', '5604 ********* St', 'SURPRISE', 'AZ', 'USA', '85388-1502', 'i*******@aol.com', '1954-10-04', 'California', 'Slot', 124.05, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 100930273, 'Mo*******', 'Che*', 0, 'Y', '1886 ******* Pl', 'SPARKS', 'NV', 'USA', '89431', 'm********@gmail.com', '1969-08-14', 'California', 'Slot', 1110.65, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 101689212, 'Br*****', 'O''B****', 0, 'Y', '8234 ******* St', 'SPARKS', 'NV', 'USA', '89436', 'b**********@mac.com', '1991-10-12', 'California', 'Slot', 183.92, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200133337, 'So****', 'Dev**', 0, 'Y', '6497 ****** Pl', 'RENO', 'NV', 'USA', '89506', 's************@cox.net', '1972-11-27', 'California', 'Slot', 134.88, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 200221240, 'Pe***', 'Gus*****', 0, 'Y', '1261 ********** St', 'BUCKEYE', 'AZ', 'USA', '85326', 'p*********@aim.com', '1978-02-19', 'California', 'Slot', 224.44, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 200250191, 'Ng****', 'Fra******', 0, 'N', '4763 *********** Ct', 'PEORIA', 'AZ', 'UNK', 'UNKNOWN', 'k**********@cox.net', '1971-01-18', 'California', 'Slot', 4553.72, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000006249, 'Da****', 'Lor*', 0, 'Y', '5413 *********** Pl', 'LAS VEGAS', 'NV', 'USA', '89129', 'm*******@mac.com', '1987-01-01', 'LasVegas', 'Table', 61.46, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000011814, 'Me****', 'Vil*********', 0, 'Y', '5675 ************ Dr', 'PEORIA', 'AZ', 'USA', '85383', NULL, '1979-09-26', 'California', 'Slot', 28.91, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000036780, 'Aa***', 'Alv****', 0, 'Y', '3293 ***** Dr', 'COMPTON', 'CA', 'USA', '90220', 'a***************@rocketmail.com', '1964-12-08', 'California', 'Table', 2062.76, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000063976, 'Yi*****', 'Lop**', 0, 'N', '2047 ******** Dr', 'GLENDALE', 'CA', 'UNK', 'UNKNOWN', 'y***@att.net', '1967-04-03', 'California', 'Table', 451.06, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000087288, 'Kr*****', 'Ran***', 0, 'Y', '3095 ******* Ct', 'OCEANSIDE', 'CA', 'USA', '92056', 'k************@hotmail.com', '1978-07-22', 'California', 'Table', 28.18, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000115632, 'Ir**', 'Ban***', 0, 'Y', '270 *********** Rd', 'BAKERSFIELD', 'CA', 'USA', '93312', 'i***********@roadrunner.com', '1969-09-26', 'California', 'Slot', 1998.40, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10536704, 'Ed****', 'Eat**', 0, 'N', '2237 ********* St', 'HENDERSON', 'NV', 'UNK', 'UNKNOWN', 's************@yahoo.com', '1969-08-07', 'California', 'Slot', 1614.53, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10247877, 'Ji*****', 'Men****', 0, 'Y', '7656 ************ Dr', 'LAS VEGAS', 'NV', 'USA', '89138', 'j*************@gmail.com', '1987-12-25', 'LasVegas', 'Table', 1322.92, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10381307, 'Mi*****', 'Men***', 0, 'Y', '4661 ************ Pl', 'SCOTTSDALE', 'AZ', 'USA', '85259', 'm******@hotmail.com', '1968-05-22', 'California', 'Slot', 73.22, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10385402, 'Pe***', 'Pha*', 0, 'Y', '2359 ******** Dr', 'TEMECULA', 'CA', 'USA', '92592', 'p******@hotmail.net', '1959-11-28', 'California', 'Slot', 1973.37, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10437444, 'Om**', 'Kam****', 0, 'Y', '3739 *********** Ct', 'WHITTIER', 'CA', 'USA', '90603', 'o******@outlook.com', '1976-02-06', 'California', 'Slot', 172.36, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10337180, 'Ju****', 'Mcb****', 0, 'Y', '3881 ******** Dr', 'HENDERSON', 'NV', 'USA', '84758', NULL, '1987-05-25', 'California', 'Table', 837.15, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10387668, 'Yo******', 'Sex***', 0, 'Y', '996 ********** Rd', 'LOS ANGELES', 'CA', 'USA', '90044', NULL, '1969-02-01', 'California', 'Slot', 36.18, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10542664, 'Li**', 'Pri**', 0, 'Y', '3438 *********** Ct', 'MURRIETA', 'CA', 'USA', '84770', 'j******@hotmail.com', '1960-03-22', 'California', 'Slot', 3880.04, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10552379, 'Ed*****', 'Oad**', 0, 'N', '549 ****** Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1987-10-27', 'California', 'Table', 150.45, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10553215, 'Mi*****', 'Dua***', 0, 'Y', '894 ***** Pl', 'GILBERT', 'AZ', 'USA', '85298', 'm********@comcast.net', '1973-02-10', 'California', 'Table', 829.98, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000009535, 'Vi*****', 'Vel****', 0, 'Y', '187 ******* Ave', 'LONG BEACH', 'CA', 'USA', '90805', 't**********@hotmail.net', '1975-05-28', 'California', 'Slot', 166.18, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000062545, 'Sh****', 'Guo', 0, 'Y', '7886 ******** Dr', 'TARZANA', 'CA', 'USA', '91356', NULL, '1994-06-19', 'California', 'Table', 4543.26, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000079359, 'Gr******', 'Naw***', 0, 'Y', '2382 ******** Pl', 'WINNETKA', 'CA', 'USA', '91306', 'g***********@gmail.com', '1982-05-04', 'California', 'Slot', 53.99, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000084546, 'Bo*****', 'Sar******', 0, 'N', '3502 ***** Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', NULL, '1988-03-21', 'California', 'Table', 1170.05, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000093471, 'Da***', 'Her****', 0, 'Y', '8524 ******* Dr', 'BEVERLY HILLS', 'CA', 'USA', '90211', NULL, '1978-07-22', 'California', 'Table', 285.52, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000091842, 'Ja****', 'Cro***', 0, 'Y', '3281 ***** St', 'FONTANA', 'CA', 'USA', '92336', 'j***********@hotmail.net', '1966-02-04', 'California', 'Slot', 43.51, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000113608, 'Er**', 'Mcl*****', 0, 'Y', '8590 ********** St', 'SAN DIEGO', 'CA', 'USA', '92105', 'e*************@comcast.net', '1976-07-12', 'California', 'Table', 1345.61, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000118869, 'Ar****', 'Fer****', 0, 'Y', '3236 ************ Ct', 'GLENDALE', 'CA', 'USA', '91203', NULL, '1979-09-05', 'California', 'Table', 612.29, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000121105, 'Ma*****', 'Vil*****', 0, 'Y', '6783 ********** Ave', 'LOS ANGELES', 'CA', 'USA', '90034', 's*************@aol.com', '1972-11-04', 'California', 'Slot', 45.12, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000153883, 'An****', 'Lar*****', 0, 'N', '5445 ********* Ct', 'HENDERSON', 'NV', 'UNK', 'UNKNOWN', 'c************@cox.net', '1992-04-23', 'California', 'Table', 477.85, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000217564, 'Ma****', 'Ten*', 0, 'N', '7830 ***** Rd', 'LAKE HAVASU CITY', 'AZ', 'UNK', 'UNKNOWN', 'm***************@rocketmail.com', '1962-02-28', 'California', 'Both', 84.31, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000237515, 'Or**', 'Joh****', 0, 'Y', '5431 ********** Pl', 'LOS ANGELES', 'CA', 'USA', '90028', NULL, '1995-11-25', 'California', 'Table', 133.93, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000234192, 'An****', 'Car*****', 0, 'Y', '6048 ******** Pl', 'LEMON GROVE', 'CA', 'USA', '91945', 'a**********@verizon.net', '2001-09-14', 'California', 'Slot', 141.08, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000245665, 'Ca****', 'Lau', 0, 'Y', '8648 ***** Rd', 'MONTEREY PARK', 'CA', 'USA', '91754', 'c**************@rocketmail.com', '1973-10-14', 'California', 'Table', 344.40, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000267983, 'He*******', 'Sha***', 0, 'Y', '1611 ****** Ave', 'LAS VEGAS', 'NV', 'USA', '89129', 'c***********@aim.com', '1990-05-03', 'LasVegas', 'Slot', 1225.62, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000276917, 'Na****', 'Ros***', 0, 'Y', '7520 ********* Rd', 'CHINO', 'CA', 'USA', '91708', 'n***********@yahoo.com', '1973-10-09', 'California', 'Slot', 115.38, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000293018, 'Ta******', 'Ngu***', 0, 'Y', '8947 ********** Dr', 'SAN PEDRO', 'CA', 'USA', '90732', 'v***********@mac.com', '1981-04-05', 'California', 'Slot', 6445.90, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10261794, 'Br***', 'Ara***', 0, 'Y', '6631 ******* Ct', 'FALLBROOK', 'CA', 'USA', '92028', NULL, '1949-09-21', 'California', 'Slot', 1166.19, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10276570, 'Ja***', 'Kin***', 0, 'Y', '8989 ********* Rd', 'LOS ANGELES', 'CA', 'USA', '90053', 'j*******@hotmail.com', '1969-07-28', 'California', 'Slot', 693.40, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10072139, 'Tr**', 'Chh**', 0, 'Y', '9342 *********** Rd', 'LAS VEGAS', 'NV', 'USA', '89121', 'o*********@aim.com', '1969-01-13', 'LasVegas', 'Slot', 112.99, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10472708, 'Yo******', 'Lap*****', 0, 'Y', '538 ***** St', 'NORTH HOLLYWOOD', 'CA', 'USA', '91606', NULL, '1976-03-21', 'California', 'Table', 773.87, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10212348, 'Em******', 'Lev', 0, 'Y', '388 ******* Ave', 'LAS VEGAS', 'NV', 'USA', '89122', NULL, '1987-03-13', 'LasVegas', 'Table', 39.10, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10258774, 'Ro****', 'Sil**', 0, 'Y', '7505 ****** Ct', 'LAS VEGAS', 'NV', 'USA', '89139', NULL, '1989-09-24', 'LasVegas', 'Slot', 256.03, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10417417, 'Lo****', 'Cam****', 0, 'Y', '6773 ****** Dr', 'FULLERTON', 'CA', 'USA', '92831', 'm***************@aol.com', '1967-02-04', 'California', 'Slot', 26.99, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10513615, 'Ha****', 'Dea***', 0, 'Y', '2239 ******* Rd', 'ALTADENA', 'CA', 'USA', '91001', 'h***********@outlook.com', '1966-03-14', 'California', 'Slot', 961.12, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000071571, 'Ca*******', 'Nie**', 0, 'Y', '6453 ***** Dr', 'WELDON', 'CA', 'USA', '93283', NULL, '1954-11-23', 'California', 'Slot', 325.32, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 200294925, 'An****', 'Yam***', 0, 'Y', '2343 ****** Rd', 'SANTA CLARITA', 'CA', 'USA', '91350', 'a***************@sbcglobal.net', '1982-04-17', 'California', 'Both', 1273.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000006653, 'Ed****', 'Eme****', 0, 'Y', '7203 ********* Dr', 'LOMITA', 'CA', 'USA', '90717', NULL, '1991-08-16', 'California', 'Table', 4565.12, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000161469, 'De*****', 'Pip***', 0, 'Y', '3151 ********* Dr', 'BEAUMONT', 'CA', 'USA', '92223', 'd********@sbcglobal.net', '1968-04-19', 'California', 'Slot', 511.37, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000185551, 'Cr***', 'Lee', 0, 'Y', '6115 ***** Dr', 'GLENDALE', 'AZ', 'USA', '85306', 'l************@netscape.net', '1968-08-13', 'California', 'Slot', 2911.14, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000088877, 'Ro****', 'Kim', 0, 'Y', '6536 ******* St', 'GILBERT', 'AZ', 'USA', '85233', 'd*******@yahoo.com', '1952-11-27', 'California', 'Slot', 1383.54, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000242241, 'Ne**', 'Oca**********', 0, 'Y', '9083 ***** Rd', 'SAN PEDRO', 'CA', 'USA', '90732', 'n*******@hotmail.net', '1966-11-25', 'California', 'Slot', 3904.56, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000253328, 'Sh****', 'Yan*', 0, 'N', '8883 ********** Rd', 'HENDERSON', 'NV', 'UNK', 'UNKNOWN', 'm*********@gmail.com', '1958-03-07', 'California', 'Slot', 336.64, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000132924, 'Qi******', 'Moc******', 0, 'N', '9318 ********** Pl', 'HOLTVILLE', 'CA', 'UNK', 'UNKNOWN', NULL, '1984-06-03', 'California', 'Table', 581.14, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000135489, 'St****', 'Say***', 0, 'Y', '3297 *********** Ct', 'LAS VEGAS', 'NV', 'USA', '89183', 'u********@outlook.com', '1980-07-28', 'LasVegas', 'Table', 398.86, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000204348, 'Al****', 'Cla**', 0, 'Y', '4511 ***** St', 'HESPERIA', 'CA', 'USA', '92345', 'g*******@mac.com', '1971-07-16', 'California', 'Table', 52.21, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000227144, 'Ga*****', 'Bri*****', 0, 'Y', '2449 ************ Dr', 'BEAUMONT', 'CA', 'USA', '92223', 'g*************@gmail.com', '1970-05-07', 'California', 'Slot', 682.32, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000226217, 'Br*****', 'Boy**', 0, 'Y', '4529 ********* Ct', 'GARDEN GROVE', 'CA', 'USA', '92840', 'b*********@rocketmail.com', '1994-10-05', 'California', 'Slot', 82.71, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000265172, 'St****', 'Kin*', 0, 'N', '9081 ***** Pl', 'OXNARD', 'CA', 'UNK', 'UNKNOWN', 's************@outlook.com', '1991-06-23', 'California', 'Slot', 87.01, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000311441, 'Ne*******', 'How***', 0, 'Y', '3115 ***** Pl', 'YUCAIPA', 'CA', 'USA', '92399', 'n*********@roadrunner.com', '1983-06-25', 'California', 'Table', 33.11, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000284957, 'Jo****', 'Doh****', 0, 'Y', '5456 ********* Ct', 'SAN DIEGO', 'CA', 'USA', '92101', NULL, '1984-06-06', 'California', 'Slot', 82.93, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10235946, 'Ti**', 'Mcd*****', 0, 'Y', '8162 ************ St', 'LAS VEGAS', 'NV', 'USA', '89109', 'i****************@att.net', '1981-05-06', 'LasVegas', 'Slot', 40.07, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000122176, 'Lo***', 'Mad*****', 0, 'N', '3156 ******* Pl', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1988-01-27', 'California', 'Table', 172.83, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000246094, 'Li****', 'Ren', 0, 'Y', '3984 ****** Ave', 'FULLERTON', 'CA', 'USA', '92835', 'l***********@sbcglobal.net', '1962-05-15', 'California', 'Slot', 1770.29, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000268058, 'Ar**', 'Ber****', 0, 'Y', '6737 ********* St', 'VALLEY VILLAGE', 'CA', 'USA', '91607', 'a************@outlook.com', '1999-04-05', 'California', 'Both', 55.92, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000305527, 'Fr*******', 'Gar**', 0, 'Y', '8984 ********* Ave', 'LAS VEGAS', 'NV', 'USA', '89117', 'f******@roadrunner.com', '1975-06-24', 'LasVegas', 'Slot', 52.14, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000266197, 'Al***', 'Par***', 0, 'Y', '5405 ********** Dr', 'CLAREMONT', 'CA', 'USA', '91711', 'a*********@mac.com', '1978-12-06', 'California', 'Table', 2711.34, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000266967, 'Ke***', 'Pha*', 0, 'Y', '1564 ****** Ct', 'BAKERSFIELD', 'CA', 'USA', '93387', 'm********@rocketmail.com', '1975-07-02', 'California', 'Slot', 6436.88, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000274544, 'Re****', 'Dea*', 0, 'Y', '8923 ********** Ct', 'LOS ANGELES', 'CA', 'USA', '90047', NULL, '1963-01-23', 'California', 'Table', 44.88, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10563530, 'Ti**', 'Col***', 0, 'Y', '1998 *********** St', 'CHULA VISTA', 'CA', 'USA', '91915', 't*********@hotmail.com', '1963-11-11', 'California', 'Table', 115.39, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 101213286, 'Wi*****', 'And******', 0, 'N', '905 ******** Dr', 'SEAL BEACH', 'CA', 'UNK', 'UNKNOWN', NULL, '1944-12-10', 'California', 'Slot', 35.42, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101563198, 'Ca**', 'Tua*', 0, 'Y', '5061 ****** Dr', 'FALLON', 'NV', 'USA', '89406', 'c*******@cox.net', '1970-12-07', 'California', 'Slot', 510.56, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 101597994, 'Ji***', 'Sav******', 0, 'N', '3530 ****** Dr', 'LAS VEGAS', 'NV', 'UNK', 'UNKNOWN', 'j**********@att.net', '1969-06-23', 'California', 'Slot', 299.74, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 781101944, 'Ro*****', 'Lee', 0, 'Y', '5824 *********** St', 'WEST COVINA', 'CA', 'USA', '91790', 'r********@cox.net', '1966-11-16', 'California', 'Slot', 2437.92, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000020487, 'Ke*****', 'Mes******', 0, 'Y', '4909 ******* Ct', 'CHINO HILLS', 'CA', 'USA', '91709', NULL, '1967-09-12', 'California', 'Slot', 1569.32, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000025116, 'Wi*****', 'Chu**', 0, 'N', '4281 ****** Ave', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'b***@yahoo.com', '1948-12-05', 'California', 'Table', 109.46, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000052172, 'Ma***', 'Kim', 0, 'Y', '2206 ****** Rd', 'BURBANK', 'CA', 'USA', '91502', NULL, '1936-04-16', 'California', 'Slot', 1207.61, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 100628044, 'Ro****', 'Pat***', 0, 'Y', '3980 ************ St', 'FALLON', 'NV', 'USA', '89406', 'h*******@verizon.net', '1956-12-19', 'California', 'Slot', 27.26, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 100676256, 'Ho**', 'Man******', 0, 'Y', '8005 ********* Ave', 'LAS VEGAS', 'NV', 'USA', '89103', NULL, '1971-06-13', 'LasVegas', 'Table', 1130.14, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 101497577, 'Ga**', 'Mor***', 0, 'Y', '1076 *********** Pl', 'WINNEMUCCA', 'NV', 'USA', '89445', 'a************@aim.com', '1969-01-11', 'California', 'Slot', 147.53, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000005654, 'Je******', 'Can*****', 0, 'Y', '2370 *********** Ct', 'LANCASTER', 'CA', 'USA', '93536', 'j*********@verizon.net', '1980-09-26', 'California', 'Table', 73.57, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101284937, 'Ky****', 'Cha*****', 0, 'Y', '42 ************ Ave', 'RENO', 'NV', 'USA', '89523', 'g************@hotmail.net', '1963-03-25', 'California', 'Slot', 4140.62, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 200159878, 'Da***', 'Cle***', 0, 'Y', '150 ******** Rd', 'SELMA', 'CA', 'USA', '93662', 'd*********@verizon.net', '1975-12-24', 'California', 'Table', 2374.40, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 410398482, 'Jo*', 'Tsa*', 0, 'Y', '5746 ****** Dr', 'GLENDALE', 'CA', 'USA', '91204', 'j********@gmail.com', '1964-12-24', 'California', 'Slot', 582.66, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000057550, 'Ra*', 'Sim****', 0, 'N', '801 ****** Ct', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1972-09-28', 'California', 'Slot', 104.68, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000073860, 'Ju*****', 'Bar***', 0, 'Y', '8333 ******** Rd', 'YUMA', 'AZ', 'USA', '85367', NULL, '1940-04-20', 'California', 'Slot', 1644.18, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000169787, 'Al*', 'Rac***', 0, 'N', '5196 ****** St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1964-09-20', 'California', 'Slot', 1033.60, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000194321, 'Fe**', 'Pri****', 0, 'Y', '966 ********* Ct', 'LAS VEGAS', 'NV', 'USA', '89178', 'l**********@sbcglobal.net', '1964-07-13', 'LasVegas', 'Table', 33.03, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000223369, 'Al*****', 'Tra*', 0, 'Y', '7047 ************ Ave', 'LAKE FOREST', 'CA', 'USA', '92630', NULL, '1978-05-07', 'California', 'Slot', 1100.15, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000317908, 'Ig**', 'Hud****', 0, 'Y', '7819 ***** Pl', 'SAN DIEGO', 'CA', 'USA', '92101', 'j****************@rocketmail.com', '1999-09-17', 'California', 'Table', 1114.54, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10173379, 'Ch**', 'Whi*****', 0, 'N', '9005 ******* Dr', 'SAN DIEGO', 'CA', 'UNK', 'UNKNOWN', NULL, '1992-07-13', 'California', 'Table', 88.46, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10475322, 'Se*****', 'Seo', 0, 'Y', '4967 ************ Ave', 'OCEANSIDE', 'CA', 'USA', '92054', 's************@aol.com', '1946-11-28', 'California', 'Slot', 1703.09, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10532230, 'Ru*****', 'Kaz******', 0, 'Y', '78 ******* Pl', 'SUN CITY', 'AZ', 'USA', '85373', 'r***********@mac.com', '1953-12-18', 'California', 'Slot', 3428.92, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10538679, 'Sa*', 'Ver******', 0, 'N', '7733 ******** Rd', 'CARSON CITY', 'NV', 'UNK', 'UNKNOWN', NULL, '1984-04-19', 'California', 'Table', 4288.53, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10579641, 'Se****', 'Bra***', 0, 'Y', '2537 ******* Ct', 'BONITA', 'CA', 'USA', '91902', 'v**********@aim.com', '1976-09-19', 'California', 'Slot', 286.57, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10140658, 'Si******', 'Sel***', 0, 'Y', '8361 *********** Ave', 'BURBANK', 'CA', 'USA', '91504', 's*******@att.net', '1976-02-15', 'California', 'Slot', 385.56, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10249526, 'Sh******', 'Kiz******', 0, 'Y', '7464 ******** St', 'MISSION HILLS', 'CA', 'USA', '91345', NULL, '1973-02-01', 'California', 'Table', 2934.82, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000021537, 'Yo*****', 'Sit*', 0, 'Y', '358 ******** St', 'PALMDALE', 'CA', 'USA', '93552', NULL, '1971-09-13', 'California', 'Slot', 64.43, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10529717, 'Fe******', 'Oro**********', 0, 'Y', '6497 ******* Rd', 'LOS ANGELES', 'CA', 'USA', '90006', 'f***********@aim.com', '1984-10-02', 'California', 'Table', 178.45, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 10530344, 'Ji', 'Hos*****', 0, 'Y', '787 ********** Ct', 'LOS ANGELES', 'CA', 'USA', '90005', NULL, '1978-02-18', 'California', 'Slot', 197.79, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 127643540, 'Yu', 'Chu', 0, 'Y', '4240 ******* Dr', 'LAS VEGAS', 'NV', 'USA', '89148', 'm**************@roadrunner.com', '1967-01-16', 'LasVegas', 'Table', 1170.52, 1002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000127746, 'Lu**', 'Pad****', 0, 'Y', '6950 *********** St', 'MENIFEE', 'CA', 'USA', '92584', NULL, '1976-03-01', 'California', 'Table', 60.61, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000137555, 'Le**', 'Wil***', 0, 'Y', '2662 ********** Ave', 'FOUNTAIN HILLS', 'AZ', 'USA', '85268', 'l***************@comcast.net', '1954-07-18', 'California', 'Slot', 2044.28, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000030468, 'Ju*****', 'Thu****', 0, 'Y', '7338 ******* Dr', 'LOS ANGELES', 'CA', 'USA', '90011', NULL, '1951-03-21', 'California', 'Slot', 586.16, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000209689, 'Ph***', 'Ves**', 0, 'Y', '3069 ************ Pl', 'SANTA ANA', 'CA', 'USA', '92707', NULL, '1978-07-18', 'California', 'Table', 40.82, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000209789, 'Be**', 'Ave*****', 0, 'Y', '7571 ****** Dr', 'LAS VEGAS', 'NV', 'USA', '89135', NULL, '1974-03-20', 'LasVegas', 'Table', 129.14, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000093213, 'Ga****', 'Bro**', 0, 'Y', '4603 ****** Ct', 'TUCSON', 'AZ', 'USA', '85739', 'l******@aol.com', '1952-09-13', 'California', 'Slot', 649.17, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000299098, 'Ry**', 'For****', 0, 'Y', '3915 *********** Rd', 'LAS VEGAS', 'NV', 'USA', '89138', 'd*********@rocketmail.com', '1982-06-23', 'LasVegas', 'Slot', 104.27, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000315010, 'Ab******', 'Cho*', 0, 'N', '704 ********** St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'a******@mac.com', '1978-06-21', 'California', 'Table', 40.77, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000161408, 'Sh*****', 'Kee', 0, 'Y', '1231 ********* Ave', 'LOS ANGELES', 'CA', 'USA', '90044', 's*********@verizon.net', '1988-02-17', 'California', 'Slot', 3212.16, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000178331, 'Th************', 'Zha**', 0, 'Y', '5433 *********** Ct', 'LA MESA', 'AZ', 'USA', '85206', 't********@comcast.net', '1965-01-23', 'California', 'Table', 2212.77, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000210110, 'He****', 'Hut*****', 0, 'Y', '4321 ****** Rd', 'GLENDALE', 'AZ', 'USA', '85303', 'h*********************@comcast.net', '1990-03-17', 'California', 'Table', 6529.59, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000223798, 'Al***', 'Chr********', 0, 'N', '6313 ******* Ct', 'ORANGE', 'CA', 'UNK', 'UNKNOWN', 'a*****@netscape.net', '1985-07-04', 'California', 'Table', 157.59, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000146317, 'Se*******', 'Mur***', 0, 'N', '7803 ********* St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', NULL, '1985-02-24', 'California', 'Table', 1110.21, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000267322, 'Ji*****', 'Din*', 0, 'Y', '5431 ************ St', 'LAS VEGAS', 'NV', 'USA', '89103', NULL, '1982-08-09', 'LasVegas', 'Table', 51.47, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000204863, 'Ch*********', 'Kum***', 0, 'N', '3083 *********** Ct', 'WOODLAND HILLS', 'CA', 'UNK', 'UNKNOWN', 'c************@yahoo.com', '1979-06-26', 'California', 'Slot', 485.85, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000213828, 'Ag*****', 'Llo***', 0, 'Y', '9190 ****** Pl', 'RESEDA', 'CA', 'USA', '91337', 'r*********@rocketmail.com', '1965-08-18', 'California', 'Table', 160.97, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000303838, 'Jo**', 'Kis*****', 0, 'Y', '7840 ********* Pl', 'ROSEMEAD', 'CA', 'USA', '91770', 'j***********@att.net', '1975-01-23', 'California', 'Slot', 4242.04, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000304657, 'Sh*****', 'Ala***', 0, 'Y', '4818 ********** Dr', 'SAN DIEGO', 'CA', 'USA', '92107', NULL, '1973-05-03', 'California', 'Table', 454.64, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000293473, 'Er**', 'Mal***', 0, 'Y', '8848 ******* Ave', 'LAS VEGAS', 'NV', 'USA', '89131', 'k***********@netscape.net', '1957-05-10', 'LasVegas', 'Slot', 139.52, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000307668, 'Be*****', 'Sei***', 0, 'Y', '1275 *********** St', 'MONTEREY PARK', 'CA', 'USA', '91754', 'a******@aim.com', '1976-07-25', 'California', 'Slot', 1918.31, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000315299, 'Sa**', 'Rod******', 0, 'Y', '3936 ************ Ct', 'LAS VEGAS', 'NV', 'USA', '89144', 'o**********@hotmail.com', '1985-09-08', 'LasVegas', 'Table', 470.17, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000087637, 'Pe******', 'Iyi*****', 0, 'Y', '3818 ********* St', 'SAN DIEGO', 'CA', 'USA', '92110', 'n***********@hotmail.net', '1982-05-22', 'California', 'Slot', 62.11, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10582738, 'Er****', 'Yan*', 0, 'Y', '4149 ******* Ct', 'COMPTON', 'CA', 'USA', '90220', NULL, '1990-10-18', 'California', 'Table', 27.93, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000199307, 'Pe***', 'Hou', 0, 'Y', '7623 ********** Dr', 'TOLLESON', 'AZ', 'USA', '85353', 'm********@sbcglobal.net', '1960-10-11', 'California', 'Slot', 1548.13, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000280930, 'Yo*****', 'Sce*****', 0, 'Y', '1141 ****** St', 'LONG BEACH', 'CA', 'USA', '90805', 'y****************@comcast.net', '1958-10-24', 'California', 'Slot', 59.63, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10008660, 'Co*****', 'Her****', 0, 'Y', '6007 ******* Rd', 'LAS VEGAS', 'NV', 'USA', '89109', 'e**@netscape.net', '1959-08-08', 'LasVegas', 'Slot', 126.31, 1005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10077413, 'Or**', 'Tra*', 0, 'Y', '3784 ****** Ct', 'VALLEY VILLAGE', 'CA', 'USA', '91607', 'o********@netscape.net', '1993-11-17', 'California', 'Table', 436.72, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10417607, 'Ti**', 'Vya*', 0, 'Y', '4587 ************ Dr', 'LAS VEGAS', 'NV', 'USA', '89107', 'l********@netscape.net', '1960-11-15', 'LasVegas', 'Slot', 1371.83, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10355380, 'St*******', 'Cam*************', 0, 'Y', '3585 ******* St', 'LAS VEGAS', 'NV', 'USA', '89115', 's********@verizon.net', '1973-05-12', 'LasVegas', 'Slot', 58.53, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10412011, 'Em******', 'Uri**', 0, 'Y', '9381 ******** St', 'VALENCIA', 'CA', 'USA', '91354', 'a******@netscape.net', '1961-02-26', 'California', 'Table', 1595.11, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10551123, 'Ho***', 'Sch******', 0, 'Y', '6250 ********** St', 'LOS ANGELES', 'CA', 'USA', '90038', 'f**********@aol.com', '1989-03-18', 'California', 'Slot', 132.33, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10551624, 'Su******', 'Per****', 0, 'Y', '4662 ********* Dr', 'TEMPE', 'AZ', 'USA', '85281', 'n********@sbcglobal.net', '1982-09-06', 'California', 'Slot', 42.79, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10522968, 'Th****', 'Sim**', 0, 'N', '8424 ********** St', 'LA CRESCENTA', 'CA', 'UNK', 'UNKNOWN', 't******@gmail.com', '1988-02-18', 'California', 'Slot', 4012.62, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10531026, 'Co****', 'San****', 0, 'Y', '6594 ****** Ave', 'FRESNO', 'CA', 'USA', '93720', 'c******@yahoo.com', '1954-12-11', 'California', 'Slot', 43.92, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10532228, 'Jo**', 'San***', 0, 'N', '9072 ******** Rd', 'WEST COVINA', 'CA', 'UNK', 'UNKNOWN', 'j*********@hotmail.net', '1976-01-28', 'California', 'Slot', 695.20, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10092209, 'Sa****', 'Elk****', 0, 'Y', '282 ******* Ave', 'LAS VEGAS', 'NV', 'USA', '89169', NULL, '1960-12-01', 'LasVegas', 'Slot', 581.14, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 100986075, 'Vi******', 'She***', 0, 'Y', '123 ****** Rd', 'MONROVIA', 'CA', 'USA', '91016', 'v************@att.net', '1959-09-24', 'California', 'Slot', 93.74, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10359834, 'Mi*****', 'Wad****', 0, 'Y', '937 *********** Pl', 'SCOTTSDALE', 'AZ', 'USA', '85259', 'm*******@yahoo.com', '1965-07-01', 'California', 'Slot', 3193.27, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 118402085, 'Ma***', 'Can***', 0, 'Y', '3001 ********** Rd', 'AZUSA', 'CA', 'USA', '91702', 'm********@hotmail.com', '1969-12-07', 'California', 'Slot', 336.53, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 200211223, 'Ni******', 'Wri***', 0, 'Y', '6738 ***** Dr', 'SPARKS', 'NV', 'USA', '89436', 'n****************@sbcglobal.net', '1994-08-19', 'California', 'Table', 63.59, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 101421162, 'De*****', 'Fin*', 0, 'Y', '3187 ************ Ave', 'RENO', 'NV', 'USA', '89509', 'd**********@aim.com', '1967-11-23', 'California', 'Slot', 85.55, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10368885, 'Me**', 'Nan****', 0, 'Y', '5757 ******** Rd', 'WALNUT CREEK', 'CA', 'USA', '91789', NULL, '1944-04-21', 'California', 'Slot', 3191.35, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 918262094, 'Ho****', 'Cas***', 0, 'Y', '8414 *********** Ave', 'NORTH LAS VEGAS', 'NV', 'USA', '89081', 'm************@hotmail.net', '1972-05-05', 'LasVegas', 'Slot', 1892.59, 1001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10559575, 'Ma**', 'Rey**', 0, 'Y', '1118 ***** Ave', 'LAS VEGAS', 'NV', 'USA', '89104', 'x*****@gmail.com', '1950-07-27', 'LasVegas', 'Table', 66.54, 1007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10530611, 'Ma**********', 'Pha*', 0, 'Y', '269 ****** Ave', 'ONTARIO', 'CA', 'USA', '91761', 'm****************@yahoo.com', '1987-10-25', 'California', 'Slot', 124.18, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10540764, 'De****', 'Tum', 0, 'N', '3571 ********* Ct', 'SAN DIEGO', 'CA', 'UNK', 'UNKNOWN', 'd**********@sbcglobal.net', '1960-10-21', 'California', 'Slot', 2278.11, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 302127803, 'Di***', 'Ric*****', 0, 'Y', '6787 ******* Ave', 'BAKERSFIELD', 'CA', 'USA', '93309', 's***********@outlook.com', '1960-04-15', 'California', 'Slot', 2096.87, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 100434057, 'Al**', 'Naw***', 0, 'Y', '2308 ****** Dr', 'SPARKS', 'NV', 'USA', '89436', 'a*******@cox.net', '1942-05-15', 'California', 'Slot', 1221.46, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10493446, 'Do***', 'Chi****', 0, 'Y', '760 ******* Ave', 'RANCHO CUCAMONGA', 'CA', 'USA', '91739', 'd************@outlook.com', '1958-01-09', 'California', 'Slot', 5903.50, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10575037, 'Wi****', 'Car******', 0, 'Y', '4089 ****** St', 'SURPRISE', 'AZ', 'USA', '85378', 'w**********@comcast.net', '1954-07-05', 'California', 'Slot', 576.23, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000224960, 'Je***', 'Hal*****', 0, 'Y', '3438 ********* Rd', 'BELLFLOWER', 'CA', 'USA', '90706', 'j***********@gmail.com', '1993-11-13', 'California', 'Slot', 180.82, 2006, 'ADT'),
    ('2023-03-14', '2023-06-01', 101249313, 'Ju*********', 'Loa*', 0, 'Y', '2989 ***** Ave', 'HAWTHORNER', 'NV', 'USA', '89415', 'r*************@hotmail.com', '1960-10-17', 'California', 'Slot', 131.14, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000254533, 'Ri**', 'Ben*****', 0, 'Y', '1535 ****** Rd', 'LAGUNA NIGUEL', 'CA', 'USA', '92677', 'r*******@aim.com', '1968-06-16', 'California', 'Slot', 5895.83, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 612120497, 'Se**', 'Mil***', 0, 'Y', '6231 ********** Rd', 'LAS VEGAS', 'NV', 'USA', '89104', 'n************@outlook.com', '1986-05-04', 'LasVegas', 'Slot', 28.84, 1008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000042288, 'Jo**', 'Man****', 0, 'Y', '5102 ******* Ave', 'ESCONDIDO', 'CA', 'USA', '92026', NULL, '1955-07-18', 'California', 'Slot', 57.69, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000107832, 'Au***', 'Noh***', 0, 'Y', '2974 ****** Dr', 'GLENDALE', 'AZ', 'USA', '85305', NULL, '1958-12-28', 'California', 'Slot', 449.66, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000121102, 'Ne****', 'War*', 0, 'Y', '1572 ***** Ave', 'LOS ANGELES', 'CA', 'USA', '90034', 'r**************@hotmail.com', '1947-01-05', 'California', 'Slot', 1335.95, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000122708, 'Cl*****', 'Sho**', 0, 'Y', '3950 ********* St', 'LA MESA', 'AZ', 'USA', '85201', 'c********@aim.com', '1955-11-01', 'California', 'Slot', 3551.05, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10306311, 'Ne****', 'Mou****', 0, 'Y', '2308 ******* Rd', 'OXNARD', 'CA', 'USA', '93033', 'j*******@cox.net', '1978-07-17', 'California', 'Table', 4131.10, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10544424, 'Re*****', 'Lat**', 0, 'Y', '5849 ********* Ct', 'BEAUMONT', 'CA', 'USA', '92223', NULL, '1973-03-27', 'California', 'Slot', 330.07, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10115646, 'Jo**', 'Cal*****', 0, 'Y', '9397 ********** St', 'LOS ANGELES', 'CA', 'USA', '90044', 'j*******************@aim.com', '1991-08-09', 'California', 'Slot', 62.88, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10356888, 'Ib*****', 'Ven****', 0, 'Y', '7379 ******* Ct', 'VAN NUYS', 'CA', 'USA', '91401', 'l*********@verizon.net', '1957-10-07', 'California', 'Slot', 1670.87, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10525100, 'Cl******', 'Sim****', 0, 'Y', '5375 *********** Pl', 'CHATSWORTH', 'CA', 'USA', '91311', 'c****************@aim.com', '1988-10-28', 'California', 'Table', 443.19, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10535694, 'Re*****', 'Ngu***', 0, 'Y', '5293 ********* Pl', 'PHOENIX', 'AZ', 'USA', '84713', 'r**********@yahoo.com', '1961-11-02', 'California', 'Slot', 103.36, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 200353549, 'Ka******', 'Zap***', 0, 'Y', '2681 ************ Rd', 'SPARKS', 'NV', 'USA', '89431', 'k********@hotmail.com', '1976-03-12', 'California', 'Slot', 2820.30, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 358716148, 'Ho******', 'Vis***', 0, 'Y', '8148 ********* Pl', 'LOS ANGELES', 'CA', 'USA', '90043', NULL, '1954-02-23', 'California', 'Slot', 1327.30, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 767578921, 'Vl*******', 'Tho*****', 0, 'N', '1880 ********* Ct', 'PHOENIX', 'AZ', 'UNK', 'UNKNOWN', NULL, '1971-08-05', 'California', 'Table', 622.10, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000034015, 'Ka****', 'Gro**', 0, 'Y', '4042 ****** Pl', 'LAS VEGAS', 'NV', 'USA', '89183', NULL, '1972-02-13', 'LasVegas', 'Table', 675.67, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000042253, 'So***', 'Pas****', 0, 'Y', '2135 *********** St', 'LONG BEACH', 'CA', 'USA', '90805', 's**********@sbcglobal.net', '1980-12-08', 'California', 'Slot', 53.59, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000063979, 'Ch*****', 'Lov***', 0, 'N', '1366 *********** Dr', 'GLENDALE', 'AZ', 'UNK', 'UNKNOWN', '9*********@comcast.net', '1963-01-16', 'California', 'Table', 315.36, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000071902, 'Mi******', 'Sal****', 0, 'Y', '2738 ********** Ct', 'BLOOMINGTON', 'CA', 'USA', '92316', NULL, '1969-09-24', 'California', 'Slot', 3098.58, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000058454, 'Ge****', 'Akh*****', 0, 'Y', '872 ******** Rd', 'PORTERVILLE', 'CA', 'USA', '93257', NULL, '1949-03-08', 'California', 'Slot', 2627.79, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 101724818, 'An****', 'Abe******', 0, 'Y', '7842 *********** Dr', 'WINNEMUCCA', 'NV', 'USA', '89445', 'a*************@roadrunner.com', '1977-03-22', 'California', 'Slot', 130.13, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 200479013, 'An******', 'Har***', 0, 'Y', '8660 ******* St', 'SPARKS', 'NV', 'USA', '89431', 'a*****************@gmail.com', '1983-11-10', 'California', 'Table', 118.38, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000167050, 'Qi******', 'Deg*****', 0, 'Y', '836 ****** Dr', 'SAN GABRIEL', 'CA', 'USA', '91776', NULL, '1979-02-13', 'California', 'Table', 1704.28, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 10064084, 'Vi****', 'Osm**', 0, 'N', '7215 ************ St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'v**********@rocketmail.com', '1964-09-02', 'California', 'Both', 270.62, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10442200, 'Je*****', 'Lon**', 0, 'Y', '6690 *********** Ct', 'GLENDALE', 'AZ', 'USA', '85304', 'j**********@roadrunner.com', '1960-10-16', 'California', 'Slot', 593.36, 2004, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000112003, 'Pe', 'Che*', 0, 'Y', '9236 ***** Ave', 'ALHAMBRA', 'CA', 'USA', '91801', 'p***********@mac.com', '1969-09-28', 'California', 'Table', 54.15, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000126862, 'Sh*****', 'Ort***', 0, 'Y', '8189 *********** St', 'SAN GABRIEL', 'CA', 'USA', '91776', 'j*************@comcast.net', '1976-02-28', 'California', 'Table', 3992.61, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000186258, 'Mi******', 'Lee', 0, 'Y', '146 ****** Pl', 'LAKE HAVASU CITY', 'AZ', 'USA', '86406', 'm**************@gmail.com', '1988-06-26', 'California', 'Table', 1360.35, 2003, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000202617, 'Ra**', 'Har***', 0, 'Y', '8669 ********* Dr', 'LAS VEGAS', 'NV', 'USA', '89109', NULL, '1977-01-17', 'LasVegas', 'Slot', 121.53, 1006, 'ADT'),
    ('2023-03-14', '2023-06-01', 2000299934, 'Ar***', 'Gut******', 0, 'N', '7436 ****** Ave', 'GILBERT', 'AZ', 'UNK', 'UNKNOWN', NULL, '1985-01-23', 'California', 'Table', 338.53, 2005, 'ADT'),
    ('2023-03-14', '2023-06-01', 10049903, 'Be*****', 'Rab****', 0, 'Y', '8833 ***** St', 'LA MIRADA', 'CA', 'USA', '90638', 'b*************@att.net', '1986-04-04', 'California', 'Slot', 105.46, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10057775, 'We*', 'Mar*****', 0, 'Y', '6363 ******** Ave', 'LAS VEGAS', 'NV', 'USA', '89139', 'c***********@aim.com', '1979-04-28', 'LasVegas', 'Table', 284.43, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 10189419, 'Ad**', 'Wan*', 0, 'Y', '2778 ************ St', 'LAS VEGAS', 'NV', 'USA', '89113', NULL, '1994-10-10', 'LasVegas', 'Table', 518.38, 1003, 'ADT'),
    ('2023-03-14', '2023-06-01', 100487661, 'To**', 'Ort********', 0, 'Y', '9227 ********* Dr', 'LAS VEGAS', 'NV', 'USA', '89138', 't******@cox.net', '1975-06-19', 'LasVegas', 'Slot', 385.14, 1004, 'ADT'),
    ('2023-03-14', '2023-06-01', 200136245, 'My***', 'Abb***', 0, 'Y', '6399 ***** Ave', 'LOS ANGELES', 'CA', 'USA', '90034', 'm**********@aim.com', '1956-02-03', 'California', 'Slot', 6520.46, 2001, 'ADT'),
    ('2023-03-14', '2023-06-01', 10164298, 'Cy*****', 'Tha*', 0, 'N', '6133 ******* Rd', 'LA CRESCENTA', 'CA', 'UNK', 'UNKNOWN', 'c*******@mac.com', '1974-03-07', 'California', 'Slot', 36.51, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10204676, 'Ch******', 'Xie', 0, 'Y', '3712 ****** Ave', 'SAN DIEGO', 'CA', 'USA', '92123', 'c*******@aim.com', '1977-12-16', 'California', 'Slot', 139.94, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10254712, 'Ch**', 'Fro****', 0, 'N', '4471 ********** St', 'LOS ANGELES', 'CA', 'UNK', 'UNKNOWN', 'x*********@hotmail.net', '1987-03-02', 'California', 'Table', 39.17, 2008, 'ADT'),
    ('2023-03-14', '2023-06-01', 10312522, 'Le****', 'Eli****', 0, 'Y', '6119 ****** Dr', 'FONTANA', 'CA', 'USA', '92336', 'l********@att.net', '1952-01-14', 'California', 'Slot', 135.63, 2007, 'ADT'),
    ('2023-03-14', '2023-06-01', 10358003, 'Ma*****', 'Aqu***', 0, 'Y', '3543 ******* St', 'SUN CITY', 'AZ', 'USA', '85351', 'e**********@comcast.net', '1955-07-04', 'California', 'Slot', 3408.41, 2002, 'ADT'),
    ('2023-03-14', '2023-06-01', 10551821, 'An**', 'Chu**', 0, 'Y', '5638 *********** Ave', 'SANTA ANA', 'CA', 'USA', '92703', 'm*******@gmail.com', '1982-09-09', 'California', 'Table', 99.28, 2007, 'ADT')
;
```

```sql
with vMsgPivot as (
    select 
        OfferMonth, Tier, Geo,
        GamingSubhead, DiningSubhead, OutsideHeadline, GamingHeadline, DiningHeadline
    from (
      select 
        t.OfferMonth, t.Tier, m.Placement, m.Message, t.MinValue, t.MaxValue, t.Measure, t.Geo
      from 
        tblTiers t inner join tblMessages m on t.Tier >= m.TierMin and t.Tier <= m.TierMax
    ) as tblMsgTiers

    pivot (
      max(Message)
      for Placement in ([GamingSubhead], [DiningSubhead], [OutsideHeadline], [GamingHeadline], [DiningHeadline])
    ) as pvt
),

vTierOffersbyGame as (
    select 
        OfferMonth, Tier,
        value as GamePref,
        case when value = 'Slot' then SlotAmount else TableAmount end as GamingAmount,
        case when value = 'Slot' then 'SlotCredits' else TableType end as GamingType,
        GamingFrequency, DiningAmount, DiningFrequency, HotelCode, HotelFrequency
    from
        tblTierOffers
    cross apply 
        STRING_SPLIT('Slot;Table', ';')
),

vTieredOffersPlusHotel as (
  select
      t.OfferMonth, t.Tier, m.Geo, t.GamePref,
      replace(
        replace(m.OutsideHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
        '^^', format(t.GamingAmount * t.GamingFrequency + t.DiningAmount * t.DiningFrequency, 'C0')
      ) as OutsideHeadline,
      h.HotelHeadline,
      h.HotelDescription,
      t.HotelCode,

      replace(
        replace(
          replace(
            replace(m.GamingHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingHeadline,

      replace(
        replace(
          replace(
            replace(m.GamingSubhead, '##', FORMAT(t.OfferMonth, 'MMMM')),
            '**', t.GamingType
          ), '@@', format(t.GamingAmount * t.GamingFrequency, 'C0')
        ), '~~', format(t.GamingAmount, 'C0')
      ) as GamingSubhead,

      replace(
        replace(
          replace(m.DiningHeadline, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningHeadline,

      replace(
        replace(
          replace(m.DiningSubhead, '##', UPPER(FORMAT(t.OfferMonth, 'MMMM'))),
          '@@', format(t.DiningAmount * t.DiningFrequency, 'C0')
        ), '~~', format(t.DiningAmount, 'C0')
      ) as DiningSubhead


  from
      vTierOffersbyGame t 
        left join vMsgPivot m on t.OfferMonth = m.OfferMonth and t.Tier = m.Tier
        left join tblHotelOffers h on t.OfferMonth = h.OfferMonth and t.HotelCode = h.HotelCode
),


vCalendars as (
  select 
    OfferMonth, Geo,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [HotelCalendar1] end as HotelCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [HotelCalendar2] end as HotelCalendar2,
    case when Geo = 'California' then [CalGeoAllCalendar1] else [GamingCalendar1] end as GamingCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [GamingCalendar2] end as GamingCalendar2,
    [GamingCalendar3], [GamingCalendar4], 
    [GamingCalendar5], [GamingCalendar6],
    [GamingCalendar7], [GamingCalendar8], 
    [GamingCalendar9], [GamingCalendar10],
    case when Geo = 'California' then [CalGeoAllCalendar1] else [DiningCalendar1] end as DiningCalendar1,
    case when Geo = 'California' then [CalGeoAllCalendar2] else [DiningCalendar2] end as DiningCalendar2,
    [DiningCalendar3], [DiningCalendar4]
    
  from (
    select 
      OfferMonth, 
      concat(CalendarName, Period) as CalName,
      case when 
        datepart(m, StartDate) = datepart(m, EndDate) and datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + '-' + format(EndDate, 'd, yyyy')
      when 
        datepart(yy, StartDate) = datepart(yy, EndDate)
        then format(StartDate, 'MMM. d') + ' to ' + format(EndDate, 'MMM. d, yyyy')
      else
        format(StartDate, 'MMM. d, yyyy') + ' to ' + format(EndDate, 'MMM. d, yyyy') end 
      as DateRange,
      case when left(CalendarName, 3) = 'Cal' then 'California' else 'LasVegas' end as Geo
    from
      tblCalendars
  ) as cc
  pivot (
    max(DateRange)
    for CalName in (
      [HotelCalendar1], [HotelCalendar2],
      [GamingCalendar1], [GamingCalendar2], [GamingCalendar3], [GamingCalendar4], 
      [GamingCalendar5], [GamingCalendar6], [GamingCalendar7], [GamingCalendar8], 
      [GamingCalendar9], [GamingCalendar10], 
      [DiningCalendar1], [DiningCalendar2], [DiningCalendar3], [DiningCalendar4],
      [CalGeoAllCalendar1], [CalGeoAllCalendar2]
    )
  ) as pvt
  
),

vAllOfferData as (
  select 
    t.OfferMonth, t.Tier, t.GamePref,
    t.OutsideHeadline, t.HotelHeadline, t.HotelDescription, t.HotelCode, 
    t.GamingHeadline, t.GamingSubhead, 
    t.DiningHeadline, t.DiningSubhead, 
    coalesce(c.HotelCalendar1, '') as HotelCalendar1, coalesce(c.HotelCalendar2, '') as HotelCalendar2, 
    coalesce(c.GamingCalendar1, '') as GamingCalendar1, coalesce(c.GamingCalendar2, '') as GamingCalendar2, 
    coalesce(c.GamingCalendar3, '') as GamingCalendar3, coalesce(c.GamingCalendar4, '') as GamingCalendar4, 
    coalesce(c.GamingCalendar5, '') as GamingCalendar5, coalesce(c.GamingCalendar6, '') as GamingCalendar6, 
    coalesce(c.GamingCalendar7, '') as GamingCalendar7, coalesce(c.GamingCalendar8, '') as GamingCalendar8, 
    coalesce(c.GamingCalendar9, '') as GamingCalendar9, coalesce(c.GamingCalendar10, '') as GamingCalendar10, 
    coalesce(c.DiningCalendar1, '') as DiningCalendar1, coalesce(c.DiningCalendar2, '') as DiningCalendar2, 
    coalesce(c.DiningCalendar3, '') as DiningCalendar3, coalesce(c.DiningCalendar4, '') as DiningCalendar4

  from 
    vTieredOffersPlusHotel t
      left join vCalendars c on t.OfferMonth = c.OfferMonth and t.Geo = c.Geo
)

select *
from vAllOfferData
```